In [2]:
import pandas as pd
from bs4 import BeautifulSoup 
import requests
import time 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import re

# Recommendor

In [407]:
df = pd.read_csv('df_parsed_2.csv')
df.head(5)

,Unnamed: 0,recipe_urls,recipe_name,ingredients,prep_time,cook_time,serve,level_of_cooking,taste,ingredients_parsed
0,0,https://www.sanjeevkapoor.com/Recipe/-SABUDANA...,Sabudana Vada,"['Sago (sabudana) ,soaked 1 1/2 cups', 'Chutne...",360.0-420.0,11-15,4,Moderate,Mild,"['sago sabudana', 'chutney', 'coconut', 'coria..."
1,1,https://www.sanjeevkapoor.com/Recipe/PUNJABI-S...,Punjabi Samosa,"['Refined flour (maida) 1 1/2 cups', 'Potatoes...",31-40,41-50,4,Difficult,Tangy,"['refined flour maida', 'potato', 'carom seed ..."
2,2,https://www.sanjeevkapoor.com/Recipe/PANI-PURI...,Pani Puri,"['Puris as required', 'Potatoes , boiled and p...",31-40,0-5,4,Moderate,Tangy,"['puris', 'potato', 'moong sprout', 'roasted c..."
3,3,https://www.sanjeevkapoor.com/Recipe/MOONG-DAL...,Moong Dal Pakodi,['Split skinless green gram (dhuli moong dal) ...,198.0-240.0,6-10,4,Moderate,Mild,"['split skinless green dhuli moong dal', 'gree..."
4,4,https://www.sanjeevkapoor.com/Recipe/UPMA.html,Upma,"['Semolina (rawa/suji) 1 cup', 'Oil 2 tablespo...",6-10,16-20,4,Easy,Mild,"['semolina rawa suji', 'mustard seed', 'split ..."


In [408]:
def get_and_sort_corpus(data):
    corpus_sorted = []
    for doc in data.ingredients_parsed.values:
        doc_list = ast.literal_eval(doc)
        doc_list.sort()
        corpus_sorted.append(doc_list)
    return corpus_sorted

corpus = get_and_sort_corpus(df)
print(f"Length of corpus: {len(corpus)}")

Length of corpus: 10324


In [409]:
corpus

[['chutney',
  'coconut',
  'coriander leaf',
  'cumin seed',
  'cumin seed',
  'ginger green chilli paste',
  'green chilli',
  'potato',
  'roasted peanut',
  'roasted peanut',
  'sago sabudana',
  'sweetened yogurt',
  'vada',
  'yogurt'],
 ['carom seed ajwain',
  'coriander leaf',
  'coriander seed',
  'cumin seed',
  'date tamarind chutney',
  'dried mango powder',
  'dried pomegranate seed',
  'garam masala powder',
  'ghee',
  'ginger',
  'green chilli',
  'green chilli',
  'green chutney',
  'green pea',
  'potato',
  'refined flour maida'],
 ['boondi',
  'chaat masala',
  'chaat masala',
  'coriander leaf',
  'fried papdis',
  'green chilli',
  'imli chutney',
  'jaggery gur',
  'juice',
  'mint leaf',
  'moong sprout',
  'potato',
  'pudine ka pani',
  'puris',
  'ragda',
  'red chilli powder',
  'red chilli powder',
  'roasted cumin powder',
  'roasted cumin powder',
  'roasted cumin powder',
  'seedless date',
  'sev',
  'tamarind'],
 ['baking soda',
  'carom seed ajwain',


In [410]:
# calculate average length of each document 
lengths = [len(doc) for doc in corpus]
avg_len = float(sum(lengths)) / len(lengths)
avg_len

9.36371561410306

## Model 1: size = 100, 0.25

In [541]:
from gensim.models import Word2Vec
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 4 # number of CPUs
window = 9 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow1 = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=100, alpha = 0.25)

In [542]:
print(model_cbow1)

Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


In [543]:
#Summarize vocabulary
words = list(model_cbow1.wv.index_to_key)
words.sort()
# print(words)

In [509]:
# print(model_cbow.wv['chicken'])

In [544]:
model_cbow1.wv.most_similar('chicken')
# model_cbow.wv.similarity('cauliflower', 'cauliflower floret')

[('boneless chicken breast', 0.5832631587982178),
 ('boneless chicken', 0.5519208908081055),
 ('mutton', 0.5258955359458923),
 ('bottle gourd lauki', 0.5134784579277039),
 ('chicken bone', 0.5110452175140381),
 ('boneless mutton', 0.5101003050804138),
 ('curry powder', 0.5035622119903564),
 ('chicken boneless', 0.501754105091095),
 ('split green skin', 0.494008868932724),
 ('chicken skinned', 0.49216750264167786)]

In [545]:
model_cbow1.wv.most_similar('potato')
# model_cbow.wv.similarity('cauliflower', 'cauliflower floret')

[('red amaranth leaf lalmaat', 0.5695667266845703),
 ('soya granule warm water', 0.5483099222183228),
 ('pressed rice poh', 0.5374945402145386),
 ('millet water', 0.4810677766799927),
 ('pavs slit horizontally', 0.479654461145401),
 ('potato cubed', 0.4486950933933258),
 ('soya granule', 0.4385823905467987),
 ('raw shrimp', 0.4383868873119354),
 ('punjabi urad dal wadi', 0.42685335874557495),
 ('ripe guava', 0.4254465699195862)]

In [546]:
model_cbow1.save('model_cbow1.bin')

## Model 2: size = 50, 0.025

In [817]:
from gensim.models import Word2Vec
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 4 # number of CPUs
window = 9 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow2 = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=50)

In [818]:
print(model_cbow2)

Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


In [819]:
#Summarize vocabulary
words = list(model_cbow2.wv.index_to_key)
words.sort()
# print(words)

In [820]:
# print(model_cbow.wv['chicken'])

In [821]:
model_cbow2.wv.most_similar('potato')
# model_cbow.wv.similarity('cauliflower', 'cauliflower floret')

[('juice', 0.9904186725616455),
 ('onion', 0.9895870089530945),
 ('pavs', 0.9878970384597778),
 ('pav bhaji masala', 0.9867902398109436),
 ('green chilli', 0.9844996929168701),
 ('tomato pureed', 0.9839933514595032),
 ('fenugreek leaf leaf separated', 0.9808180332183838),
 ('tomato puree', 0.9793086647987366),
 ('ginger', 0.9780596494674683),
 ('tomato', 0.9779348969459534)]

In [822]:
model_cbow2.save('model_cbow2.bin')

## Model 3: size = 100, 0.025

In [698]:
from gensim.models import Word2Vec
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 4 # number of CPUs
window = 9 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow3 = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=100)

In [699]:
print(model_cbow3)

Word2Vec(vocab=7019, vector_size=100, alpha=0.025)


In [700]:
#Summarize vocabulary
words = list(model_cbow3.wv.index_to_key)
words.sort()
# print(words)

In [701]:
# print(model_cbow.wv['chicken'])

In [702]:
model_cbow3.wv.most_similar('chicken')
# model_cbow.wv.similarity('cauliflower', 'cauliflower floret')

[('cauliflower floret', 0.9911337494850159),
 ('chicken bone', 0.989937424659729),
 ('gravy', 0.9871789216995239),
 ('boneless mutton', 0.9871488809585571),
 ('cashewnut paste', 0.98399817943573),
 ('cauliflower separated small floret', 0.9818456768989563),
 ('coriander sprig', 0.9788594245910645),
 ('coconut milk', 0.9785735011100769),
 ('caraway seed shahi jeera', 0.9745041131973267),
 ('coriander seed', 0.973697304725647)]

In [703]:
model_cbow3.save('model_cbow3.bin')

## Model 4: size = 100, 0.05

In [723]:
from gensim.models import Word2Vec
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 4 # number of CPUs
window = 9 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow4 = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=100, alpha=0.05)

In [724]:
print(model_cbow4)

Word2Vec(vocab=7019, vector_size=100, alpha=0.05)


In [725]:
#Summarize vocabulary
words = list(model_cbow4.wv.index_to_key)
words.sort()
# print(words)

In [726]:
# print(model_cbow.wv['chicken'])

In [727]:
model_cbow4.wv.most_similar('chicken')
# model_cbow.wv.similarity('cauliflower', 'cauliflower floret')

[('chicken bone', 0.9836676120758057),
 ('bay leaf', 0.9633663892745972),
 ('cauliflower small floret', 0.9498786926269531),
 ('chicken curry', 0.9487498998641968),
 ('basmati rice', 0.9397714734077454),
 ('cauliflower floret', 0.9382387399673462),
 ('caraway seed shahi jeera', 0.936244785785675),
 ('clove', 0.9307880997657776),
 ('brown pressed rice', 0.9295415282249451),
 ('cinnamon', 0.9251725077629089)]

In [728]:
model_cbow4.save('model_cbow4.bin')

## Model 5: size = 100, 0.1

In [729]:
from gensim.models import Word2Vec
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 4 # number of CPUs
window = 9 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow5 = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=100, alpha=0.1)

In [730]:
print(model_cbow5)

Word2Vec(vocab=7019, vector_size=100, alpha=0.1)


In [731]:
#Summarize vocabulary
words = list(model_cbow5.wv.index_to_key)
words.sort()
# print(words)

In [732]:
# print(model_cbow.wv['chicken'])

In [733]:
model_cbow5.wv.most_similar('chicken')
# model_cbow.wv.similarity('cauliflower', 'cauliflower floret')

[('chicken bone', 0.9410198926925659),
 ('boneless mutton', 0.8835244178771973),
 ('fenugreek leaf kasuri methi powder', 0.8598482608795166),
 ('cinnamon broken', 0.8486147522926331),
 ('clam opened flesh sifted', 0.804892361164093),
 ('black cardamom', 0.7973526120185852),
 ('cashewnut paste', 0.797248899936676),
 ('bay leaf', 0.7959133982658386),
 ('coriander seed roasted', 0.7810258865356445),
 ('biryani masala', 0.7798755764961243)]

In [734]:
model_cbow5.save('model_cbow5.bin')

## Model 6: size = 100, 0.15

In [748]:
from gensim.models import Word2Vec
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 4 # number of CPUs
window = 9 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow6 = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=100, alpha=0.15)

In [749]:
print(model_cbow6)

Word2Vec(vocab=7019, vector_size=100, alpha=0.15)


In [750]:
#Summarize vocabulary
words = list(model_cbow6.wv.index_to_key)
words.sort()
# print(words)

In [751]:
# print(model_cbow.wv['chicken'])

In [752]:
model_cbow6.wv.most_similar('chicken')
# model_cbow.wv.similarity('cauliflower', 'cauliflower floret')

[('chicken bone', 0.7575456500053406),
 ('boneless chicken', 0.6761292815208435),
 ('black stock', 0.6612759828567505),
 ('chicken leg', 0.6284501552581787),
 ('green capsicum thin s', 0.6263337135314941),
 ('chicken mince', 0.6258883476257324),
 ('boneless chicken leg', 0.6258543133735657),
 ('chatpata spice powder', 0.6251262426376343),
 ('garlic tablespoon', 0.624141275882721),
 ('boneless mutton', 0.6200828552246094)]

In [753]:
model_cbow6.save('model_cbow6.bin')

## Model 7: size = 100, 0.2

In [757]:
from gensim.models import Word2Vec
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 4 # number of CPUs
window = 9 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow7 = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=100, alpha=0.2)

In [758]:
print(model_cbow7)

Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


In [759]:
#Summarize vocabulary
words = list(model_cbow7.wv.index_to_key)
words.sort()
# print(words)

In [751]:
# print(model_cbow.wv['chicken'])

In [760]:
model_cbow7.wv.most_similar('chicken')
# model_cbow.wv.similarity('cauliflower', 'cauliflower floret')

[('chicken bone', 0.6747059226036072),
 ('boneless chicken', 0.6136428117752075),
 ('boneless chicken breast', 0.5642450451850891),
 ('coriander leave', 0.5463516116142273),
 ('chicken mince keema', 0.5405824184417725),
 ('king fish', 0.5310322642326355),
 ('coriander powder roasted', 0.5240634083747864),
 ('dried plum', 0.523983895778656),
 ('boneless mutton', 0.522004246711731),
 ('chicken masala', 0.5202990174293518)]

In [761]:
model_cbow7.save('model_cbow7.bin')

## Model 8: size = 50, 0.15

In [773]:
from gensim.models import Word2Vec
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 4 # number of CPUs
window = 9 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow8 = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=50, alpha=0.15)

In [774]:
print(model_cbow8)

Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


In [775]:
#Summarize vocabulary
words = list(model_cbow8.wv.index_to_key)
words.sort()
# print(words)

In [649]:
# print(model_cbow.wv['chicken'])

In [776]:
model_cbow8.wv.most_similar('potato')
# model_cbow.wv.similarity('cauliflower', 'cauliflower floret')

[('red amaranth leaf lalmaat', 0.6382747292518616),
 ('khatta leaf', 0.626788318157196),
 ('ridge gourd', 0.6204877495765686),
 ('mixed vegetable cabbage', 0.6176308393478394),
 ('red kidney bean rajma cooked', 0.5783786773681641),
 ('readymade samosa dough', 0.5699367523193359),
 ('cauliflower floret', 0.5612820386886597),
 ('potato curry', 0.5612260103225708),
 ('red pumpkin bhopla kaddu', 0.5596011877059937),
 ('capsicum small', 0.5590975284576416)]

In [777]:
model_cbow8.save('model_cbow8.bin')

## Model 9: size = 50, 0.2

In [783]:
from gensim.models import Word2Vec
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 4 # number of CPUs
window = 9 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow9 = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=50, alpha=0.2)

In [784]:
print(model_cbow9)

Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


In [785]:
#Summarize vocabulary
words = list(model_cbow9.wv.index_to_key)
words.sort()
# print(words)

In [786]:
# print(model_cbow.wv['chicken'])

In [787]:
model_cbow9.wv.most_similar('potato')
# model_cbow.wv.similarity('cauliflower', 'cauliflower floret')

[('red amaranth leaf lalmaat', 0.5899707078933716),
 ('iranian pistachio', 0.5424157381057739),
 ('mutton mince kee', 0.5310397148132324),
 ('prawn approx', 0.5292549133300781),
 ('low fat coconut milk', 0.49453139305114746),
 ('fenugreek methi', 0.49451014399528503),
 ('tata sampann dal tadka masala sachet', 0.49191752076148987),
 ('samosa patti', 0.4834938645362854),
 ('caramel sauce:', 0.47800543904304504),
 ('fusilli pasta', 0.47589826583862305)]

In [788]:
model_cbow9.save('model_cbow9.bin')

## Model 10: size = 50, 0.25

In [789]:
from gensim.models import Word2Vec
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 4 # number of CPUs
window = 9 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow10 = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=50, alpha=0.25)

In [790]:
print(model_cbow10)

Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


In [791]:
#Summarize vocabulary
words = list(model_cbow10.wv.index_to_key)
words.sort()
# print(words)

In [649]:
# print(model_cbow.wv['chicken'])

In [792]:
model_cbow10.wv.most_similar('potato')
# model_cbow.wv.similarity('cauliflower', 'cauliflower floret')

[('split skinless green chilkewali moong dal', 0.5329377055168152),
 ('french bean diamond', 0.5249711871147156),
 ('punjabi garam masala powder', 0.5185133814811707),
 ('lapsi dalia', 0.5037508606910706),
 ('simple indian masalas', 0.4912166893482208),
 ('roasted peanut powdered', 0.47582101821899414),
 ('papad', 0.47086358070373535),
 ('potato cubed', 0.4705715775489807),
 ('recheado masala', 0.469439834356308),
 ('split green', 0.457701176404953)]

In [793]:
model_cbow10.save('model_cbow10.bin')

## Model 11: size = 150, 0.15

In [794]:
from gensim.models import Word2Vec
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 4 # number of CPUs
window = 9 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow11 = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=150, alpha=0.15)

In [795]:
print(model_cbow11)

Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


In [796]:
#Summarize vocabulary
words = list(model_cbow11.wv.index_to_key)
words.sort()
# print(words)

In [797]:
# print(model_cbow.wv['chicken'])

In [798]:
model_cbow11.wv.most_similar('potato')
# model_cbow.wv.similarity('cauliflower', 'cauliflower floret')

[('ready made samosa dough roundel', 0.6052340269088745),
 ('pav', 0.5932689905166626),
 ('roasted vermicelli sevaiyan', 0.5890089273452759),
 ('urad dal wadi', 0.588714063167572),
 ('pressed rice poh', 0.5741035342216492),
 ('lalmaat', 0.5720818042755127),
 ('soya nugget', 0.5641121864318848),
 ('potato cubed', 0.5580288171768188),
 ('bengal chana dal', 0.5574693083763123),
 ('mixed dal masoor', 0.5533422827720642)]

In [799]:
model_cbow11.save('model_cbow11.bin')

## Model 12: size = 150, 0.2

In [800]:
from gensim.models import Word2Vec
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 4 # number of CPUs
window = 9 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow12 = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=150, alpha=0.2)

In [801]:
print(model_cbow12)

Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


In [802]:
#Summarize vocabulary
words = list(model_cbow12.wv.index_to_key)
words.sort()
# print(words)

In [649]:
# print(model_cbow.wv['chicken'])

In [803]:
model_cbow12.wv.most_similar('potato')
# model_cbow.wv.similarity('cauliflower', 'cauliflower floret')

[('whole red lentil sabut masoor', 0.5782532095909119),
 ('potato curry', 0.5256633758544922),
 ('refined flour maida dough', 0.519300103187561),
 ('potato straw', 0.510066032409668),
 ('red bell pepper curl', 0.5098786950111389),
 ('samosa patti', 0.5098337531089783),
 ('basmati rice tukda', 0.5039233565330505),
 ('potato cubed', 0.5019713640213013),
 ('turmeric paste', 0.4991665482521057),
 ('tamarin paste', 0.4989997446537018)]

In [804]:
model_cbow12.save('model_cbow12.bin')

## Model 13: size = 150, 0.25

In [805]:
from gensim.models import Word2Vec
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 4 # number of CPUs
window = 9 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow13 = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=150, alpha=0.25)

In [806]:
print(model_cbow13)

Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


In [807]:
#Summarize vocabulary
words = list(model_cbow13.wv.index_to_key)
words.sort()
# print(words)

In [649]:
# print(model_cbow.wv['chicken'])

In [808]:
model_cbow13.wv.most_similar('potato')
# model_cbow.wv.similarity('cauliflower', 'cauliflower floret')

[('turnip', 0.48113125562667847),
 ('split skinless pigeon pea', 0.44859015941619873),
 ('rice rice', 0.4434053301811218),
 ('carrot peel', 0.41983693838119507),
 ('fenugreek seed methi dana roasted', 0.418783962726593),
 ('dry breadcrumb', 0.41447797417640686),
 ('split bengal chana dal', 0.4142076075077057),
 ('tomato ripe', 0.4140036106109619),
 ('raw mango kachchi kairi', 0.4135831594467163),
 ('whole black sabut urad', 0.4128839075565338)]

In [809]:
model_cbow13.save('model_cbow13.bin')

In [301]:
# class MeanEmbeddingVectorizer(object):

#     def __init__(self, word_model):
#         self.word_model = word_model
#         self.vector_size = word_model.wv.vector_size

#     def fit(self):  # comply with scikit-learn transformer requirement
#         return self

#     def transform(self, docs):  # comply with scikit-learn transformer requirement
#         doc_word_vector = self.word_average_list(docs)
#         return doc_word_vector

#     def word_average(self, sent):
#         """
#         Compute average word vector for a single doc/sentence.
#         :param sent: list of sentence tokens
#         :return:
#             mean: float of averaging word vectors
#         """
#         mean = []
#         for word in sent:
#             if word in self.word_model.wv.index_to_key:
#                 mean.append(self.word_model.wv.get_vector(word))

#         if not mean:  # empty words
#             # If a text is empty, return a vector of zeros.
#             logging.warning("cannot compute average owing to no vector for {}".format(sent))
#             return np.zeros(self.vector_size)
#         else:
#             mean = np.array(mean).mean(axis=0)
#             return mean


#     def word_average_list(self, docs):
#         """
#         Compute average word vector for multiple docs, where docs had been tokenized.
#         :param docs: list of sentence in list of separated tokens
#         :return:
#             array of average word vector in shape (len(docs),)
#         """
#         return np.vstack([self.word_average(sent) for sent in docs])

In [302]:
# # encode document by averaging word embeddings

# # load model 
# loaded_model = Word2Vec.load('model_cbow.bin')
# if loaded_model:
#     print("Successfully loaded model")

# mean_vec_tr = MeanEmbeddingVectorizer(loaded_model)
# doc_vec = mean_vec_tr.transform(corpus)

Successfully loaded model


In [1013]:
import os
import sys
import logging
import unidecode
import ast

import numpy as np
import pandas as pd

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# import config
# from ingredient_parser import ingredient_parser

In [1014]:
def get_and_sort_corpus(data):
    """
    Get corpus with the documents sorted in alphabetical order
    """
    corpus_sorted = []
    for doc in data.parsed.values:
        if isinstance(doc, list):
            doc_list = doc
        else:
            doc_list = ast.literal_eval(doc)
#         doc_list = ast.literal_eval(doc)
        doc_list.sort()
        corpus_sorted.append(doc_list)
    return corpus_sorted

In [1015]:
def get_recommendations(N, scores):
    """
    Top-N recomendations order by score
    """
    # load in recipe dataset
    df_recipes = pd.read_csv('df_parsed_2.csv')
    # order the scores with and filter to get the highest N scores
    top = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:N]
    # create dataframe to load in recommendations
    recommendation = pd.DataFrame(columns=["recipe", "ingredients", "score", "url"])
    count = 0
    for i in top:
        recommendation.at[count, "recipe"] = title_parser(df_recipes["recipe_name"][i])
        recommendation.at[count, "ingredients"] = ingredient_parser_final(
            df_recipes["ingredients"][i]
        )
        recommendation.at[count, "url"] = df_recipes["recipe_urls"][i]
        recommendation.at[count, "preperation_time"] = df_recipes["prep_time"][i]
        recommendation.at[count, "cook_time"] = df_recipes["cook_time"][i]
        recommendation.at[count, "serves"] = df_recipes["serve"][i]
        recommendation.at[count, "level_of_cooking"] = df_recipes["level_of_cooking"][i]
        recommendation.at[count, "taste"] = df_recipes["taste"][i]
        recommendation.at[count, "score"] = f"{scores[i]}"
        count += 1
    return recommendation

In [1016]:
def title_parser(title):
    title = unidecode.unidecode(title)
    return title

In [1017]:
def ingredient_parser_final(ingredient):
    """
    neaten the ingredients being outputted
    """
    if isinstance(ingredient, list):
        ingredients = ingredient
    else:
        ingredients = ast.literal_eval(ingredient)

    ingredients = ",".join(ingredients)
    ingredients = unidecode.unidecode(ingredients)
    return ingredients

In [1018]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word_model):
        self.word_model = word_model
        self.vector_size = word_model.wv.vector_size

    def fit(self):  # comply with scikit-learn transformer requirement
        return self

    def transform(self, docs):  # comply with scikit-learn transformer requirement
        doc_word_vector = self.word_average_list(docs)
        return doc_word_vector

    def word_average(self, sent):
        """
        Compute average word vector for a single doc/sentence.
        :param sent: list of sentence tokens
        :return:
            mean: float of averaging word vectors
        """
        mean = []
        for word in sent:
            if word in self.word_model.wv.index_to_key:
                mean.append(self.word_model.wv.get_vector(word))

        if not mean:  # empty words
            # If a text is empty, return a vector of zeros.
            # logging.warning(
            #     "cannot compute average owing to no vector for {}".format(sent)
            # )
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean

    def word_average_list(self, docs):
        """
        Compute average word vector for multiple docs, where docs had been tokenized.
        :param docs: list of sentence in list of separated tokens
        :return:
            array of average word vector in shape (len(docs),)
        """
        return np.vstack([self.word_average(sent) for sent in docs])

In [1019]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word_model):

        self.word_model = word_model
        self.word_idf_weight = None
        self.vector_size = word_model.wv.vector_size

    def fit(self, docs):  # comply with scikit-learn transformer requirement
        """
        Fit in a list of docs, which had been preprocessed and tokenized,
        such as word bi-grammed, stop-words removed, lemmatized, part of speech filtered.
        Then build up a tfidf model to compute each word's idf as its weight.
        Noted that tf weight is already involved when constructing average word vectors, and thus omitted.
        :param
            pre_processed_docs: list of docs, which are tokenized
        :return:
            self
        """

        text_docs = []
        for doc in docs:
            text_docs.append(" ".join(doc))

        tfidf = TfidfVectorizer()
        tfidf.fit(text_docs)  # must be list of text string

        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of
        # known idf's
        max_idf = max(tfidf.idf_)  # used as default value for defaultdict
        self.word_idf_weight = defaultdict(
            lambda: max_idf,
            [(word, tfidf.idf_[i]) for word, i in tfidf.vocabulary_.items()],
        )
        return self

    def transform(self, docs):  # comply with scikit-learn transformer requirement
        doc_word_vector = self.word_average_list(docs)
        return doc_word_vector

    def word_average(self, sent):
        """
        Compute average word vector for a single doc/sentence.
        :param sent: list of sentence tokens
        :return:
            mean: float of averaging word vectors
        """

        mean = []
        for word in sent:
            if word in self.word_model.wv.index_to_key:
                mean.append(
                    self.word_model.wv.get_vector(word) * self.word_idf_weight[word]
                )  # idf weighted

        if not mean:  # empty words
            # If a text is empty, return a vector of zeros.
            # logging.warning(
            #     "cannot compute average owing to no vector for {}".format(sent)
            # )
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean

    def word_average_list(self, docs):
        """
        Compute average word vector for multiple docs, where docs had been tokenized.
        :param docs: list of sentence in list of separated tokens
        :return:
            array of average word vector in shape (len(docs),)
        """
        return np.vstack([self.word_average(sent) for sent in docs])

In [1020]:
def get_recs(ingredients, N=5, mean=True):
    # load in word2vec model
    model = Word2Vec.load("model_cbow7.bin")
    model.init_sims(replace=True)
    if model:
        print("Successfully loaded model")
        print(model)
    # load in data
    data = pd.read_csv("df_parsed_2.csv")
    # parse ingredients
    data["parsed"] = data.ingredients.apply(ingredient_cleaner)
    # create corpus
    corpus = get_and_sort_corpus(data)

    if mean:
        # get average embdeddings for each document
        mean_vec_tr = MeanEmbeddingVectorizer(model)
        doc_vec = mean_vec_tr.transform(corpus)
        doc_vec = [doc.reshape(1, -1) for doc in doc_vec]
        assert len(doc_vec) == len(corpus)
    else:
        # use TF-IDF as weights for each word embedding
        tfidf_vec_tr = TfidfEmbeddingVectorizer(model)
        tfidf_vec_tr.fit(corpus)
        doc_vec = tfidf_vec_tr.transform(corpus)
        doc_vec = [doc.reshape(1, -1) for doc in doc_vec]
        assert len(doc_vec) == len(corpus)

    # create embessing for input text
    input = ingredients
    # create tokens with elements
    input = input.split(",")
    # parse ingredient list
    input = ingredient_cleaner(input)
    # get embeddings for ingredient doc
    if mean:
        input_embedding = mean_vec_tr.transform([input])[0].reshape(1, -1)
    else:
        input_embedding = tfidf_vec_tr.transform([input])[0].reshape(1, -1)

    # get cosine similarity between input embedding and all the document embeddings
    cos_sim = map(lambda x: cosine_similarity(input_embedding, x)[0][0], doc_vec)
    scores = list(cos_sim)
    # Filter top N recommendations
    recommendations = get_recommendations(N, scores)
    return recommendations

In [502]:
# if __name__ == "__main__":
#     input = "chicken"
#     rec = get_recs(input)
#     print(rec)
#     rec.head(5)

# Final Model Selection

In [1002]:
# Model 1: True Means + 0.25 + 100
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Apricot Chicken,"Chicken 1 cup,Dried apricots 500 grams,Butter ...",0.5371984909965504,https://www.sanjeevkapoor.com/Recipe/Apricot-C...
1,Pollo A La Brasa,"Chicken cut into 4 pieces, 1 whole,Soy sauce 1...",0.5161871864060724,https://www.sanjeevkapoor.com/Recipe/Pollo-a-L...
2,Malaysian Chicken,"Chicken legs with bone 2,Oyster sauce 1 1/2 t...",0.49434011410935985,https://www.sanjeevkapoor.com/Recipe/Malaysian...
3,Coorg Style Dry Chicken,Chicken cut into 8 pieces on the bone 750 gram...,0.4873498939970887,https://www.sanjeevkapoor.com/Recipe/Coorg-Sty...
4,Kokur Yakhni,"Chicken 750 grams,Fennel seed (saunf) powder 1...",0.45179051545463295,https://www.sanjeevkapoor.com/Recipe/Kokur-Yak...


In [1021]:
# Model 7: True Means + 0.2 + 100
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url,preperation_time,cook_time,serves,level_of_cooking,taste
0,Country Captain Chicken,"Chicken cut into pieces 1 kilogram,Oil 1 table...",0.5369949339739375,https://www.sanjeevkapoor.com/Recipe/Country-C...,11-15,21-25,4,Moderate,Mild
1,Coorg Style Dry Chicken,Chicken cut into 8 pieces on the bone 750 gram...,0.5296900021167451,https://www.sanjeevkapoor.com/Recipe/Coorg-Sty...,6-10,26-30,4,Easy,Spicy
2,Chicken Stew- Sk Khazana,Boneless chicken breasts cut into 1/2 inch cub...,0.5173323377048464,https://www.sanjeevkapoor.com/Recipe/Chicken-S...,6-10,21-25,4,Easy,Mild
3,Pollo A La Brasa,"Chicken cut into 4 pieces, 1 whole,Soy sauce 1...",0.5121328081191735,https://www.sanjeevkapoor.com/Recipe/Pollo-a-L...,900.0-1200.0,16-20,4,Moderate,Mild
4,Chicken Tagine With Chickpeas And Mint,Chickpeas (kabuli chana) soaked and parboiled ...,0.5070411692371333,https://www.sanjeevkapoor.com/Recipe/Chicken-T...,420.0-480.0,26-30,4,Moderate,Mild


In [1004]:
# Model 1: True Means + 0.25 + 100
input = "pumpkin, potato"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Potato Wafers,"Potatoes peeled 4 medium,Oil to deep fry,Salt ...",0.8145865262137119,https://www.sanjeevkapoor.com/Recipe/Potato-Wa...
1,Ultimate Potato Puree,"Potatoes 1 inch cubes 4 large,Fresh cream 1/2 ...",0.6774905641975875,https://www.sanjeevkapoor.com/Recipe/Ultimate-...
2,Mashed Potatoes-Sk Khazana,"Medium potatoes peeled 4-5,Salt to taste,Milk ...",0.6681669560732919,https://www.sanjeevkapoor.com/Recipe/Mashed-Po...
3,Cheese Potato Chips,"Cheese balls 4 cups,Potatoes 5 medium,Olive oi...",0.6523754115206936,https://www.sanjeevkapoor.com/Recipe/Cheese-Po...
4,Tomato And Basil Pulao,"Tomato 4 tablespoons,Pasta sauce,Oil 1 tablesp...",0.6223443984721946,https://www.sanjeevkapoor.com/Recipe/Tomato-An...


In [1022]:
# Model 7: True Means + 0.2 + 100
input = "pumpkin, potato"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url,preperation_time,cook_time,serves,level_of_cooking,taste
0,Potato Wafers,"Potatoes peeled 4 medium,Oil to deep fry,Salt ...",0.825571541576656,https://www.sanjeevkapoor.com/Recipe/Potato-Wa...,0-5,11-15,4,Easy,Mild
1,Lamb Boulangere,"Mutton Leg 1 kilogram,Potatoes 2 tablespoons,G...",0.7662949606524574,https://www.sanjeevkapoor.com/Recipe/Lamb-Boul...,16-20,26-30,,Moderate,Mild
2,Pumpkin And Corn Chowder,Yellow pumpkin peeled and cut into small cubes...,0.702472875214353,https://www.sanjeevkapoor.com/Recipe/Pumpkin-a...,6-10,16-20,4,Easy,Mild
3,Potato And Rosemary Focaccia,Potatoes peeled cut into small cubes and deep-...,0.6901523410779941,https://www.sanjeevkapoor.com/Recipe/Potato-an...,51-60,16-20,4,Moderate,Sweet
4,Spinach Gnocchi In Cheese Garlic Sauce,"Spinach puree 2 tablespoons,Potatoes 250 grams...",0.6888262022147668,https://www.sanjeevkapoor.com/Recipe/Spinach-G...,16-20,26-30,4,Easy,Spicy


In [1009]:
# Model 1: True Means + 0.25 + 100
input = "chickpea, onion, tomato"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Fried Onions For Biryani,"Onions 3 medium,Oil to deep fry",0.7507012311457452,https://www.sanjeevkapoor.com/Recipe/Fried-Oni...
1,Stuffed Karela,"Bitter Gourd 600 grams,Onions chopped 3/4 cup,...",0.7454876370576909,https://www.sanjeevkapoor.com/Recipe/Stuffed-K...
2,Bhajnee Thalipeeth,"Bhajnee flour 3 cups,Onions,Salt to taste,Turm...",0.7315693301080745,https://www.sanjeevkapoor.com/Recipe/Bhajnee-T...
3,Chickpea And Potato Burger,"Chickpeas soaked, boiled and mashed 1 cup,Pota...",0.7276674173387259,https://www.sanjeevkapoor.com/Recipe/Chickpea-...
4,Lebanese Kibbeh,"Lamb mince 300 grams,Burghul (bulgur or burgul...",0.725046839258009,https://www.sanjeevkapoor.com/Recipe/Lebanese-...


In [1023]:
# Model 7: True Means + 0.2 + 100
input = "chickpea, onion, tomato"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url,preperation_time,cook_time,serves,level_of_cooking,taste
0,Red Kidney Bean And Coriander Burgers,Red kidney beans soaked overnight and boiled 2...,0.7858444260302048,https://www.sanjeevkapoor.com/Recipe/Red-Kidne...,6-10,16-20,4,Moderate,Mild
1,Falafel Balls,"Chickpeas , soaked overnight and drained 1 cup...",0.7840371304441172,https://www.sanjeevkapoor.com/Recipe/Falafel-B...,480.0-600.0,11-15,4,Easy,Mild
2,Spicy Sprouts Sandwich,"Mixed sprouts boiled 1 cup,Brown bread slices ...",0.7591985633579306,https://www.sanjeevkapoor.com/Recipe/Spicy-Spr...,26-30,26-30,4,Easy,Spicy
3,Chickpea And Potato Burger,"Chickpeas soaked, boiled and mashed 1 cup,Pota...",0.756628354934491,https://www.sanjeevkapoor.com/Recipe/Chickpea-...,31-40,26-30,4,Moderate,Spicy
4,Fried Onions For Biryani,"Onions 3 medium,Oil to deep fry",0.7489572086597559,https://www.sanjeevkapoor.com/Recipe/Fried-Oni...,6-10,16-20,,Easy,Mild


In [1010]:
# Model 1: True Means + 0.25 + 100
input = "babycorn, spring onion, soy sauce"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Hakka Noodles - Sk Khazana,"Noodles 200 grams,Oil 2 tablespoons,Medium car...",0.7954689108328319,https://www.sanjeevkapoor.com/Recipe/Hakka-Noo...
1,Veg Chowmein,"Noodles , boiled 2 cups,Button mushrooms , qua...",0.794291594527605,https://www.sanjeevkapoor.com/Recipe/Veg-Chowm...
2,Chicken Pad Thai,Boneless chicken cut into thin strips 100 gram...,0.7923148068015394,https://www.sanjeevkapoor.com/Recipe/Chicken-P...
3,Stir Fried Broccoli And Tofu With Cracked Pepper,"Broccoli 1 small florets,Tofu 1/2 inch pieces ...",0.7790586622300928,https://www.sanjeevkapoor.com/Recipe/Stir-Frie...
4,Oodles Of Noodles,"Noodles 200 grams,Sweetcorn Kernels 1/2 cup,...",0.7762370588984114,https://www.sanjeevkapoor.com/Recipe/Oodles-Of...


In [1024]:
# Model 7: True Means + 0.2 + 100
input = "babycorn, spring onion, soy sauce"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url,preperation_time,cook_time,serves,level_of_cooking,taste
0,Oodles Of Noodles,"Noodles 200 grams,Sweetcorn Kernels 1/2 cup,...",0.8723170281234729,https://www.sanjeevkapoor.com/Recipe/Oodles-Of...,11-15,16-20,4,Easy,Mild
1,Veg Chowmein,"Noodles , boiled 2 cups,Button mushrooms , qua...",0.8701859851711385,https://www.sanjeevkapoor.com/Recipe/Veg-Chowm...,21-25,11-15,4,Easy,Mild
2,Crispy Wontons,"Wonton wrappers 24,Oil 2 tbsps + to deep fry,G...",0.8602570382887704,https://www.sanjeevkapoor.com/Recipe/Crispy-Wo...,31-40,11-15,4,Moderate,Mild
3,Oodles Of Noodles,"Noodles boiled 200 grams,Oil 1 tablespoon,Spri...",0.8529082013634977,https://www.sanjeevkapoor.com/Recipe/Oodles-of...,11-15,6-10,4,Easy,Mild
4,Hakka Noodles - Sk Khazana,"Noodles 200 grams,Oil 2 tablespoons,Medium car...",0.8525205106615458,https://www.sanjeevkapoor.com/Recipe/Hakka-Noo...,0-5,26-30,4,Easy,Mild


### Chicken

In [625]:
# Model 1 + False tests
input = "chicken"
rec = get_recs(input)
rec

### 1 does NOT have chicken

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2863015847.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.49665223866438624,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
1,Malaysian Chicken,"Chicken legs with bone 2,Oyster sauce 1 1/2 t...",0.49434010548391194,https://www.sanjeevkapoor.com/Recipe/Malaysian...
2,Masoor Dal,"Split Red Lentil soaked in water 2 cups,Curry ...",0.47338747253336616,https://www.sanjeevkapoor.com/Recipe/Masoor-Da...
3,Apricot Chicken On Skewers,"Boneless chicken leg, cut into 1 inch pieces 1...",0.4704421144504205,https://www.sanjeevkapoor.com/Recipe/Apricot-C...
4,Chilli Chicken Tomato Pulao,"Chicken boneless 1 inch cubes 250 grams,Basmat...",0.46886252526807504,https://www.sanjeevkapoor.com/Recipe/Chilli-Ch...


In [636]:
# Model 1 + True tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Apricot Chicken,"Chicken 1 cup,Dried apricots 500 grams,Butter ...",0.5371984909965504,https://www.sanjeevkapoor.com/Recipe/Apricot-C...
1,Pollo A La Brasa,"Chicken cut into 4 pieces, 1 whole,Soy sauce 1...",0.5161871864060724,https://www.sanjeevkapoor.com/Recipe/Pollo-a-L...
2,Malaysian Chicken,"Chicken legs with bone 2,Oyster sauce 1 1/2 t...",0.49434011410935985,https://www.sanjeevkapoor.com/Recipe/Malaysian...
3,Coorg Style Dry Chicken,Chicken cut into 8 pieces on the bone 750 gram...,0.4873498939970887,https://www.sanjeevkapoor.com/Recipe/Coorg-Sty...
4,Kokur Yakhni,"Chicken 750 grams,Fennel seed (saunf) powder 1...",0.45179051545463295,https://www.sanjeevkapoor.com/Recipe/Kokur-Yak...


In [705]:
# Model 3(1) + False tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3921877797.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.025)


,recipe,ingredients,score,url
0,Tomato Sabzi Pulao,"Basmati Rice 3/4 cup,Tomatoes 2,French beans 8...",0.9980031762486433,https://www.sanjeevkapoor.com/Recipe/Tomato-Sa...
1,Vegetable Pulao,"Basmati rice boiled 2 cups,Cauliflower separat...",0.9976381406207055,https://www.sanjeevkapoor.com/Recipe/Vegetable...
2,Sabzi Aur Tamatar Ka Pulao,"Basmati rice 1 1/2 cups,Tomatoes chopped 3-4,G...",0.9975750139661882,https://www.sanjeevkapoor.com/Recipe/Sabzi-Aur...
3,Barley And Peanut Soup,"Barley 1 cup,Peanuts 1/2 cup,Oil 2 tablespoons...",0.9975527357220831,https://www.sanjeevkapoor.com/Recipe/Barley-An...
4,Simple Vegetable Pulao,"Cauliflower florets 8-10,French beans stringed...",0.9970122101352196,https://www.sanjeevkapoor.com/Recipe/Simple-Ve...


In [712]:
# Model 3(1) + True tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2415953633.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.025)


,recipe,ingredients,score,url
0,Tomato Sabzi Pulao,"Basmati Rice 3/4 cup,Tomatoes 2,French beans 8...",0.9970375917508905,https://www.sanjeevkapoor.com/Recipe/Tomato-Sa...
1,Vegetable Pulao,"Basmati rice boiled 2 cups,Cauliflower separat...",0.9966917115518217,https://www.sanjeevkapoor.com/Recipe/Vegetable...
2,Simple Vegetable Pulao,"Cauliflower florets 8-10,French beans stringed...",0.9966201838088008,https://www.sanjeevkapoor.com/Recipe/Simple-Ve...
3,Dahiwale Babycorn And Mushroom,"Babycorns cut into 1 inch pieces 12-15,Button ...",0.9965456419250028,https://www.sanjeevkapoor.com/Recipe/Dahiwale-...
4,Beetroot And Mustard Pulao,"Basmati rice soaked 2 cups,Beetroots boiled 3-...",0.9963253923338378,https://www.sanjeevkapoor.com/Recipe/Beetroot-...


In [714]:
# Model 4(1) + False tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1733032538.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.05)


,recipe,ingredients,score,url
0,Chicken Stew,Chicken on the bone cut into 2 inch pieces 750...,0.9689094050129532,https://www.sanjeevkapoor.com/Recipe/Chicken-S...
1,Quick Chicken Pulao,"Basmati rice 1 1/2 cups,Boneless chicken breas...",0.967294607440748,https://www.sanjeevkapoor.com/Recipe/Quick-Chi...
2,Mutton Pulao - Sk Khazana,Boneless mutton cut into 2 inch pieces 500 gra...,0.9643700673793139,https://www.sanjeevkapoor.com/Recipe/Mutton-Pu...
3,Brown Rice- Sk Khazana,"Basmati rice washed, soaked for 30 minutes and...",0.9632156023603337,https://www.sanjeevkapoor.com/Recipe/Brown-Ric...
4,Achari Chana Pulao,"Chilli Pickle 1/4 cup,Bengal Gram ( Chana) Soa...",0.9623854017518257,https://www.sanjeevkapoor.com/Recipe/Achari-Ch...


In [716]:
# Model 4(1) + True tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2943517101.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.05)


,recipe,ingredients,score,url
0,Chicken Stew,Chicken on the bone cut into 2 inch pieces 750...,0.9705568349700573,https://www.sanjeevkapoor.com/Recipe/Chicken-S...
1,Achari Chana Pulao,"Chilli Pickle 1/4 cup,Bengal Gram ( Chana) Soa...",0.9629529036359328,https://www.sanjeevkapoor.com/Recipe/Achari-Ch...
2,Dahiwale Babycorn And Mushroom,"Babycorns cut into 1 inch pieces 12-15,Button ...",0.9614143393852803,https://www.sanjeevkapoor.com/Recipe/Dahiwale-...
3,Matar Wale Chawal,"Green peas (matar) 1 cup,Basmati rice soaked f...",0.9572876082340862,https://www.sanjeevkapoor.com/Recipe/Matar-Wal...
4,Chilli Chicken Tomato Pulao,"Chicken boneless 1 inch cubes 250 grams,Basmat...",0.9564051342854825,https://www.sanjeevkapoor.com/Recipe/Chilli-Ch...


In [736]:
# Model 5(1) + False tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\16460547.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.1)


,recipe,ingredients,score,url
0,Chicken Stew,Chicken on the bone cut into 2 inch pieces 750...,0.8755186726618194,https://www.sanjeevkapoor.com/Recipe/Chicken-S...
1,Mutton Pulao - Sk Khazana,Boneless mutton cut into 2 inch pieces 500 gra...,0.8640928407119512,https://www.sanjeevkapoor.com/Recipe/Mutton-Pu...
2,Mutton Kalimiri,Mutton cut into 1 1/2 inch pieces on the bone...,0.8634548305976223,https://www.sanjeevkapoor.com/Recipe/Mutton-Ka...
3,Kali Mirch Ka Gosht,"Crushed Black Peppercorns 1 tablespoon,Mutton ...",0.8634548305976223,https://www.sanjeevkapoor.com/Recipe/Kali-Mirc...
4,Quick Chicken Pulao,"Basmati rice 1 1/2 cups,Boneless chicken breas...",0.8631838744209654,https://www.sanjeevkapoor.com/Recipe/Quick-Chi...


In [739]:
# Model 5(1) + True tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3538546324.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.1)


,recipe,ingredients,score,url
0,Chicken Stew,Chicken on the bone cut into 2 inch pieces 750...,0.8793934274969905,https://www.sanjeevkapoor.com/Recipe/Chicken-S...
1,Chicken Jehangiri- Sk Khazana,Chicken cut into 2 inch pieces on the bone 750...,0.8523213860258877,https://www.sanjeevkapoor.com/Recipe/Chicken-J...
2,Lemon Grass Flavoured Chicken Curry,Chicken Curry Cut curry cut pieces on the bone...,0.8503245500051946,https://www.sanjeevkapoor.com/Recipe/Lemon-Gra...
3,Dahiwale Babycorn And Mushroom,"Babycorns cut into 1 inch pieces 12-15,Button ...",0.8499427269173718,https://www.sanjeevkapoor.com/Recipe/Dahiwale-...
4,Malvani Chicken Hara Masala,Chicken skinned and cut into 16 pieces on the ...,0.8467804358851931,https://www.sanjeevkapoor.com/Recipe/Malvani-C...


In [763]:
# Model 6(1) + False tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1875641014.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.15)


,recipe,ingredients,score,url
0,Apricot Chicken On Skewers,"Boneless chicken leg, cut into 1 inch pieces 1...",0.6320891286699086,https://www.sanjeevkapoor.com/Recipe/Apricot-C...
1,Nimbuwala Pulao,"Basmati Rice soaked 3 cups,Lemongrass 1 bunch,...",0.6310633312936424,https://www.sanjeevkapoor.com/Recipe/Nimbuwala...
2,Jeera Rice - Sk Khazana,"cumin seeds 2 teaspoons,basmati rice soaked fo...",0.6087524976396295,https://www.sanjeevkapoor.com/Recipe/Jeera-Ric...
3,Takiya Pulava Kabab,"Chicken Breast 8,Mutton Mince 250 grams,Ginger...",0.6048183187765808,https://www.sanjeevkapoor.com/Recipe/Takiya-Pu...
4,Chutney Pulao With Paneer And Corn,"Basmati Rice washed and soaked 1 1/2 cups,Cott...",0.6014030082617401,https://www.sanjeevkapoor.com/Recipe/Chutney-P...


In [765]:
# Model 6(1) + True tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\245278411.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.15)


,recipe,ingredients,score,url
0,Nimbuwala Pulao,"Basmati Rice soaked 3 cups,Lemongrass 1 bunch,...",0.6250052754587554,https://www.sanjeevkapoor.com/Recipe/Nimbuwala...
1,Chicken Jehangiri- Sk Khazana,Chicken cut into 2 inch pieces on the bone 750...,0.6083006134158484,https://www.sanjeevkapoor.com/Recipe/Chicken-J...
2,Jeera Rice - Sk Khazana,"cumin seeds 2 teaspoons,basmati rice soaked fo...",0.6061083518784007,https://www.sanjeevkapoor.com/Recipe/Jeera-Ric...
3,Apricot Chicken On Skewers,"Boneless chicken leg, cut into 1 inch pieces 1...",0.6002655922738451,https://www.sanjeevkapoor.com/Recipe/Apricot-C...
4,Takiya Pulava Kabab,"Chicken Breast 8,Mutton Mince 250 grams,Ginger...",0.5969064793545048,https://www.sanjeevkapoor.com/Recipe/Takiya-Pu...


In [767]:
# Model 7(1) + False tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1219415236.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url
0,Chicken Stock,"Chicken bones 200 grams,Onion peeled 1 medium,...",0.49656475775358155,https://www.sanjeevkapoor.com/Recipe/Chicken-S...
1,Aloo Chutney Pulao,"Baby potatoes boiled and peeled 10-12,Green ch...",0.4844859316760416,https://www.sanjeevkapoor.com/Recipe/Aloo-Chut...
2,Chicken Stew- Sk Khazana,Boneless chicken breasts cut into 1/2 inch cub...,0.48432783915435546,https://www.sanjeevkapoor.com/Recipe/Chicken-S...
3,Beetroot And Mustard Pulao,"Basmati rice soaked 2 cups,Beetroots boiled 3-...",0.4753659139620941,https://www.sanjeevkapoor.com/Recipe/Beetroot-...
4,Sangameshwari Chicken,Chicken on the bone cut into bite sized pieces...,0.47294792692768084,https://www.sanjeevkapoor.com/Recipe/Sangamesh...


In [769]:
# Model 7(1) + True tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url
0,Country Captain Chicken,"Chicken cut into pieces 1 kilogram,Oil 1 table...",0.5369949339739375,https://www.sanjeevkapoor.com/Recipe/Country-C...
1,Coorg Style Dry Chicken,Chicken cut into 8 pieces on the bone 750 gram...,0.5296900021167451,https://www.sanjeevkapoor.com/Recipe/Coorg-Sty...
2,Chicken Stew- Sk Khazana,Boneless chicken breasts cut into 1/2 inch cub...,0.5173323377048464,https://www.sanjeevkapoor.com/Recipe/Chicken-S...
3,Pollo A La Brasa,"Chicken cut into 4 pieces, 1 whole,Soy sauce 1...",0.5121328081191735,https://www.sanjeevkapoor.com/Recipe/Pollo-a-L...
4,Chicken Tagine With Chickpeas And Mint,Chickpeas (kabuli chana) soaked and parboiled ...,0.5070411692371333,https://www.sanjeevkapoor.com/Recipe/Chicken-T...


In [824]:
# Model 2 + False tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1633375022.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Barley And Peanut Soup,"Barley 1 cup,Peanuts 1/2 cup,Oil 2 tablespoons...",0.9953359721824703,https://www.sanjeevkapoor.com/Recipe/Barley-An...
1,Mirchi Coconut Pulao,"Coconut 4-5,Basmati Rice 4 tablespoons,Coconut...",0.9929451365520628,https://www.sanjeevkapoor.com/Recipe/Mirchi-Co...
2,Matar Wale Chawal,"Green peas (matar) 1 cup,Basmati rice soaked f...",0.992923098830375,https://www.sanjeevkapoor.com/Recipe/Matar-Wal...
3,Tomato Sabzi Pulao,"Basmati Rice 3/4 cup,Tomatoes 2,French beans 8...",0.992814742626201,https://www.sanjeevkapoor.com/Recipe/Tomato-Sa...
4,Sabzi Aur Tamatar Ka Pulao,"Basmati rice 1 1/2 cups,Tomatoes chopped 3-4,G...",0.9928115974783036,https://www.sanjeevkapoor.com/Recipe/Sabzi-Aur...


In [836]:
# Model 2 + True tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1992341500.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Goan Peas Coconut Pulao,"Green peas 1 cup,Coconut milk 1/2 cup,Rice ric...",0.9933071292628066,https://www.sanjeevkapoor.com/Recipe/Goan-Peas...
1,Vegetable Pulao,"Basmati rice boiled 2 cups,Cauliflower separat...",0.9922503212262767,https://www.sanjeevkapoor.com/Recipe/Vegetable...
2,Simple Vegetable Pulao,"Cauliflower florets 8-10,French beans stringed...",0.9909764469776249,https://www.sanjeevkapoor.com/Recipe/Simple-Ve...
3,Achari Chana Pulao,"Chilli Pickle 1/4 cup,Bengal Gram ( Chana) Soa...",0.9899388234108268,https://www.sanjeevkapoor.com/Recipe/Achari-Ch...
4,Beetroot And Mustard Pulao,"Basmati rice soaked 2 cups,Beetroots boiled 3-...",0.9898093479776909,https://www.sanjeevkapoor.com/Recipe/Beetroot-...


In [849]:
# Model 8(2) + False tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3447942711.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Takiya Pulava Kabab,"Chicken Breast 8,Mutton Mince 250 grams,Ginger...",0.6899612268255014,https://www.sanjeevkapoor.com/Recipe/Takiya-Pu...
1,Chicken Pulao,"Chicken on bone, cut into pieces 500 grams,Bas...",0.6803836738214017,https://www.sanjeevkapoor.com/Recipe/Chicken-P...
2,Matar Wale Chawal,"Green peas (matar) 1 cup,Basmati rice soaked f...",0.6786507340394422,https://www.sanjeevkapoor.com/Recipe/Matar-Wal...
3,Bhooga Chawar,"Basmati Rice soaked 1 1/2 cups,Onion 3 tablesp...",0.6746047308677894,https://www.sanjeevkapoor.com/Recipe/Bhooga-Ch...
4,Chicken Stuffed Tomatoes,"Chicken mince 1 cup,Tomatoes 4 large,Red chill...",0.6685620121534837,https://www.sanjeevkapoor.com/Recipe/Chicken-S...


In [863]:
# Model 8(2) + True tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4078852487.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Pepper Garlic Malai Prawns,"Medium prawns shelled and deveined 20,Garlic c...",0.6722427477424859,https://www.sanjeevkapoor.com/Recipe/Pepper-Ga...
1,Country Captain Chicken,"Chicken cut into pieces 1 kilogram,Oil 1 table...",0.6621820065311914,https://www.sanjeevkapoor.com/Recipe/Country-C...
2,Matar Wale Chawal,"Green peas (matar) 1 cup,Basmati rice soaked f...",0.6525207926785597,https://www.sanjeevkapoor.com/Recipe/Matar-Wal...
3,Chicken Stuffed Tomatoes,"Chicken mince 1 cup,Tomatoes 4 large,Red chill...",0.6505914640950644,https://www.sanjeevkapoor.com/Recipe/Chicken-S...
4,Chicken Cafreal,"Chicken cut into 8 pieces 1 kilogram,Onion sli...",0.650333874457291,https://www.sanjeevkapoor.com/Recipe/Chicken-C...


In [875]:
# Model 9(2) + False tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2934798296.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Mutton Vindaloo Bunny Chow,Boneless mutton cut into 1 inch cubes 1 kilogr...,0.5121025771459433,https://www.sanjeevkapoor.com/Recipe/Mutton-Vi...
1,Chicken Curry,"Chicken on the bone , cut into 1 inch pieces o...",0.5083072557304597,https://www.sanjeevkapoor.com/Recipe/Chicken-C...
2,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.5069461022812455,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
3,Karavalli Mutton Curry,"Mutton cut into 1/2 inch pieces 500 grams,Vi...",0.49730601524816875,https://www.sanjeevkapoor.com/Recipe/Karavalli...
4,Chicken Pulao,"Chicken on bone, cut into pieces 500 grams,Bas...",0.49479229634893074,https://www.sanjeevkapoor.com/Recipe/Chicken-P...


In [887]:
# Model 9(2) + True tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2669468391.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Dhaniya Aur Tamatar Ka Shorba,"Fresh Coriander Stems 1 stalks,Tomato chopped ...",0.5063750675862267,https://www.sanjeevkapoor.com/Recipe/Dhaniya-A...
1,Chicken With Bamboo Shoots,"Chicken cut into 1 inch cubes 600 grams,Bamboo...",0.49696650960298727,https://www.sanjeevkapoor.com/Recipe/Chicken-W...
2,Kokur Nadur,"Chicken 1/2 kilogram,Lotus root 1/4 kilogram,R...",0.49432982913129475,https://www.sanjeevkapoor.com/Recipe/Kokur-Nad...
3,Pollo A La Brasa,"Chicken cut into 4 pieces, 1 whole,Soy sauce 1...",0.49134312569206284,https://www.sanjeevkapoor.com/Recipe/Pollo-a-L...
4,Bhuna Kukda,"Chicken 1 kilogram,Dried Red Chillies 20,Garli...",0.4899445566707354,https://www.sanjeevkapoor.com/Recipe/Bhuna-Kuk...


In [899]:
# Model 10(2) + False tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2013326127.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.4738223254956892,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
1,Kokur Nadur,"Chicken 1/2 kilogram,Lotus root 1/4 kilogram,R...",0.45322888219227536,https://www.sanjeevkapoor.com/Recipe/Kokur-Nad...
2,Wadian Pulao,"Urad Dal Wadi roughly crushed 5-6,Basmati Rice...",0.44408565445058423,https://www.sanjeevkapoor.com/Recipe/Wadian-Pu...
3,Chicken Stock,"Chicken bones 200 grams,Onion peeled 1 medium,...",0.43599410361549856,https://www.sanjeevkapoor.com/Recipe/Chicken-S...
4,Chicken Tagine With Chickpeas And Mint,Chickpeas (kabuli chana) soaked and parboiled ...,0.4275950582927483,https://www.sanjeevkapoor.com/Recipe/Chicken-T...


In [911]:
# Model 10(2) + True tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4131289077.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Kokur Nadur,"Chicken 1/2 kilogram,Lotus root 1/4 kilogram,R...",0.526728169145068,https://www.sanjeevkapoor.com/Recipe/Kokur-Nad...
1,Pollo A La Brasa,"Chicken cut into 4 pieces, 1 whole,Soy sauce 1...",0.5078466080859707,https://www.sanjeevkapoor.com/Recipe/Pollo-a-L...
2,Chicken Tagine With Chickpeas And Mint,Chickpeas (kabuli chana) soaked and parboiled ...,0.48895617549410697,https://www.sanjeevkapoor.com/Recipe/Chicken-T...
3,Apricot Chicken,"Chicken 1 cup,Dried apricots 500 grams,Butter ...",0.45536008924496385,https://www.sanjeevkapoor.com/Recipe/Apricot-C...
4,Sarson Saag Chicken,"Chicken cut into 8 pieces on bone 750 grams,Sp...",0.4331491392758545,https://www.sanjeevkapoor.com/Recipe/Sarson-Sa...


In [923]:
# Model 11(2) + False tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\535754922.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Bhooga Chawar,"Basmati Rice soaked 1 1/2 cups,Onion 3 tablesp...",0.6914513301015194,https://www.sanjeevkapoor.com/Recipe/Bhooga-Ch...
1,Fish Kofta Pulao,Boneless fish fillets cut into small pieces 20...,0.6708211071387018,https://www.sanjeevkapoor.com/Recipe/Fish-Koft...
2,Mutton Kalimiri,Mutton cut into 1 1/2 inch pieces on the bone...,0.6643665807781379,https://www.sanjeevkapoor.com/Recipe/Mutton-Ka...
3,Kali Mirch Ka Gosht,"Crushed Black Peppercorns 1 tablespoon,Mutton ...",0.6643665807781379,https://www.sanjeevkapoor.com/Recipe/Kali-Mirc...
4,Chicken Pulao,"Chicken on bone, cut into pieces 500 grams,Bas...",0.6635845163976241,https://www.sanjeevkapoor.com/Recipe/Chicken-P...


In [935]:
# Model 11(2) + True tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3489482770.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Fish Kofta Pulao,Boneless fish fillets cut into small pieces 20...,0.6946104916887654,https://www.sanjeevkapoor.com/Recipe/Fish-Koft...
1,Bhooga Chawar,"Basmati Rice soaked 1 1/2 cups,Onion 3 tablesp...",0.6901560438146477,https://www.sanjeevkapoor.com/Recipe/Bhooga-Ch...
2,Chicken Pulao,"Chicken 1 1/2 inch pieces 600 grams,Rice,Basma...",0.686350633886529,https://www.sanjeevkapoor.com/Recipe/Chicken-P...
3,Chicken Jehangiri- Sk Khazana,Chicken cut into 2 inch pieces on the bone 750...,0.6851258305385007,https://www.sanjeevkapoor.com/Recipe/Chicken-J...
4,Lal Gosht,"Mutton on bones pressure cooked 750 grams,Oil ...",0.6849230516615772,https://www.sanjeevkapoor.com/Recipe/Lal-Gosht...


In [948]:
# Model 12(2) + False tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4082322462.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Chicken Stock,"Chicken bones 200 grams,Onion peeled 1 medium,...",0.5781726754372694,https://www.sanjeevkapoor.com/Recipe/Chicken-S...
1,Wadian Pulao,"Urad Dal Wadi roughly crushed 5-6,Basmati Rice...",0.5688065087483885,https://www.sanjeevkapoor.com/Recipe/Wadian-Pu...
2,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.5473625744812706,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
3,Bhooga Chawar,"Basmati Rice soaked 1 1/2 cups,Onion 3 tablesp...",0.5386728998126611,https://www.sanjeevkapoor.com/Recipe/Bhooga-Ch...
4,Rice,"Basmati rice 2 cups,Salt to taste",0.5380482636520122,https://www.sanjeevkapoor.com/Recipe/Rice.html


In [960]:
# Model 12(2) + True tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2426173970.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Kombdi Rassa,Chicken cut into 8 pieces on the bone 750 gram...,0.57186545734433,https://www.sanjeevkapoor.com/Recipe/Kombdi-Ra...
1,Chicken With Bamboo Shoots,"Chicken cut into 1 inch cubes 600 grams,Bamboo...",0.5713938628452118,https://www.sanjeevkapoor.com/Recipe/Chicken-W...
2,Kokur Yakhni,"Chicken 750 grams,Fennel seed (saunf) powder 1...",0.5638490650259141,https://www.sanjeevkapoor.com/Recipe/Kokur-Yak...
3,Coorg Style Dry Chicken,Chicken cut into 8 pieces on the bone 750 gram...,0.5570615653431252,https://www.sanjeevkapoor.com/Recipe/Coorg-Sty...
4,Chicken Pulao,"Chicken 1 1/2 inch pieces 600 grams,Rice,Basma...",0.5553405815858372,https://www.sanjeevkapoor.com/Recipe/Chicken-P...


In [972]:
# Model 13(2) + False tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1986226641.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Chicken Stock,"Chicken bones 200 grams,Onion peeled 1 medium,...",0.45817833021349014,https://www.sanjeevkapoor.com/Recipe/Chicken-S...
1,Rice,"Basmati rice 2 cups,Salt to taste",0.4527238615575522,https://www.sanjeevkapoor.com/Recipe/Rice.html
2,Aloo Chutney Pulao,"Baby potatoes boiled and peeled 10-12,Green ch...",0.45194690497056894,https://www.sanjeevkapoor.com/Recipe/Aloo-Chut...
3,Wadian Pulao,"Urad Dal Wadi roughly crushed 5-6,Basmati Rice...",0.4505626202201757,https://www.sanjeevkapoor.com/Recipe/Wadian-Pu...
4,Mango Coriander Chicken,"Ripe mango peeled and cut into cubes 1 large,F...",0.4473894507595031,https://www.sanjeevkapoor.com/Recipe/Mango-Cor...


In [984]:
# Model 13(2) + True tests
input = "chicken"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4233235582.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Chicken Pulao,"Chicken 1 1/2 inch pieces 600 grams,Rice,Basma...",0.4572502085409031,https://www.sanjeevkapoor.com/Recipe/Chicken-P...
1,Rice,"Basmati rice 2 cups,Salt to taste",0.4527238606867675,https://www.sanjeevkapoor.com/Recipe/Rice.html
2,Thai Chicken Mango Curry-Cook Smart,Chicken cut into bite sized pieces on the bone...,0.4496938061790154,https://www.sanjeevkapoor.com/Recipe/Thai-Chic...
3,Kokur Yakhni,"Chicken 750 grams,Fennel seed (saunf) powder 1...",0.44756895844811273,https://www.sanjeevkapoor.com/Recipe/Kokur-Yak...
4,Sarson Saag Chicken,"Chicken cut into 8 pieces on bone 750 grams,Sp...",0.44386755979483294,https://www.sanjeevkapoor.com/Recipe/Sarson-Sa...


### Cottage Cheese

In [626]:
# Model 1 + False tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2863015847.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7205979400762295,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.6704200361524906,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
2,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.6372157640794749,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
3,Paneer Kulcha,"Refined flour 2 cups,Cottage cheese 1/2 teaspo...",0.5841168992974202,https://www.sanjeevkapoor.com/Recipe/Paneer-Ku...
4,Grilled Cottage Cheese And Cabbage Sandwich,"Cottage Cheese grated 150 grams,Cabbage 8,Cabb...",0.5824300413526164,https://www.sanjeevkapoor.com/Recipe/Grilled-C...


In [637]:
# Model 1 + True tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7205979501176643,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.6090211982738374,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
2,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.5993283781798565,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
3,Paneer Aur Prune Kebab,"Cottage Cheese grated 1/2 cup,Prunes 8,Potatoe...",0.5821511809409101,https://www.sanjeevkapoor.com/Recipe/Paneer-Au...
4,Grilled Cottage Cheese And Cabbage Sandwich,"Cottage Cheese grated 150 grams,Cabbage 8,Cabb...",0.5665053366462196,https://www.sanjeevkapoor.com/Recipe/Grilled-C...


In [770]:
# Model 7(1) + True tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7729553692821229,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Makki Paneer Pakora,"Corn Kernels 200 grams,Cottage Cheese 1 cup,Pa...",0.6516645891487156,https://www.sanjeevkapoor.com/Recipe/Makki-Pan...
2,Mirchi Roll,"Bhavnagri Green Chiilies slit 4,Cottage Cheese...",0.6344077616143302,https://www.sanjeevkapoor.com/Recipe/Mirchi-Ro...
3,Grilled Cottage Cheese And Cabbage Sandwich,"Cottage Cheese grated 150 grams,Cabbage 8,Cabb...",0.6213193183994024,https://www.sanjeevkapoor.com/Recipe/Grilled-C...
4,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.5987849197147761,https://www.sanjeevkapoor.com/Recipe/Chicken-A...


In [825]:
# Model 2 + False tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1633375022.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Chicken Tikka Tortilla Wrap,"Boneless chicken cut into dices 400 grams,Tort...",0.9966320209065712,https://www.sanjeevkapoor.com/Recipe/Chicken-T...
1,Khaas Seekh-Sk Khazana,"Chicken mince 1 cup,Cottage cheese (paneer) 1 ...",0.9963065093054291,https://www.sanjeevkapoor.com/Recipe/Khaas-See...
2,Prune & Spinach Kebab,"Prunes 8,Spinach 2 bunches,Baby tomatoes 8,Oil...",0.9958807996389472,https://www.sanjeevkapoor.com/Recipe/Prune-&-S...
3,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.9957454111358011,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
4,Afghani Chicken Tikka - Sk Khazana,Boneless chicken legs Cut into 1 1/2 inch piec...,0.9955858092753227,https://www.sanjeevkapoor.com/Recipe/Afghani-C...


In [837]:
# Model 2 + True tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1992341500.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Marghi Na Farcha,"Chicken pieces on bone 800 grams,Breadcrumbs,S...",0.9969686576395234,https://www.sanjeevkapoor.com/Recipe/Marghi-Na...
1,Chicken Farcha,"Chicken drumsticks 8,Ginger-garlic paste 2 tab...",0.9964078777275964,https://www.sanjeevkapoor.com/Recipe/Chicken-F...
2,Khaas Seekh-Sk Khazana,"Chicken mince 1 cup,Cottage cheese (paneer) 1 ...",0.9961534635143524,https://www.sanjeevkapoor.com/Recipe/Khaas-See...
3,Makhanwala Paneer,Paneer (cottage cheese) cut into 1 inch cubes ...,0.9958703951832386,https://www.sanjeevkapoor.com/Recipe/Makhanwal...
4,Paneer Taash Kabab,"Cottage Cheese 400 grams,Mint Chutney cut into...",0.9958348898065073,https://www.sanjeevkapoor.com/Recipe/Paneer-Ta...


In [850]:
# Model 8(2) + False tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3447942711.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7969813308941304,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.7520415877832752,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
2,Kasoori Paneer Tikka,"Cottage Cheese 250 grams,Green capsicums 2 lar...",0.7292817648911787,https://www.sanjeevkapoor.com/Recipe/Kasoori-P...
3,Paneer Aur Prune Kebab,"Cottage Cheese grated 1/2 cup,Prunes 8,Potatoe...",0.7110082968551494,https://www.sanjeevkapoor.com/Recipe/Paneer-Au...
4,Mushroom Tikka-Sk Khazana,"Button mushrooms 10-12,Oil for greasing,Hung y...",0.6842164512624265,https://www.sanjeevkapoor.com/Recipe/Mushroom-...


In [864]:
# Model 8(2) + True tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4078852487.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7969813344185456,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Kasoori Paneer Tikka,"Cottage Cheese 250 grams,Green capsicums 2 lar...",0.7467402289893889,https://www.sanjeevkapoor.com/Recipe/Kasoori-P...
2,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.7110376267374414,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
3,Lehsuni Palak-Sk Khazana,"Spinach (palak) cleaned 1 Large bunch,Garlic f...",0.700495318871211,https://www.sanjeevkapoor.com/Recipe/Lehsuni-P...
4,Paneer Aur Prune Kebab,"Cottage Cheese grated 1/2 cup,Prunes 8,Potatoe...",0.6999422193598033,https://www.sanjeevkapoor.com/Recipe/Paneer-Au...


In [876]:
# Model 9(2) + False tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2934798296.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7281933029666556,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.6205409505832393,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
2,Paneer Aur Prune Kebab,"Cottage Cheese grated 1/2 cup,Prunes 8,Potatoe...",0.6149918754471064,https://www.sanjeevkapoor.com/Recipe/Paneer-Au...
3,Makki Paneer Pakora,"Corn Kernels 200 grams,Cottage Cheese 1 cup,Pa...",0.607911428587178,https://www.sanjeevkapoor.com/Recipe/Makki-Pan...
4,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.604615595079597,https://www.sanjeevkapoor.com/Recipe/Eggplant-...


In [888]:
# Model 9(2) + True tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2669468391.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7281933014502665,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Makki Paneer Pakora,"Corn Kernels 200 grams,Cottage Cheese 1 cup,Pa...",0.6079114213893541,https://www.sanjeevkapoor.com/Recipe/Makki-Pan...
2,Paneer Aur Prune Kebab,"Cottage Cheese grated 1/2 cup,Prunes 8,Potatoe...",0.5992330960050156,https://www.sanjeevkapoor.com/Recipe/Paneer-Au...
3,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.581392328481222,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
4,Kumbh Shaslik,Button Mushrooms 1 1/2 inch pieces layers sepa...,0.5752191938452064,https://www.sanjeevkapoor.com/Recipe/Kumbh-Sha...


In [900]:
# Model 10(2) + False tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2013326127.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.6743227007084013,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.6505162391009391,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
2,Mushroom And Spinach Calzone,"Fresh Pizza Dough 450 grams,Cottage Cheese 16,...",0.6461926315068172,https://www.sanjeevkapoor.com/Recipe/Mushroom-...
3,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.6383132903483513,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
4,Makki Paneer Pakora,"Corn Kernels 200 grams,Cottage Cheese 1 cup,Pa...",0.5647055539882653,https://www.sanjeevkapoor.com/Recipe/Makki-Pan...


In [912]:
# Model 10(2) + True tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4131289077.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.6743226976835538,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Mushroom And Spinach Calzone,"Fresh Pizza Dough 450 grams,Cottage Cheese 16,...",0.5948414013261737,https://www.sanjeevkapoor.com/Recipe/Mushroom-...
2,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.5931286265230002,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
3,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.5731471166531286,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
4,Makki Paneer Pakora,"Corn Kernels 200 grams,Cottage Cheese 1 cup,Pa...",0.5647055429758014,https://www.sanjeevkapoor.com/Recipe/Makki-Pan...


In [924]:
# Model 11(2) + False tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\535754922.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.85759218144697,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Makki Paneer Pakora,"Corn Kernels 200 grams,Cottage Cheese 1 cup,Pa...",0.7939764482944723,https://www.sanjeevkapoor.com/Recipe/Makki-Pan...
2,Kasoori Paneer Tikka,"Cottage Cheese 250 grams,Green capsicums 2 lar...",0.7924049676411955,https://www.sanjeevkapoor.com/Recipe/Kasoori-P...
3,Angoori Paneer Tikka,"Cottage cheese 400 grams,Black grapes 2 tables...",0.7904636783872512,https://www.sanjeevkapoor.com/Recipe/Angoori-P...
4,Mirchi Roll,"Bhavnagri Green Chiilies slit 4,Cottage Cheese...",0.7650667060631364,https://www.sanjeevkapoor.com/Recipe/Mirchi-Ro...


In [936]:
# Model 11(2) + True tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3489482770.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.85759217980392,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Makki Paneer Pakora,"Corn Kernels 200 grams,Cottage Cheese 1 cup,Pa...",0.7939764508119592,https://www.sanjeevkapoor.com/Recipe/Makki-Pan...
2,Kasoori Paneer Tikka,"Cottage Cheese 250 grams,Green capsicums 2 lar...",0.7937057868116619,https://www.sanjeevkapoor.com/Recipe/Kasoori-P...
3,Angoori Paneer Tikka,"Cottage cheese 400 grams,Black grapes 2 tables...",0.7841479443119197,https://www.sanjeevkapoor.com/Recipe/Angoori-P...
4,Tangdi Kabab - Sk Khazana,"Chicken drumsticks 4,Ginger-garlic paste 1 tab...",0.7549194553782005,https://www.sanjeevkapoor.com/Recipe/Tangdi-Ka...


In [949]:
# Model 12(2) + False tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4082322462.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7517731641609562,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Lehsuni Palak-Sk Khazana,"Spinach (palak) cleaned 1 Large bunch,Garlic f...",0.674572696526085,https://www.sanjeevkapoor.com/Recipe/Lehsuni-P...
2,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.6740181118369153,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
3,Shahi Paneer - Sk Khazana,"Cottage cheese (paneer) 300 grams,cashewnut pa...",0.6676293768236026,https://www.sanjeevkapoor.com/Recipe/Shahi-Pan...
4,Kasoori Paneer Tikka,"Cottage Cheese 250 grams,Green capsicums 2 lar...",0.6546924536002954,https://www.sanjeevkapoor.com/Recipe/Kasoori-P...


In [961]:
# Model 12(2) + True tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2426173970.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7517731628087044,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Lemongrass Chicken Tikka,"Lemon grass stalk roughly chopped 1,Chicken le...",0.6506730292345456,https://www.sanjeevkapoor.com/Recipe/Lemongras...
2,Paneer Tikka,"Cottage cheese (paneer) , one-inch cubes 8,Oil...",0.6471989170051785,https://www.sanjeevkapoor.com/Recipe/Paneer-Ti...
3,Kasoori Paneer Tikka,"Cottage Cheese 250 grams,Green capsicums 2 lar...",0.6427585773415958,https://www.sanjeevkapoor.com/Recipe/Kasoori-P...
4,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.6363498790616708,https://www.sanjeevkapoor.com/Recipe/Eggplant-...


In [973]:
# Model 13(2) + False tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1986226641.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7219923726046948,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.646973212194343,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
2,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.6231627617048847,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
3,Paneer Snacker,"Cottage Cheese 150 grams,Salted Biscuits 1 tea...",0.6141811708158199,https://www.sanjeevkapoor.com/Recipe/Paneer-Sn...
4,Paneer Di Soti Boti,"Cottage cheese 1/2 inch cubes 300 grams,Gram f...",0.5885477832441418,https://www.sanjeevkapoor.com/Recipe/Paneer-Di...


In [985]:
# Model 13(2) + True tests
input = "cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4233235582.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7219923703376283,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Paneer Snacker,"Cottage Cheese 150 grams,Salted Biscuits 1 tea...",0.6141811673715252,https://www.sanjeevkapoor.com/Recipe/Paneer-Sn...
2,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.608293742741191,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
3,Paneer Aur Prune Kebab,"Cottage Cheese grated 1/2 cup,Prunes 8,Potatoe...",0.5823904150188225,https://www.sanjeevkapoor.com/Recipe/Paneer-Au...
4,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.5818967174883822,https://www.sanjeevkapoor.com/Recipe/Chicken-A...


### Egg

In [627]:
# Model 1 + False tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2863015847.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",1.0000000000000002,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.6693128931968078,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
2,Choco Almond Muffin,"Cocoa powder 4 tablespoons,Refined flour coars...",0.5315863258874481,https://www.sanjeevkapoor.com/Recipe/Choco-Alm...
3,Spanish Omelette - Sk Khazana,"Eggs 5,Oil 1 tablespoon,Medium onion cut into ...",0.5191813589434888,https://www.sanjeevkapoor.com/Recipe/Spanish-O...
4,Red Onion Boiled Eggs,"Red Onion Peels,Eggs 6,Sea salt 1 tablespoon,O...",0.49949618320387046,https://www.sanjeevkapoor.com/Recipe/Red-Onion...


In [638]:
# Model 1 + True tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",0.9999999999999998,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.848920635844948,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
2,Red Onion Boiled Eggs,"Red Onion Peels,Eggs 6,Sea salt 1 tablespoon,O...",0.7880499082291668,https://www.sanjeevkapoor.com/Recipe/Red-Onion...
3,Fresh Pasta Dough,"Refined flour (maida) 300 grams,Salt to taste,...",0.7855633779643167,https://www.sanjeevkapoor.com/Recipe/Fresh-Pas...
4,Swiss Buttercream,"Butter , softened 225 grams,Eggs , at room tem...",0.7517778876202283,https://www.sanjeevkapoor.com/Recipe/Swiss-But...


In [771]:
# Model 7(1) + True tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",1.0,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.8669021155444404,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
2,Fresh Pasta Dough,"Refined flour (maida) 300 grams,Salt to taste,...",0.8040713839859716,https://www.sanjeevkapoor.com/Recipe/Fresh-Pas...
3,Swiss Buttercream,"Butter , softened 225 grams,Eggs , at room tem...",0.7590770795117624,https://www.sanjeevkapoor.com/Recipe/Swiss-But...
4,Red Onion Boiled Eggs,"Red Onion Peels,Eggs 6,Sea salt 1 tablespoon,O...",0.7553312799118763,https://www.sanjeevkapoor.com/Recipe/Red-Onion...


In [826]:
# Model 2 + False tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1633375022.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",1.0,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Corn Dog,"Corn meal 1 cup,Chicken sausages 4,Refined flo...",0.9916253109529263,https://www.sanjeevkapoor.com/Recipe/Corn-Dog....
2,Pizza Tart-Sk Khazana,Refined flour (maida) 1 1/2 cups + for dusting...,0.9892420006858333,https://www.sanjeevkapoor.com/Recipe/Pizza-Tar...
3,Fresh Pasta Dough,"Refined flour (maida) 300 grams,Salt to taste,...",0.9890496691240387,https://www.sanjeevkapoor.com/Recipe/Fresh-Pas...
4,Herbed Bread,"Dried oregano 1 teaspoon,Refined flour (maida)...",0.9887519112895055,https://www.sanjeevkapoor.com/Recipe/Herbed-Br...


In [838]:
# Model 2 + True tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1992341500.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",1.0,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Fresh Pasta Dough,"Refined flour (maida) 300 grams,Salt to taste,...",0.9952515199781712,https://www.sanjeevkapoor.com/Recipe/Fresh-Pas...
2,Corn Dog,"Corn meal 1 cup,Chicken sausages 4,Refined flo...",0.990145911340436,https://www.sanjeevkapoor.com/Recipe/Corn-Dog....
3,Shortbread Cookies,"Refined flour (maida) 250 grams for dusting,Va...",0.9898157914434116,https://www.sanjeevkapoor.com/Recipe/Shortbrea...
4,Pizza Tart-Sk Khazana,Refined flour (maida) 1 1/2 cups + for dusting...,0.9894941965631538,https://www.sanjeevkapoor.com/Recipe/Pizza-Tar...


In [851]:
# Model 8(2) + False tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3447942711.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",1.0,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.7353792908932704,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
2,Warm Souffle With Grand Marnier,"Eggs 3,Refined flour (maida) 5 tablespoons,But...",0.6178001931407441,https://www.sanjeevkapoor.com/Recipe/Warm-Souf...
3,Lemon Coconut Slice,"Refined flour (maida) (For slice) 3/4 cup,Butt...",0.610149281796139,https://www.sanjeevkapoor.com/Recipe/Lemon-Coc...
4,Almond Crusted Prawns With Wasabi Mayo,"Almond 1 cup,Prawns peeled and deveined boiled...",0.5899486404692331,https://www.sanjeevkapoor.com/Recipe/Almond-Cr...


In [865]:
# Model 8(2) + True tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4078852487.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",1.0,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.8580014454273979,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
2,Fresh Pasta Dough,"Refined flour (maida) 300 grams,Salt to taste,...",0.7918631221779244,https://www.sanjeevkapoor.com/Recipe/Fresh-Pas...
3,Swiss Buttercream,"Butter , softened 225 grams,Eggs , at room tem...",0.7774691608625064,https://www.sanjeevkapoor.com/Recipe/Swiss-But...
4,Christmas Creme Brulee,"Cream 2 1/2 cups,Eggs 4,Castor sugar (caster ...",0.7770968484683397,https://www.sanjeevkapoor.com/Recipe/Christmas...


In [877]:
# Model 9(2) + False tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2934798296.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",1.0,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.6685415166218873,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
2,Lemon Coconut Slice,"Refined flour (maida) (For slice) 3/4 cup,Butt...",0.5819906409612435,https://www.sanjeevkapoor.com/Recipe/Lemon-Coc...
3,Savoiardi Biscuits (Lady Finger Biscuits),"Eggs 3,Castor sugar 1/3 cup,Vanilla essence 1,...",0.5517955698584459,https://www.sanjeevkapoor.com/Recipe/Savoiardi...
4,Jam Swiss Roll,"Refined flour (maida) 35 grams,Mixed fruit jam...",0.5456894996861301,https://www.sanjeevkapoor.com/Recipe/Jam-Swiss...


In [889]:
# Model 9(2) + True tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2669468391.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",0.9999999999999999,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.836371560757824,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
2,Swiss Buttercream,"Butter , softened 225 grams,Eggs , at room tem...",0.7869945449638022,https://www.sanjeevkapoor.com/Recipe/Swiss-But...
3,Fresh Pasta Dough,"Refined flour (maida) 300 grams,Salt to taste,...",0.7832627638337758,https://www.sanjeevkapoor.com/Recipe/Fresh-Pas...
4,Hot Chocolate Souffle,"Dark chocolate chopped 70 grams,Butter for gre...",0.7691742384774856,https://www.sanjeevkapoor.com/Recipe/Hot-Choco...


In [901]:
# Model 10(2) + False tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2013326127.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",0.9999999999999999,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.6075273758899095,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
2,Mango Lemon Bars,"Refined flour (maida) 1 1/2 cups,Castor sugar...",0.5537670579133506,https://www.sanjeevkapoor.com/Recipe/Mango-Lem...
3,Coconut Squares With Blueberries,"Desiccated coconut 75 grams,Refined flour (mai...",0.5427717109240061,https://www.sanjeevkapoor.com/Recipe/Coconut-S...
4,Chicken Cheese Bombs-Sk Khazana,"Chicken mince 1 1/2 cups,Mozzarella cheese cut...",0.540557000463272,https://www.sanjeevkapoor.com/Recipe/Chicken-C...


In [913]:
# Model 10(2) + True tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4131289077.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",0.9999999999999998,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.8262655494945056,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
2,Fresh Pasta Dough,"Refined flour (maida) 300 grams,Salt to taste,...",0.7783273057964121,https://www.sanjeevkapoor.com/Recipe/Fresh-Pas...
3,Swiss Buttercream,"Butter , softened 225 grams,Eggs , at room tem...",0.7387763003016473,https://www.sanjeevkapoor.com/Recipe/Swiss-But...
4,Hot Chocolate Souffle,"Dark chocolate chopped 70 grams,Butter for gre...",0.7310598516007074,https://www.sanjeevkapoor.com/Recipe/Hot-Choco...


In [925]:
# Model 11(2) + False tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\535754922.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",1.0000000000000002,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.6800874888083288,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
2,Fresh Pastas,"Refined flour (maida) 300 grams,Salt 1 teaspoo...",0.6281731701576707,https://www.sanjeevkapoor.com/Recipe/Fresh-Pas...
3,Jam Swiss Roll,"Refined flour (maida) 35 grams,Mixed fruit jam...",0.609852391032387,https://www.sanjeevkapoor.com/Recipe/Jam-Swiss...
4,Lemon Coconut Slice,"Refined flour (maida) (For slice) 3/4 cup,Butt...",0.6023992889948644,https://www.sanjeevkapoor.com/Recipe/Lemon-Coc...


In [937]:
# Model 11(2) + True tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3489482770.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",1.0,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.8496104802429445,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
2,Fresh Pasta Dough,"Refined flour (maida) 300 grams,Salt to taste,...",0.8247494140740425,https://www.sanjeevkapoor.com/Recipe/Fresh-Pas...
3,Red Onion Boiled Eggs,"Red Onion Peels,Eggs 6,Sea salt 1 tablespoon,O...",0.81535102839369,https://www.sanjeevkapoor.com/Recipe/Red-Onion...
4,Swiss Buttercream,"Butter , softened 225 grams,Eggs , at room tem...",0.8005259999732919,https://www.sanjeevkapoor.com/Recipe/Swiss-But...


In [950]:
# Model 12(2) + False tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4082322462.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",1.0000000000000004,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Spanish Omelette - Sk Khazana,"Eggs 5,Oil 1 tablespoon,Medium onion cut into ...",0.6263086489342538,https://www.sanjeevkapoor.com/Recipe/Spanish-O...
2,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.598968380341224,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
3,Lemon Coconut Slice,"Refined flour (maida) (For slice) 3/4 cup,Butt...",0.5978569759696502,https://www.sanjeevkapoor.com/Recipe/Lemon-Coc...
4,Shortbread Cookies,"Refined flour (maida) 250 grams for dusting,Va...",0.5533225205645314,https://www.sanjeevkapoor.com/Recipe/Shortbrea...


In [962]:
# Model 12(2) + True tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2426173970.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",1.0000000000000002,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.8247975125423276,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
2,Fresh Pasta Dough,"Refined flour (maida) 300 grams,Salt to taste,...",0.7907417193102457,https://www.sanjeevkapoor.com/Recipe/Fresh-Pas...
3,Swiss Buttercream,"Butter , softened 225 grams,Eggs , at room tem...",0.7704242776784065,https://www.sanjeevkapoor.com/Recipe/Swiss-But...
4,Red Onion Boiled Eggs,"Red Onion Peels,Eggs 6,Sea salt 1 tablespoon,O...",0.7595187093126984,https://www.sanjeevkapoor.com/Recipe/Red-Onion...


In [974]:
# Model 13(2) + False tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1986226641.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",1.0000000000000004,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.6278409997831501,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
2,Chocolate Meringue,"Cocoa powder 30 grams,Icing sugar 150 grams,Eg...",0.5439971414709137,https://www.sanjeevkapoor.com/Recipe/Chocolate...
3,Crumbed Chicken Fingers,"Boneless chicken breast 6,Egg beaten 1 Big,But...",0.5376414008937135,https://www.sanjeevkapoor.com/Recipe/Crumbed-C...
4,Swiss Buttercream,"Butter , softened 225 grams,Eggs , at room tem...",0.5281333778902008,https://www.sanjeevkapoor.com/Recipe/Swiss-But...


In [986]:
# Model 13(2) + True tests
input = "egg"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4233235582.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Gluten Free Peanut Butter Mug Cake,"Peanut butter creamy 3 tablespoons,Egg 1,Casto...",1.0,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
1,Crema Catalana,"Milk (500 ml) 2 1/2 cups,Eggs 1 inch stick,Egg...",0.8521239725982379,https://www.sanjeevkapoor.com/Recipe/Crema-Cat...
2,Swiss Buttercream,"Butter , softened 225 grams,Eggs , at room tem...",0.7999163861302511,https://www.sanjeevkapoor.com/Recipe/Swiss-But...
3,Fresh Pasta Dough,"Refined flour (maida) 300 grams,Salt to taste,...",0.751677731541944,https://www.sanjeevkapoor.com/Recipe/Fresh-Pas...
4,Red Onion Boiled Eggs,"Red Onion Peels,Eggs 6,Sea salt 1 tablespoon,O...",0.7480655442872424,https://www.sanjeevkapoor.com/Recipe/Red-Onion...


### onion, chicken, cottage cheese

In [628]:
# Model 1 + False tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2863015847.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7549526259679472,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.7127479815370328,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
2,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.7006595839744324,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
3,Paneer Aur Prune Kebab,"Cottage Cheese grated 1/2 cup,Prunes 8,Potatoe...",0.5961924465510192,https://www.sanjeevkapoor.com/Recipe/Paneer-Au...
4,Mirchi Roll,"Bhavnagri Green Chiilies slit 4,Cottage Cheese...",0.5888570853330914,https://www.sanjeevkapoor.com/Recipe/Mirchi-Ro...


In [639]:
# Model 1 + True tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.680434733393912,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Tawa Chicken - Sk Khazana,Chicken cut into bite sized pieces on the bone...,0.6571071809508687,https://www.sanjeevkapoor.com/Recipe/Tawa-Chic...
2,Mutton Jalfrezi,"Boneless mutton cut into cubes 500 grams,Salt ...",0.6527470115264216,https://www.sanjeevkapoor.com/Recipe/Mutton-Ja...
3,Akhrot Murgh,"Chicken blanched, peeled and powdered 1/2 cup,...",0.6428661600269792,https://www.sanjeevkapoor.com/Recipe/Akhrot-Mu...
4,"Sirka Pyaaz, Murgh Aur Hari Mirch",Chicken skinned and cut into 1 inch pieces on ...,0.6422209775387944,https://www.sanjeevkapoor.com/Recipe/Sirka-Pya...


In [772]:
# Model 7(1) + True tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url
0,Fried Onions For Biryani,"Onions 3 medium,Oil to deep fry",0.7249817848676032,https://www.sanjeevkapoor.com/Recipe/Fried-Oni...
1,Country Captain Chicken,"Chicken cut into pieces 1 kilogram,Oil 1 table...",0.69508688685048,https://www.sanjeevkapoor.com/Recipe/Country-C...
2,Tawa Chicken - Sk Khazana,Chicken cut into bite sized pieces on the bone...,0.690785663174289,https://www.sanjeevkapoor.com/Recipe/Tawa-Chic...
3,Bhooga Chawar,"Basmati Rice soaked 1 1/2 cups,Onion 3 tablesp...",0.6865013433506747,https://www.sanjeevkapoor.com/Recipe/Bhooga-Ch...
4,Kheema Bhari Mirch,"Large green chillies 8-12 large,Chicken mince ...",0.6843553432170233,https://www.sanjeevkapoor.com/Recipe/Kheema-Bh...


In [827]:
# Model 2 + False tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1633375022.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Kumbh Shaslik,Button Mushrooms 1 1/2 inch pieces layers sepa...,0.9983223902270086,https://www.sanjeevkapoor.com/Recipe/Kumbh-Sha...
1,Kofte Mein Kofta,"Chicken mince (keema) 200 grams,Oil 3 tablespo...",0.9972968503443072,https://www.sanjeevkapoor.com/Recipe/Kofte-Mei...
2,Pumpkin Soup With Kasoori Methi,"Pumpkin boiled and pureed 500 grams,Dried fenu...",0.9972429523489277,https://www.sanjeevkapoor.com/Recipe/Pumpkin-S...
3,Marghi Na Farcha,"Chicken pieces on bone 800 grams,Breadcrumbs,S...",0.997084729071668,https://www.sanjeevkapoor.com/Recipe/Marghi-Na...
4,Vegetable Makhni,"Carrot cut into 1 inch pieces 1 medium,Corn ke...",0.9970710811084103,https://www.sanjeevkapoor.com/Recipe/Vegetable...


In [839]:
# Model 2 + True tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1992341500.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Punjabi Roasted Mutton,Boneless Mutton cut into 2 inch pieces 750 gra...,0.9981993398315091,https://www.sanjeevkapoor.com/Recipe/Punjabi-R...
1,Couscous Makai Pulao,"Couscous 1 cup,American corn kernels 1 cup,Oli...",0.9972158628229243,https://www.sanjeevkapoor.com/Recipe/Couscous-...
2,Goan Chicken Stew,Boneless chicken breasts cut into thick strips...,0.9970588200042102,https://www.sanjeevkapoor.com/Recipe/Goan-Chic...
3,Chicken And Chickpea Tagine,"Chicken legs skinned and halved 3,Chickpeas (k...",0.9970201044580024,https://www.sanjeevkapoor.com/Recipe/Chicken-a...
4,Bemisaal Paneer,"Cottage Cheese 500 grams,Carrot grated 100 gra...",0.9969266246199547,https://www.sanjeevkapoor.com/Recipe/Bemisaal-...


In [852]:
# Model 8(2) + False tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3447942711.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.77217250139199,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
1,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7627385780619813,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
2,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.73956789425792,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
3,Makki Paneer Pakora,"Corn Kernels 200 grams,Cottage Cheese 1 cup,Pa...",0.7071357443730049,https://www.sanjeevkapoor.com/Recipe/Makki-Pan...
4,Makhanwala Paneer,Paneer (cottage cheese) cut into 1 inch cubes ...,0.7070040456275717,https://www.sanjeevkapoor.com/Recipe/Makhanwal...


In [866]:
# Model 8(2) + True tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4078852487.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Murgh Ka Khurchan,Boneless chicken cut into thick strips 400 gra...,0.7707804334233559,https://www.sanjeevkapoor.com/Recipe/Murgh-Ka-...
1,Egg Hara Masala,"Eggs boiled, peeled and cut into half 6-8,Fres...",0.7622713795679412,https://www.sanjeevkapoor.com/Recipe/Egg-Hara-...
2,Baked Palak Paneer,"Blanched spinach puree 1 cup,Cottage cheese cu...",0.7608191453766555,https://www.sanjeevkapoor.com/Recipe/Baked-Pal...
3,Wadian Pulao,"Urad Dal Wadi roughly crushed 5-6,Basmati Rice...",0.7575010875424297,https://www.sanjeevkapoor.com/Recipe/Wadian-Pu...
4,Bhooga Chawar,"Basmati Rice soaked 1 1/2 cups,Onion 3 tablesp...",0.7548273999139088,https://www.sanjeevkapoor.com/Recipe/Bhooga-Ch...


In [878]:
# Model 9(2) + False tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2934798296.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.717034805269215,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.7076975303238172,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
2,Makki Paneer Pakora,"Corn Kernels 200 grams,Cottage Cheese 1 cup,Pa...",0.66734186659216,https://www.sanjeevkapoor.com/Recipe/Makki-Pan...
3,Paneer Pudina Kalimirch,"Cottage Cheese 400 grams,Fresh Mint Leaves Pas...",0.6623422283691598,https://www.sanjeevkapoor.com/Recipe/Paneer-Pu...
4,Mirchi Roll,"Bhavnagri Green Chiilies slit 4,Cottage Cheese...",0.6578884175957072,https://www.sanjeevkapoor.com/Recipe/Mirchi-Ro...


In [890]:
# Model 9(2) + True tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2669468391.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Paneer Pudina Kalimirch,"Cottage Cheese 400 grams,Fresh Mint Leaves Pas...",0.7470640329508313,https://www.sanjeevkapoor.com/Recipe/Paneer-Pu...
1,Murgh Ka Khurchan,Boneless chicken cut into thick strips 400 gra...,0.7232388336578865,https://www.sanjeevkapoor.com/Recipe/Murgh-Ka-...
2,Kadai Chicken - Sk Khazana,Boneless chicken cut into 2 inch cubes 500 gra...,0.7114588699891811,https://www.sanjeevkapoor.com/Recipe/Kadai-Chi...
3,Chicken Tikka Masala Bunny Chow,"Chicken tikka , chopped 1 cup,Burger buns 4,Oi...",0.7054527446583585,https://www.sanjeevkapoor.com/Recipe/Chicken-T...
4,Chicken Tawa Pulao,"Boneless chicken cut into cubes 400 grams,Cook...",0.6992107107045318,https://www.sanjeevkapoor.com/Recipe/Chicken-T...


In [902]:
# Model 10(2) + False tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2013326127.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.6988951414129844,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
1,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.6734969236705133,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
2,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.6696366126675249,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
3,Mushroom And Spinach Calzone,"Fresh Pizza Dough 450 grams,Cottage Cheese 16,...",0.620706466777713,https://www.sanjeevkapoor.com/Recipe/Mushroom-...
4,Makki Paneer Pakora,"Corn Kernels 200 grams,Cottage Cheese 1 cup,Pa...",0.5981485652929509,https://www.sanjeevkapoor.com/Recipe/Makki-Pan...


In [914]:
# Model 10(2) + True tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4131289077.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Paneer Bhurji,"Cottage Cheese 400 grams,Onions chopped 2 medi...",0.6761502851892767,https://www.sanjeevkapoor.com/Recipe/Paneer-Bh...
1,Chicken Kasoori Methi,Boneless Chicken cut into 1/2 inch cubes 400...,0.661608447774281,https://www.sanjeevkapoor.com/Recipe/Chicken-K...
2,Wadian Pulao,"Urad Dal Wadi roughly crushed 5-6,Basmati Rice...",0.6527241202559789,https://www.sanjeevkapoor.com/Recipe/Wadian-Pu...
3,Paneer Kadai - Sk Khazana,Cottage cheese (paneer) cut into 1 inch cubes ...,0.6523609689930965,https://www.sanjeevkapoor.com/Recipe/Paneer-Ka...
4,Paneer Pudina Kalimirch,"Cottage Cheese 400 grams,Fresh Mint Leaves Pas...",0.6513053828265264,https://www.sanjeevkapoor.com/Recipe/Paneer-Pu...


In [926]:
# Model 11(2) + False tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\535754922.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Makki Paneer Pakora,"Corn Kernels 200 grams,Cottage Cheese 1 cup,Pa...",0.8447378928536717,https://www.sanjeevkapoor.com/Recipe/Makki-Pan...
1,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.8211966680566805,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
2,Mirchi Roll,"Bhavnagri Green Chiilies slit 4,Cottage Cheese...",0.7994922359961802,https://www.sanjeevkapoor.com/Recipe/Mirchi-Ro...
3,Jain Hariyali Paneer Tikka,Paneer (cottage cheese) cut into 1 inch cubes ...,0.7793942934624469,https://www.sanjeevkapoor.com/Recipe/Jain-Hari...
4,Mushroom Tikka-Sk Khazana,"Button mushrooms 10-12,Oil for greasing,Hung y...",0.7720831056918647,https://www.sanjeevkapoor.com/Recipe/Mushroom-...


In [938]:
# Model 11(2) + True tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3489482770.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Laziz Tikka Masala,"Chicken,Yogurt 1 1/2 inch pieces 800 grams,Kas...",0.8352277086938676,https://www.sanjeevkapoor.com/Recipe/Laziz-Tik...
1,Chicken Khurchan,"Boneless chicken cut into strips 500 grams,Ghe...",0.8050737077621113,https://www.sanjeevkapoor.com/Recipe/Chicken-K...
2,Apricot Chicken On Skewers,"Boneless chicken leg, cut into 1 inch pieces 1...",0.7989460763480223,https://www.sanjeevkapoor.com/Recipe/Apricot-C...
3,Paneer Angara - Sk Khazana,Paneer (cottage cheese) cut into 1 inch cubes ...,0.7939832970374991,https://www.sanjeevkapoor.com/Recipe/Paneer-An...
4,Bhooga Chawar,"Basmati Rice soaked 1 1/2 cups,Onion 3 tablesp...",0.7892219708888564,https://www.sanjeevkapoor.com/Recipe/Bhooga-Ch...


In [951]:
# Model 12(2) + False tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4082322462.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7175717661721999,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.7048615299561273,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
2,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.6858231653324475,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
3,Lehsuni Palak-Sk Khazana,"Spinach (palak) cleaned 1 Large bunch,Garlic f...",0.6775608692387541,https://www.sanjeevkapoor.com/Recipe/Lehsuni-P...
4,Makki Paneer Pakora,"Corn Kernels 200 grams,Cottage Cheese 1 cup,Pa...",0.6707586709392878,https://www.sanjeevkapoor.com/Recipe/Makki-Pan...


In [963]:
# Model 12(2) + True tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2426173970.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Apricot Chicken On Skewers,"Boneless chicken leg, cut into 1 inch pieces 1...",0.7011921898960868,https://www.sanjeevkapoor.com/Recipe/Apricot-C...
1,Wadian Pulao,"Urad Dal Wadi roughly crushed 5-6,Basmati Rice...",0.6927922736038614,https://www.sanjeevkapoor.com/Recipe/Wadian-Pu...
2,Chicken Khurchan,"Boneless chicken cut into strips 500 grams,Ghe...",0.6896148131006877,https://www.sanjeevkapoor.com/Recipe/Chicken-K...
3,Laziz Tikka Masala,"Chicken,Yogurt 1 1/2 inch pieces 800 grams,Kas...",0.6885222977044593,https://www.sanjeevkapoor.com/Recipe/Laziz-Tik...
4,Paneer Pudina Kalimirch,"Cottage Cheese 400 grams,Fresh Mint Leaves Pas...",0.6879455613112454,https://www.sanjeevkapoor.com/Recipe/Paneer-Pu...


In [975]:
# Model 13(2) + False tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1986226641.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.7418308873262357,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
1,Chicken And Cheese,"Boneless Chicken Breasts 55 grams,Cottage Chee...",0.6826073084490416,https://www.sanjeevkapoor.com/Recipe/Chicken-A...
2,Eggplant And Paneer Kababs,"Eggplant 1 large,Cottage Cheese 100 grams,Oil ...",0.6457060407894684,https://www.sanjeevkapoor.com/Recipe/Eggplant-...
3,Makki Paneer Pakora,"Corn Kernels 200 grams,Cottage Cheese 1 cup,Pa...",0.6056526400993484,https://www.sanjeevkapoor.com/Recipe/Makki-Pan...
4,Paneer Snacker,"Cottage Cheese 150 grams,Salted Biscuits 1 tea...",0.5924621211312215,https://www.sanjeevkapoor.com/Recipe/Paneer-Sn...


In [987]:
# Model 13(2) + True tests
input = "onion, chicken, cottage cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4233235582.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Fried Onions For Biryani,"Onions 3 medium,Oil to deep fry",0.6276495850841286,https://www.sanjeevkapoor.com/Recipe/Fried-Oni...
1,Kunefe,"Sutarfeni 300 grams,Cottage cheese,Sugar 2 1/2...",0.6168956397913007,https://www.sanjeevkapoor.com/Recipe/Kunefe-Kh...
2,"Mushroom, Babycorn And Paneer Tawa Masala","Button mushrooms sliced 6,Babycorns,Babycorns ...",0.6101698495255177,"https://www.sanjeevkapoor.com/Recipe/Mushroom,..."
3,Wadian Pulao,"Urad Dal Wadi roughly crushed 5-6,Basmati Rice...",0.6078398673336357,https://www.sanjeevkapoor.com/Recipe/Wadian-Pu...
4,Tadka Salad,"Tomatoes chopped 1 kilogram,Onions to taste,On...",0.5803903557865392,https://www.sanjeevkapoor.com/Recipe/Tadka-Sal...


### Chana Dal

In [629]:
# Model 1 + False tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2863015847.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Kairi Chana Dal Achaar - Sk Khazana,"Raw mangoes medium, cut into 1 inch pieces, ma...",0.6388106349100222,https://www.sanjeevkapoor.com/Recipe/Kairi-Cha...
1,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.6250558842545519,https://www.sanjeevkapoor.com/Recipe/CABBAGE-C...
2,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.6250558842545519,https://www.sanjeevkapoor.com/Recipe/Cabbage-C...
3,Gajar Gobhi Aur Shalgam Ka Achar,"Carrots cut into 1 1/2 inch pieces 500 grams,...",0.5655600346192053,https://www.sanjeevkapoor.com/Recipe/Gajar-Gob...
4,Vadu Mango,"Baby mangoes 500 grams,Mustard powder 100 gram...",0.5575107692341611,https://www.sanjeevkapoor.com/Recipe/Vadu-Mang...


In [640]:
# Model 1 + True tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Kairi Chana Dal Achaar - Sk Khazana,"Raw mangoes medium, cut into 1 inch pieces, ma...",0.6224122234788841,https://www.sanjeevkapoor.com/Recipe/Kairi-Cha...
1,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.611864866191499,https://www.sanjeevkapoor.com/Recipe/CABBAGE-C...
2,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.611864866191499,https://www.sanjeevkapoor.com/Recipe/Cabbage-C...
3,Vadu Mango,"Baby mangoes 500 grams,Mustard powder 100 gram...",0.5525636020828638,https://www.sanjeevkapoor.com/Recipe/Vadu-Mang...
4,Gajar Gobhi Aur Shalgam Ka Achar,"Carrots cut into 1 1/2 inch pieces 500 grams,...",0.544749056688882,https://www.sanjeevkapoor.com/Recipe/Gajar-Gob...


In [811]:
# Model 7(1) + True tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url
0,Vegan Curd Rice- Sk Khazana,"Cashewnuts yogurt 1 cup,Brown rice boiled 2 cu...",0.7700926522341434,https://www.sanjeevkapoor.com/Recipe/VEGAN-CUR...
1,Medu Vada,"Split black gram skinless (dhuli urad dal) , s...",0.7235444002788296,https://www.sanjeevkapoor.com/Recipe/Medu-Vada...
2,Achari Prawn Rice,"Leftover cooked kolam rice 3 cups,Prawns (medi...",0.7176719710156729,https://www.sanjeevkapoor.com/Recipe/Achari-Pr...
3,Dosa,"Parboiled rice (ukda chawal) 2 3/4 cups,Split ...",0.7161945442105839,https://www.sanjeevkapoor.com/Recipe/Dosa.html
4,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.7133694974534647,https://www.sanjeevkapoor.com/Recipe/CABBAGE-C...


In [828]:
# Model 2 + False tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1633375022.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Guinness World Record Khichdi,"Rice soaked and drained 1 cup,Ghee 2 tablespo ...",0.9832095692310949,https://www.sanjeevkapoor.com/Recipe/Guinness-...
1,Stewed Watermelon,"Watermelon peeled, seeded and cut into cubes 1...",0.9827655000513761,https://www.sanjeevkapoor.com/Recipe/Stewed-Wa...
2,Bhajanee Thalipeeth - Sk Khazana,"Bhajanee flour,Whole wheat 1 cup,Split skinles...",0.9811728323650014,https://www.sanjeevkapoor.com/Recipe/Bhajanee-...
3,Apricot And Black Pepper Chutney,"Golden apricots chopped 35-40,Crushed black pe...",0.9803506824723924,https://www.sanjeevkapoor.com/Recipe/Apricot-A...
4,Anjeer Ki Chutney,"Figs 500 grams,Almonds 15-20,Oil 3 tablespoons...",0.9799829145229206,https://www.sanjeevkapoor.com/Recipe/Anjeer-Ki...


In [840]:
# Model 2 + True tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1992341500.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Green Tea With Fresh Ginger,"Green tea bags 3,Ginger grated 1 1/2 inch piec...",0.9829979834040192,https://www.sanjeevkapoor.com/Recipe/Green-Tea...
1,Anjeer Ki Chutney,"Figs 500 grams,Almonds 15-20,Oil 3 tablespoons...",0.9819453045997937,https://www.sanjeevkapoor.com/Recipe/Anjeer-Ki...
2,Stewed Watermelon,"Watermelon peeled, seeded and cut into cubes 1...",0.981762814505543,https://www.sanjeevkapoor.com/Recipe/Stewed-Wa...
3,Healthy Chiwda,"Corn flakes lightly roasted 1 cup,Oats roasted...",0.9807470250422908,https://www.sanjeevkapoor.com/Recipe/Healthy-C...
4,Pineapple Raisin Chutney,"Pineapple 1 medium,Ginger 1/2 inch piece,Sugar...",0.9807203346895321,https://www.sanjeevkapoor.com/Recipe/Pineapple...


In [853]:
# Model 8(2) + False tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3447942711.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Vegan Curd Rice- Sk Khazana,"Cashewnuts yogurt 1 cup,Brown rice boiled 2 cu...",0.9324116563752916,https://www.sanjeevkapoor.com/Recipe/VEGAN-CUR...
1,Kairi Chana Dal Achaar - Sk Khazana,"Raw mangoes medium, cut into 1 inch pieces, ma...",0.8640416304241122,https://www.sanjeevkapoor.com/Recipe/Kairi-Cha...
2,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.8570872677303893,https://www.sanjeevkapoor.com/Recipe/CABBAGE-C...
3,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.8570872677303893,https://www.sanjeevkapoor.com/Recipe/Cabbage-C...
4,Lemon Rice With Vegetables,"Lemon juice 2 tablespoons,Basmati rice soaked ...",0.8410800570757749,https://www.sanjeevkapoor.com/Recipe/Lemon-Ric...


In [867]:
# Model 8(2) + True tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4078852487.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Vegan Curd Rice- Sk Khazana,"Cashewnuts yogurt 1 cup,Brown rice boiled 2 cu...",0.9263639349541325,https://www.sanjeevkapoor.com/Recipe/VEGAN-CUR...
1,Kairi Chana Dal Achaar - Sk Khazana,"Raw mangoes medium, cut into 1 inch pieces, ma...",0.8642317496199149,https://www.sanjeevkapoor.com/Recipe/Kairi-Cha...
2,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.8484600946918496,https://www.sanjeevkapoor.com/Recipe/CABBAGE-C...
3,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.8484600946918496,https://www.sanjeevkapoor.com/Recipe/Cabbage-C...
4,Lemon Rice With Vegetables,"Lemon juice 2 tablespoons,Basmati rice soaked ...",0.8369603694599475,https://www.sanjeevkapoor.com/Recipe/Lemon-Ric...


In [879]:
# Model 9(2) + False tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2934798296.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Kairi Chana Dal Achaar - Sk Khazana,"Raw mangoes medium, cut into 1 inch pieces, ma...",0.7869787552973628,https://www.sanjeevkapoor.com/Recipe/Kairi-Cha...
1,Vegan Curd Rice- Sk Khazana,"Cashewnuts yogurt 1 cup,Brown rice boiled 2 cu...",0.7716022906916227,https://www.sanjeevkapoor.com/Recipe/VEGAN-CUR...
2,Achari Prawn Rice,"Leftover cooked kolam rice 3 cups,Prawns (medi...",0.7390755827739438,https://www.sanjeevkapoor.com/Recipe/Achari-Pr...
3,Carrot And Beetroot Ki Sabzi,"Carrot peeled and cut into diamonds 4,Beetroot...",0.735801239335899,https://www.sanjeevkapoor.com/Recipe/Carrot-an...
4,How To Make Dosa Batter,Parboiled rice soaked for 4-5 hours 2 3/4 cups...,0.7303296300839682,https://www.sanjeevkapoor.com/Recipe/How-to-ma...


In [891]:
# Model 9(2) + True tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2669468391.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Kairi Chana Dal Achaar - Sk Khazana,"Raw mangoes medium, cut into 1 inch pieces, ma...",0.797116918507553,https://www.sanjeevkapoor.com/Recipe/Kairi-Cha...
1,Carrot And Beetroot Ki Sabzi,"Carrot peeled and cut into diamonds 4,Beetroot...",0.7786749679202573,https://www.sanjeevkapoor.com/Recipe/Carrot-an...
2,Vegan Curd Rice- Sk Khazana,"Cashewnuts yogurt 1 cup,Brown rice boiled 2 cu...",0.775379349018537,https://www.sanjeevkapoor.com/Recipe/VEGAN-CUR...
3,Idli Chutney,"Parboiled rice 1 cup,Raw rice 1/4 cup,Skinless...",0.7348241731738732,https://www.sanjeevkapoor.com/Recipe/Idli-Chut...
4,How To Make Dosa Batter,Parboiled rice soaked for 4-5 hours 2 3/4 cups...,0.7303296340379891,https://www.sanjeevkapoor.com/Recipe/How-to-ma...


In [903]:
# Model 10(2) + False tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2013326127.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Kairi Chana Dal Achaar - Sk Khazana,"Raw mangoes medium, cut into 1 inch pieces, ma...",0.8044637057398888,https://www.sanjeevkapoor.com/Recipe/Kairi-Cha...
1,Carrot And Beetroot Ki Sabzi,"Carrot peeled and cut into diamonds 4,Beetroot...",0.7996735477272127,https://www.sanjeevkapoor.com/Recipe/Carrot-an...
2,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.7805514063103791,https://www.sanjeevkapoor.com/Recipe/CABBAGE-C...
3,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.7805514063103791,https://www.sanjeevkapoor.com/Recipe/Cabbage-C...
4,Lemon Rice With Vegetables,"Lemon juice 2 tablespoons,Basmati rice soaked ...",0.7755678509016097,https://www.sanjeevkapoor.com/Recipe/Lemon-Ric...


In [915]:
# Model 10(2) + True tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4131289077.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Kairi Chana Dal Achaar - Sk Khazana,"Raw mangoes medium, cut into 1 inch pieces, ma...",0.810134539693582,https://www.sanjeevkapoor.com/Recipe/Kairi-Cha...
1,Carrot And Beetroot Ki Sabzi,"Carrot peeled and cut into diamonds 4,Beetroot...",0.7945196812262572,https://www.sanjeevkapoor.com/Recipe/Carrot-an...
2,Tamarind And Mustard Rice,"Rice cooked 2 cups,Tamarind pulp 4 tablespoons...",0.7754761653221747,https://www.sanjeevkapoor.com/Recipe/Tamarind-...
3,Lemon Rice With Vegetables,"Lemon juice 2 tablespoons,Basmati rice soaked ...",0.7736870106676037,https://www.sanjeevkapoor.com/Recipe/Lemon-Ric...
4,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.7730775392475053,https://www.sanjeevkapoor.com/Recipe/CABBAGE-C...


In [927]:
# Model 11(2) + False tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\535754922.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.8035761368941012,https://www.sanjeevkapoor.com/Recipe/CABBAGE-C...
1,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.8035761368941012,https://www.sanjeevkapoor.com/Recipe/Cabbage-C...
2,Barley Idli,"Barley 1/4 cup,Parboiled Rice 1/4 cup,Split bl...",0.7568095573037699,https://www.sanjeevkapoor.com/Recipe/Barley-Id...
3,"Thalipeeth, The Yellow Chilli Style","Wheat 1/2 cup,Jowar 1/2 cup,Bajra 1/2 cu...",0.718901812950846,https://www.sanjeevkapoor.com/Recipe/Thalipeet...
4,Carrot And Beetroot Ki Sabzi,"Carrot peeled and cut into diamonds 4,Beetroot...",0.7167856472571729,https://www.sanjeevkapoor.com/Recipe/Carrot-an...


In [939]:
# Model 11(2) + True tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3489482770.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.7982216430796856,https://www.sanjeevkapoor.com/Recipe/CABBAGE-C...
1,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.7982216430796856,https://www.sanjeevkapoor.com/Recipe/Cabbage-C...
2,Barley Idli,"Barley 1/4 cup,Parboiled Rice 1/4 cup,Split bl...",0.754723228806667,https://www.sanjeevkapoor.com/Recipe/Barley-Id...
3,Carrot And Beetroot Ki Sabzi,"Carrot peeled and cut into diamonds 4,Beetroot...",0.747659204511796,https://www.sanjeevkapoor.com/Recipe/Carrot-an...
4,Lemon Rice With Vegetables,"Lemon juice 2 tablespoons,Basmati rice soaked ...",0.7283316873573473,https://www.sanjeevkapoor.com/Recipe/Lemon-Ric...


In [952]:
# Model 12(2) + False tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4082322462.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.7576373274635757,https://www.sanjeevkapoor.com/Recipe/CABBAGE-C...
1,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.7576373274635757,https://www.sanjeevkapoor.com/Recipe/Cabbage-C...
2,Lemon Rice With Vegetables,"Lemon juice 2 tablespoons,Basmati rice soaked ...",0.7531668548820399,https://www.sanjeevkapoor.com/Recipe/Lemon-Ric...
3,Benne Dosa - Sk Khazana,"Butter as required,Short grain rice washed, so...",0.7412479074073223,https://www.sanjeevkapoor.com/Recipe/Benne-Dos...
4,Curd Jowar,"Sorghum (jowar) soaked overnight 1 cup,Yogurt ...",0.7402600608257273,https://www.sanjeevkapoor.com/Recipe/Curd-Jowa...


In [964]:
# Model 12(2) + True tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2426173970.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.7546574569426876,https://www.sanjeevkapoor.com/Recipe/CABBAGE-C...
1,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.7546574569426876,https://www.sanjeevkapoor.com/Recipe/Cabbage-C...
2,Lemon Rice With Vegetables,"Lemon juice 2 tablespoons,Basmati rice soaked ...",0.7431005668800977,https://www.sanjeevkapoor.com/Recipe/Lemon-Ric...
3,Curd Jowar,"Sorghum (jowar) soaked overnight 1 cup,Yogurt ...",0.7401131888559493,https://www.sanjeevkapoor.com/Recipe/Curd-Jowa...
4,Benne Dosa - Sk Khazana,"Butter as required,Short grain rice washed, so...",0.7391332449593968,https://www.sanjeevkapoor.com/Recipe/Benne-Dos...


In [976]:
# Model 13(2) + False tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1986226641.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Kairi Chana Dal Achaar - Sk Khazana,"Raw mangoes medium, cut into 1 inch pieces, ma...",0.6282860291481563,https://www.sanjeevkapoor.com/Recipe/Kairi-Cha...
1,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.5952552378813536,https://www.sanjeevkapoor.com/Recipe/CABBAGE-C...
2,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.5952552378813536,https://www.sanjeevkapoor.com/Recipe/Cabbage-C...
3,Achari Prawn Rice,"Leftover cooked kolam rice 3 cups,Prawns (medi...",0.5515642813373253,https://www.sanjeevkapoor.com/Recipe/Achari-Pr...
4,Mysore Masala Dosa,"Dosa batter 3 cups,Potato bhaji 1 1/2 cups,Oil...",0.550881039075219,https://www.sanjeevkapoor.com/Recipe/Mysore-Ma...


In [988]:
# Model 13(2) + True tests
input = "chana dal"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4233235582.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Kairi Chana Dal Achaar - Sk Khazana,"Raw mangoes medium, cut into 1 inch pieces, ma...",0.6136724632901274,https://www.sanjeevkapoor.com/Recipe/Kairi-Cha...
1,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.5709228241980848,https://www.sanjeevkapoor.com/Recipe/CABBAGE-C...
2,Cabbage Chana Dal,"Cabbage shredded 500 grams,Chana Dal soaked 1...",0.5709228241980848,https://www.sanjeevkapoor.com/Recipe/Cabbage-C...
3,Raw Mango Rice-Cook Smart,"Raw mango peeled and grated 3 tablespoons,Basm...",0.5404210508818642,https://www.sanjeevkapoor.com/Recipe/Raw-Mango...
4,Secret Pizza,"Brown Bread Slices 4,Pizza Pasta Sauce 1/2 cup...",0.5281684229100878,https://www.sanjeevkapoor.com/Recipe/Secret-Pi...


### split bengal gram

In [630]:
# Model 1 + False tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2863015847.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.7469856621345742,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
1,Masala Vadai,"Split Bengal Gram 3/4 cup,Split Pigeon Pea 1/2...",0.6449693326197596,https://www.sanjeevkapoor.com/Recipe/Masala-Va...
2,Chana Dal Tikki-Cook Smart,"Split Bengal gram soaked for 2 hours 1 cup,Oil...",0.6444003545911783,https://www.sanjeevkapoor.com/Recipe/Chana-Dal...
3,Soya Shammi Kebab,"Soya Nuggets soaked in hot water 2 cups,Split ...",0.6262336336562184,https://www.sanjeevkapoor.com/Recipe/Soya-Sham...
4,Vade,"Rice 1 kilogram,Split bengal gram 250 grams,Sp...",0.6183465479705541,https://www.sanjeevkapoor.com/Recipe/Vade-Khaa...


In [641]:
# Model 1 + True tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.746985658751079,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
1,Vade,"Rice 1 kilogram,Split bengal gram 250 grams,Sp...",0.6088171536841818,https://www.sanjeevkapoor.com/Recipe/Vade-Khaa...
2,Soya Shammi Kebab,"Soya Nuggets soaked in hot water 2 cups,Split ...",0.5896273926064013,https://www.sanjeevkapoor.com/Recipe/Soya-Sham...
3,Teen Dal Ke Dahi Bhalle,"Split green gram 3/4 cup,Split bengal gram 1/4...",0.5871550381545549,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...
4,Masala Vadai,"Split Bengal Gram 3/4 cup,Split Pigeon Pea 1/2...",0.5733797764872356,https://www.sanjeevkapoor.com/Recipe/Masala-Va...


In [812]:
# Model 7(1) + True tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url
0,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.807637543373289,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
1,Healthy Muffins,"Rice soaked and drained 1/2 cup,Split pigeon p...",0.7636352594686087,https://www.sanjeevkapoor.com/Recipe/Healthy-M...
2,Teen Dal Dosa,"Split pigeon pea (toor dal/arhar dal) 1/3 cup,...",0.7633152063861786,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...
3,Vade,"Rice 1 kilogram,Split bengal gram 250 grams,Sp...",0.7510300692693499,https://www.sanjeevkapoor.com/Recipe/Vade-Khaa...
4,Teen Dal Ke Dahi Bhalle,"Split green gram 3/4 cup,Split bengal gram 1/4...",0.7437010760743273,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...


In [829]:
# Model 2 + False tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1633375022.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.9972094528388756,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
1,Dosa Recipe - Sk Khazana,Parboiled rice (ukda chawal) soaked for 8-10 h...,0.9970648408385965,https://www.sanjeevkapoor.com/Recipe/Dosa-Reci...
2,Vegetable Vermicelli Uttapam - Sk Khazana,"Vermicelli roasted 3/4 cup,green peas boiled a...",0.9965632230694473,https://www.sanjeevkapoor.com/Recipe/Vegetable...
3,Tri Coloured Dhokla,"Rice 1/2 cup,Split black gram skinless (dhuli ...",0.9959927265760932,https://www.sanjeevkapoor.com/Recipe/Tri-Colou...
4,Instant Rawa Dosa Waffle - Sk Khazana,"Semolina (rawa/suji) 1 1/2 cups,Salt to taste,...",0.9954145689072941,https://www.sanjeevkapoor.com/Recipe/Instant-R...


In [841]:
# Model 2 + True tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1992341500.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Dosa Recipe - Sk Khazana,Parboiled rice (ukda chawal) soaked for 8-10 h...,0.997211115735539,https://www.sanjeevkapoor.com/Recipe/Dosa-Reci...
1,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.9972094528892774,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
2,Instant Rawa Dosa Waffle - Sk Khazana,"Semolina (rawa/suji) 1 1/2 cups,Salt to taste,...",0.9967305758986531,https://www.sanjeevkapoor.com/Recipe/Instant-R...
3,Tri Coloured Dhokla,"Rice 1/2 cup,Split black gram skinless (dhuli ...",0.9963276959580134,https://www.sanjeevkapoor.com/Recipe/Tri-Colou...
4,Shallow-Fried Fish With Oatmeal,"Fish fillet boneless 2,Oatmeal 1 cup,Lemon jui...",0.9960489027174708,https://www.sanjeevkapoor.com/Recipe/Shallow-F...


In [854]:
# Model 8(2) + False tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3447942711.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.8559264533274814,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
1,Teen Dal Dosa,"Split pigeon pea (toor dal/arhar dal) 1/3 cup,...",0.8413816701523203,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...
2,Turai Moong Dal - Skk,Ridge gourd (turai) scraped and roughly choppe...,0.8306098328334799,https://www.sanjeevkapoor.com/Recipe/Turai-Moo...
3,Arbi Ke Patton Ki Subzi,"Colocasia Leaves finely chopped 8,Split Bengal...",0.8226657900124215,https://www.sanjeevkapoor.com/Recipe/Arbi-Ke-P...
4,Teen Dal Ke Dahi Bhalle,"Split green gram 3/4 cup,Split bengal gram 1/4...",0.8180302737342477,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...


In [868]:
# Model 8(2) + True tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4078852487.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.8559264609890178,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
1,Teen Dal Dosa,"Split pigeon pea (toor dal/arhar dal) 1/3 cup,...",0.8323419368055142,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...
2,Handvo Cups With Paneer Topping,"Rice 1 cup,Split Pigeon Pea 1/2 cup,Split gree...",0.8207364347930091,https://www.sanjeevkapoor.com/Recipe/Handvo-Cu...
3,Teen Dal Ke Dahi Bhalle,"Split green gram 3/4 cup,Split bengal gram 1/4...",0.818030270821319,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...
4,Arbi Ke Patton Ki Subzi,"Colocasia Leaves finely chopped 8,Split Bengal...",0.806992711918955,https://www.sanjeevkapoor.com/Recipe/Arbi-Ke-P...


In [880]:
# Model 9(2) + False tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2934798296.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.8085272213649111,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
1,Paruppu Urundai Kozhambu,"Toor Dal 1 cup,Split Bengal Gram 1/4 cup,Tamar...",0.7398910639146684,https://www.sanjeevkapoor.com/Recipe/Paruppu-U...
2,Teen Dal Dosa,"Split pigeon pea (toor dal/arhar dal) 1/3 cup,...",0.7383744308147382,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...
3,Arbi Ke Patton Ki Subzi,"Colocasia Leaves finely chopped 8,Split Bengal...",0.7199204841476661,https://www.sanjeevkapoor.com/Recipe/Arbi-Ke-P...
4,Chana Dal Tikki-Cook Smart,"Split Bengal gram soaked for 2 hours 1 cup,Oil...",0.7169910604457974,https://www.sanjeevkapoor.com/Recipe/Chana-Dal...


In [892]:
# Model 9(2) + True tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2669468391.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.808527227723315,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
1,Teen Dal Dosa,"Split pigeon pea (toor dal/arhar dal) 1/3 cup,...",0.7313633331293938,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...
2,Paruppu Urundai Kozhambu,"Toor Dal 1 cup,Split Bengal Gram 1/4 cup,Tamar...",0.7197799127439661,https://www.sanjeevkapoor.com/Recipe/Paruppu-U...
3,Teen Dal Ke Dahi Bhalle,"Split green gram 3/4 cup,Split bengal gram 1/4...",0.7162013953050705,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...
4,Dosa Recipe - Sk Khazana,Parboiled rice (ukda chawal) soaked for 8-10 h...,0.7057905827667297,https://www.sanjeevkapoor.com/Recipe/Dosa-Reci...


In [904]:
# Model 10(2) + False tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2013326127.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Teen Dal Dosa,"Split pigeon pea (toor dal/arhar dal) 1/3 cup,...",0.7354437905067428,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...
1,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.7329907939789867,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
2,Masala Vadai,"Split Bengal Gram 3/4 cup,Split Pigeon Pea 1/2...",0.7184901413859527,https://www.sanjeevkapoor.com/Recipe/Masala-Va...
3,Vade,"Rice 1 kilogram,Split bengal gram 250 grams,Sp...",0.7145321229223034,https://www.sanjeevkapoor.com/Recipe/Vade-Khaa...
4,Palak Mudda Bhaji,"Fresh spinach , shredded 1 medium bunches,Spli...",0.7126479394932619,https://www.sanjeevkapoor.com/Recipe/Palak-Mud...


In [916]:
# Model 10(2) + True tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4131289077.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.7329907992683622,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
1,Vade,"Rice 1 kilogram,Split bengal gram 250 grams,Sp...",0.6972263366064735,https://www.sanjeevkapoor.com/Recipe/Vade-Khaa...
2,Teen Dal Dosa,"Split pigeon pea (toor dal/arhar dal) 1/3 cup,...",0.6900148978079096,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...
3,Palak Mudda Bhaji,"Fresh spinach , shredded 1 medium bunches,Spli...",0.6836343679402714,https://www.sanjeevkapoor.com/Recipe/Palak-Mud...
4,Sesame Dal,"White sesame seeds 1 teaspoon,Sesame seeds (bl...",0.6747781952329424,https://www.sanjeevkapoor.com/Recipe/Sesame-Da...


In [928]:
# Model 11(2) + False tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\535754922.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.8976735734591327,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
1,Teen Dal Dosa,"Split pigeon pea (toor dal/arhar dal) 1/3 cup,...",0.8546352275411744,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...
2,Pachadi -Sk Khazana,"Yogurt whisked 1 1/2 cups,Tomatoes medium, fin...",0.836825438041811,https://www.sanjeevkapoor.com/Recipe/Pachadi--...
3,Keerai Chutney,"Spinach 1 medium,Onion 1 medium,Tomato 1 mediu...",0.8353863422365863,https://www.sanjeevkapoor.com/Recipe/Keerai-Ch...
4,How To Make Sprouts,Whole green grams (sabut moong) as required,0.8339187848835674,https://www.sanjeevkapoor.com/Recipe/How-to-ma...


In [940]:
# Model 11(2) + True tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3489482770.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.8976735715480242,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
1,Handvo Cups With Paneer Topping,"Rice 1 cup,Split Pigeon Pea 1/2 cup,Split gree...",0.8417563694901153,https://www.sanjeevkapoor.com/Recipe/Handvo-Cu...
2,Pachadi -Sk Khazana,"Yogurt whisked 1 1/2 cups,Tomatoes medium, fin...",0.8349071237247533,https://www.sanjeevkapoor.com/Recipe/Pachadi--...
3,How To Make Sprouts,Whole green grams (sabut moong) as required,0.83391878367268,https://www.sanjeevkapoor.com/Recipe/How-to-ma...
4,Dosa Recipe - Sk Khazana,Parboiled rice (ukda chawal) soaked for 8-10 h...,0.8310877309680275,https://www.sanjeevkapoor.com/Recipe/Dosa-Reci...


In [953]:
# Model 12(2) + False tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4082322462.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Teen Dal Dosa,"Split pigeon pea (toor dal/arhar dal) 1/3 cup,...",0.8624702383605375,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...
1,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.8296180151080057,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
2,Mixture,"Split Bengal gram (chana dal) 3 cups,For Sev,S...",0.8265327355560981,https://www.sanjeevkapoor.com/Recipe/Mixture-....
3,Sesame Dal,"White sesame seeds 1 teaspoon,Sesame seeds (bl...",0.8143300013148349,https://www.sanjeevkapoor.com/Recipe/Sesame-Da...
4,Parappu Urundai Kozhambhu,Split pigeon pea (toor dal/arhar dal) 1 1/2 cu...,0.804688300392278,https://www.sanjeevkapoor.com/Recipe/Parappu-U...


In [965]:
# Model 12(2) + True tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2426173970.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Teen Dal Dosa,"Split pigeon pea (toor dal/arhar dal) 1/3 cup,...",0.843303313084221,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...
1,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.8296180166522121,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
2,Mixture,"Split Bengal gram (chana dal) 3 cups,For Sev,S...",0.8248040308557576,https://www.sanjeevkapoor.com/Recipe/Mixture-....
3,Sesame Dal,"White sesame seeds 1 teaspoon,Sesame seeds (bl...",0.8245332526309779,https://www.sanjeevkapoor.com/Recipe/Sesame-Da...
4,Paruppu Urundai Kozhambu,"Toor Dal 1 cup,Split Bengal Gram 1/4 cup,Tamar...",0.7933758960839715,https://www.sanjeevkapoor.com/Recipe/Paruppu-U...


In [977]:
# Model 13(2) + False tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1986226641.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.7901258317848084,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
1,Vade,"Rice 1 kilogram,Split bengal gram 250 grams,Sp...",0.7424151484825374,https://www.sanjeevkapoor.com/Recipe/Vade-Khaa...
2,Kanchipuram Idli-Sk Khazana,"Rice soaked for 3-4 hours 1 cup,Split skinless...",0.7125273839227588,https://www.sanjeevkapoor.com/Recipe/Kanchipur...
3,Masala Vadai,"Split Bengal Gram 3/4 cup,Split Pigeon Pea 1/2...",0.7117990634945252,https://www.sanjeevkapoor.com/Recipe/Masala-Va...
4,Paruppu Urundai Kozhambu,"Toor Dal 1 cup,Split Bengal Gram 1/4 cup,Tamar...",0.7053714480006525,https://www.sanjeevkapoor.com/Recipe/Paruppu-U...


In [989]:
# Model 13(2) + True tests
input = "split bengal gram"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4233235582.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Steamed Dosa,"Parboiled rice soaked 2 3/4 cups,Split bengal ...",0.7901258311685843,https://www.sanjeevkapoor.com/Recipe/Steamed-D...
1,Vade,"Rice 1 kilogram,Split bengal gram 250 grams,Sp...",0.7229518118740696,https://www.sanjeevkapoor.com/Recipe/Vade-Khaa...
2,Kanchipuram Idli-Sk Khazana,"Rice soaked for 3-4 hours 1 cup,Split skinless...",0.6901452283513771,https://www.sanjeevkapoor.com/Recipe/Kanchipur...
3,Teen Dal Dosa,"Split pigeon pea (toor dal/arhar dal) 1/3 cup,...",0.6869517622721082,https://www.sanjeevkapoor.com/Recipe/Teen-Dal-...
4,Palak Mudda Bhaji,"Fresh spinach , shredded 1 medium bunches,Spli...",0.6839034726517763,https://www.sanjeevkapoor.com/Recipe/Palak-Mud...


### Watermelon

In [631]:
# Model 1 + False tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2863015847.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.8883140104398094,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Watermelon Lollipop,"Medium watermelon 1/2,Fresh or tinned orange j...",0.8260477143240947,https://www.sanjeevkapoor.com/Recipe/Watermelo...
2,Watermelon Cooler With Basil Seeds,"Watermelon 1 medium,Basil seeds (takmaria or s...",0.7896635230532589,https://www.sanjeevkapoor.com/Recipe/Watermelo...
3,Watermelon Punch - Sk Khazana,Medium watermelon peeled and cut into strips 1...,0.7856674294069488,https://www.sanjeevkapoor.com/Recipe/Watermelo...
4,Rose Watermelon Smoothie,"Rose syrup 4 tablespoons,Watermelon , peeled, ...",0.780679075019595,https://www.sanjeevkapoor.com/Recipe/Rose-Wate...


In [642]:
# Model 1 + True tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.8698078069674545,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Watermelon Cooler With Basil Seeds,"Watermelon 1 medium,Basil seeds (takmaria or s...",0.8680352291387622,https://www.sanjeevkapoor.com/Recipe/Watermelo...
2,Watermelon Lollipop,"Medium watermelon 1/2,Fresh or tinned orange j...",0.8473641136049624,https://www.sanjeevkapoor.com/Recipe/Watermelo...
3,Watermelon Punch - Sk Khazana,Medium watermelon peeled and cut into strips 1...,0.8207624512845783,https://www.sanjeevkapoor.com/Recipe/Watermelo...
4,Fruit Ball Float,"Watermelon 1 small,Musk melon 1 small,Orange j...",0.7955616560263815,https://www.sanjeevkapoor.com/Recipe/Fruit-Bal...


In [813]:
# Model 7(1) + True tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url
0,Watermelon Punch - Sk Khazana,Medium watermelon peeled and cut into strips 1...,0.9037657822955275,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.9037552581227191,https://www.sanjeevkapoor.com/Recipe/Watermelo...
2,Watermelon Slush,"Watermelon juice, put in an ice tray and froze...",0.8591890697730966,https://www.sanjeevkapoor.com/Recipe/Watermelo...
3,Watermelon Cooler With Basil Seeds,"Watermelon 1 medium,Basil seeds (takmaria or s...",0.8533433136795217,https://www.sanjeevkapoor.com/Recipe/Watermelo...
4,Watermelon Mojito-Sk Khazana,"Medium watermelon 1,Lemons 2,Fresh mint leaves...",0.8462992204412789,https://www.sanjeevkapoor.com/Recipe/Watermelo...


In [830]:
# Model 2 + False tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1633375022.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Spinach And Ricotta Ravioli - Sk Khazana,"Spinach leaves blanched and chopped 1 cup,Rico...",0.9963390881673833,https://www.sanjeevkapoor.com/Recipe/Spinach-a...
1,Fruit Kabab - Sk Khazana,"Medium apple 1,Medium pear 1,Pineapple cubes 1...",0.9962090873027543,https://www.sanjeevkapoor.com/Recipe/Fruit-Kab...
2,Pavlova - Sk Khazana,"Eggs 3,Lemon 1/4,Castor sugar (caster sugar) 1...",0.9960024312025397,https://www.sanjeevkapoor.com/Recipe/Pavlova--...
3,Gluten Free Bread - Sk Khazana,"Flax seeds powder 3 tablespoons,Fresh yeast 1 ...",0.9957293966840073,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...
4,Anti Flu Drink,"Pomegranate (anar) juice 1 cup,Pomegranate (an...",0.9956858526271593,https://www.sanjeevkapoor.com/Recipe/Anti-Flu-...


In [842]:
# Model 2 + True tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1992341500.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Spinach And Ricotta Ravioli - Sk Khazana,"Spinach leaves blanched and chopped 1 cup,Rico...",0.9971171331967237,https://www.sanjeevkapoor.com/Recipe/Spinach-a...
1,Anti Flu Drink,"Pomegranate (anar) juice 1 cup,Pomegranate (an...",0.9959686296952669,https://www.sanjeevkapoor.com/Recipe/Anti-Flu-...
2,Prune And Pear Crumble,"Prunes 10,Pears 3,Sugar 1/4 cup + tablespoons,...",0.9959560924690607,https://www.sanjeevkapoor.com/Recipe/Prune-and...
3,Cinderella,"Apple finely chopped 1 medium,Pear finely chop...",0.9958711115057141,https://www.sanjeevkapoor.com/Recipe/Cinderell...
4,Gluten Free Bread - Sk Khazana,"Flax seeds powder 3 tablespoons,Fresh yeast 1 ...",0.9957356718054156,https://www.sanjeevkapoor.com/Recipe/Gluten-Fr...


In [855]:
# Model 8(2) + False tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3447942711.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.9153582083944853,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Watermelon Punch - Sk Khazana,Medium watermelon peeled and cut into strips 1...,0.8803946105810292,https://www.sanjeevkapoor.com/Recipe/Watermelo...
2,Watermelon Mojito-Sk Khazana,"Medium watermelon 1,Lemons 2,Fresh mint leaves...",0.8698092431248312,https://www.sanjeevkapoor.com/Recipe/Watermelo...
3,Rose Watermelon Smoothie,"Rose syrup 4 tablespoons,Watermelon , peeled, ...",0.8517207352364904,https://www.sanjeevkapoor.com/Recipe/Rose-Wate...
4,Fruit Ball Float,"Watermelon 1 small,Musk melon 1 small,Orange j...",0.8456543474409169,https://www.sanjeevkapoor.com/Recipe/Fruit-Bal...


In [869]:
# Model 8(2) + True tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4078852487.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Watermelon Punch - Sk Khazana,Medium watermelon peeled and cut into strips 1...,0.9006514955584344,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Watermelon Mojito-Sk Khazana,"Medium watermelon 1,Lemons 2,Fresh mint leaves...",0.9003708522985592,https://www.sanjeevkapoor.com/Recipe/Watermelo...
2,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.8969940117644933,https://www.sanjeevkapoor.com/Recipe/Watermelo...
3,Watermelon Slush,"Watermelon juice, put in an ice tray and froze...",0.8834913175902587,https://www.sanjeevkapoor.com/Recipe/Watermelo...
4,Melon Pineapple And Grape Cocktail,"Watermelon 1/2 small,Pineapple 1-2 small,Green...",0.8642446434246471,https://www.sanjeevkapoor.com/Recipe/Melon-Pin...


In [881]:
# Model 9(2) + False tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2934798296.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.8940373809494293,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Watermelon Punch - Sk Khazana,Medium watermelon peeled and cut into strips 1...,0.8599221287139787,https://www.sanjeevkapoor.com/Recipe/Watermelo...
2,Watermelon Slush,"Watermelon juice, put in an ice tray and froze...",0.8212101665206358,https://www.sanjeevkapoor.com/Recipe/Watermelo...
3,Fruit Ball Float,"Watermelon 1 small,Musk melon 1 small,Orange j...",0.7984021272507993,https://www.sanjeevkapoor.com/Recipe/Fruit-Bal...
4,Watermelon Mojito-Sk Khazana,"Medium watermelon 1,Lemons 2,Fresh mint leaves...",0.7939101278428599,https://www.sanjeevkapoor.com/Recipe/Watermelo...


In [893]:
# Model 9(2) + True tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2669468391.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.885337499793581,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Watermelon Punch - Sk Khazana,Medium watermelon peeled and cut into strips 1...,0.8831753774361315,https://www.sanjeevkapoor.com/Recipe/Watermelo...
2,Watermelon Slush,"Watermelon juice, put in an ice tray and froze...",0.8627506957610791,https://www.sanjeevkapoor.com/Recipe/Watermelo...
3,Watermelon Mojito-Sk Khazana,"Medium watermelon 1,Lemons 2,Fresh mint leaves...",0.8415315846710527,https://www.sanjeevkapoor.com/Recipe/Watermelo...
4,Muskmelon Slush,"Medium watermelon peeled, seeded and cut into ...",0.8392622674829083,https://www.sanjeevkapoor.com/Recipe/Muskmelon...


In [905]:
# Model 10(2) + False tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2013326127.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.8744432985702209,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Rose Watermelon Smoothie,"Rose syrup 4 tablespoons,Watermelon , peeled, ...",0.7621507568626066,https://www.sanjeevkapoor.com/Recipe/Rose-Wate...
2,Watermelon Lollipop,"Medium watermelon 1/2,Fresh or tinned orange j...",0.7585908084689191,https://www.sanjeevkapoor.com/Recipe/Watermelo...
3,Watermelon Cooler With Basil Seeds,"Watermelon 1 medium,Basil seeds (takmaria or s...",0.7552459259163363,https://www.sanjeevkapoor.com/Recipe/Watermelo...
4,Watermelon Punch - Sk Khazana,Medium watermelon peeled and cut into strips 1...,0.7434825555439715,https://www.sanjeevkapoor.com/Recipe/Watermelo...


In [917]:
# Model 10(2) + True tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4131289077.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.8676859555016927,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Watermelon Cooler With Basil Seeds,"Watermelon 1 medium,Basil seeds (takmaria or s...",0.8497916011025242,https://www.sanjeevkapoor.com/Recipe/Watermelo...
2,Watermelon Mojito-Sk Khazana,"Medium watermelon 1,Lemons 2,Fresh mint leaves...",0.8064366453311038,https://www.sanjeevkapoor.com/Recipe/Watermelo...
3,Watermelon Slush,"Watermelon juice, put in an ice tray and froze...",0.7857485920646703,https://www.sanjeevkapoor.com/Recipe/Watermelo...
4,Watermelon Punch - Sk Khazana,Medium watermelon peeled and cut into strips 1...,0.7844535113554021,https://www.sanjeevkapoor.com/Recipe/Watermelo...


In [929]:
# Model 11(2) + False tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\535754922.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.9144631910345151,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Watermelon Punch - Sk Khazana,Medium watermelon peeled and cut into strips 1...,0.891316415263776,https://www.sanjeevkapoor.com/Recipe/Watermelo...
2,Fruit Ball Float,"Watermelon 1 small,Musk melon 1 small,Orange j...",0.8861314968981431,https://www.sanjeevkapoor.com/Recipe/Fruit-Bal...
3,Watermelon Mojito-Sk Khazana,"Medium watermelon 1,Lemons 2,Fresh mint leaves...",0.8699268368798574,https://www.sanjeevkapoor.com/Recipe/Watermelo...
4,Margarita Pops,"Powdered sugar 4 teaspoons,Lemon juice 4 teasp...",0.8476566226457733,https://www.sanjeevkapoor.com/Recipe/Margarita...


In [941]:
# Model 11(2) + True tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3489482770.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Watermelon Punch - Sk Khazana,Medium watermelon peeled and cut into strips 1...,0.9095482781010619,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Watermelon Mojito-Sk Khazana,"Medium watermelon 1,Lemons 2,Fresh mint leaves...",0.9005980605632886,https://www.sanjeevkapoor.com/Recipe/Watermelo...
2,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.8989492869916771,https://www.sanjeevkapoor.com/Recipe/Watermelo...
3,Fruit Ball Float,"Watermelon 1 small,Musk melon 1 small,Orange j...",0.8931971770187972,https://www.sanjeevkapoor.com/Recipe/Fruit-Bal...
4,Watermelon Cooler With Basil Seeds,"Watermelon 1 medium,Basil seeds (takmaria or s...",0.8879807121261998,https://www.sanjeevkapoor.com/Recipe/Watermelo...


In [954]:
# Model 12(2) + False tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4082322462.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.8927953611597235,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Watermelon Punch - Sk Khazana,Medium watermelon peeled and cut into strips 1...,0.8701392089175881,https://www.sanjeevkapoor.com/Recipe/Watermelo...
2,Watermelon Fruit Basket,"Medium watermelon 1,Medium green apple cored a...",0.8293971418637069,https://www.sanjeevkapoor.com/Recipe/Watermelo...
3,Rose Watermelon Smoothie,"Rose syrup 4 tablespoons,Watermelon , peeled, ...",0.8050499826016679,https://www.sanjeevkapoor.com/Recipe/Rose-Wate...
4,Leafy Salad In Pomegranate Dressing,"Medium cucumber peeled 1,Medium apple 1/2,Medi...",0.8040484257905847,https://www.sanjeevkapoor.com/Recipe/Leafy-Sal...


In [966]:
# Model 12(2) + True tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2426173970.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Watermelon Punch - Sk Khazana,Medium watermelon peeled and cut into strips 1...,0.891600463113409,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.8807630579876119,https://www.sanjeevkapoor.com/Recipe/Watermelo...
2,Watermelon Fruit Basket,"Medium watermelon 1,Medium green apple cored a...",0.8418454662310814,https://www.sanjeevkapoor.com/Recipe/Watermelo...
3,Watermelon Mojito-Sk Khazana,"Medium watermelon 1,Lemons 2,Fresh mint leaves...",0.8332170417732059,https://www.sanjeevkapoor.com/Recipe/Watermelo...
4,Watermelon Cooler With Basil Seeds,"Watermelon 1 medium,Basil seeds (takmaria or s...",0.8304053078367941,https://www.sanjeevkapoor.com/Recipe/Watermelo...


In [978]:
# Model 13(2) + False tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1986226641.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.8968146240044993,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Rose Watermelon Smoothie,"Rose syrup 4 tablespoons,Watermelon , peeled, ...",0.7898983641835544,https://www.sanjeevkapoor.com/Recipe/Rose-Wate...
2,Watermelon Slush,"Watermelon juice, put in an ice tray and froze...",0.7649189967829227,https://www.sanjeevkapoor.com/Recipe/Watermelo...
3,Watermelon Cooler With Basil Seeds,"Watermelon 1 medium,Basil seeds (takmaria or s...",0.7517307122577437,https://www.sanjeevkapoor.com/Recipe/Watermelo...
4,Fruit Ball Float,"Watermelon 1 small,Musk melon 1 small,Orange j...",0.7372406540142393,https://www.sanjeevkapoor.com/Recipe/Fruit-Bal...


In [990]:
# Model 13(2) + True tests
input = "watermelon"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4233235582.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Watermelon And Rose Smoothie,"Medium watermelon peeled, cut into medium cube...",0.8891874483479867,https://www.sanjeevkapoor.com/Recipe/Watermelo...
1,Watermelon Cooler With Basil Seeds,"Watermelon 1 medium,Basil seeds (takmaria or s...",0.8482813656368502,https://www.sanjeevkapoor.com/Recipe/Watermelo...
2,Watermelon Slush,"Watermelon juice, put in an ice tray and froze...",0.8217735040276385,https://www.sanjeevkapoor.com/Recipe/Watermelo...
3,Stewed Watermelon,"Watermelon peeled, seeded and cut into cubes 1...",0.7888880378518008,https://www.sanjeevkapoor.com/Recipe/Stewed-Wa...
4,Muskmelon Slush,"Medium watermelon peeled, seeded and cut into ...",0.7869567268652211,https://www.sanjeevkapoor.com/Recipe/Muskmelon...


### tomato sauce, pasta, zucchini, onion, cheese

In [632]:
# Model 1 + False tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2863015847.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Lightning Lasagne,"Lasagne sheets 6-8,Tomato sauce to taste,Green...",0.8415128421445492,https://www.sanjeevkapoor.com/Recipe/Lightning...
1,Italian Thin Crust Pizza,"Refined flour 450 gms + for dusting,Tomato sau...",0.7970602179282756,https://www.sanjeevkapoor.com/Recipe/Italian-T...
2,Deep Pan Pizza,"Refined flour 1 1/4 cups,Tomato sauce 1/2 cup,...",0.7723468528791446,https://www.sanjeevkapoor.com/Recipe/Deep-Pan-...
3,Vegetable Lasagna,"Lasagna Sheets 6,White Sauce to taste,White sa...",0.7673981975808093,https://www.sanjeevkapoor.com/Recipe/Vegetable...
4,Baked Vegetable Cannelloni,Green zucchini cut into batons with skin 1 sma...,0.7664015208186863,https://www.sanjeevkapoor.com/Recipe/Baked-Veg...


In [643]:
# Model 1 + True tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Vegetable Lasagna,"Lasagna Sheets 6,White Sauce to taste,White sa...",0.8206307188933262,https://www.sanjeevkapoor.com/Recipe/Vegetable...
1,Roasted Vegetable Lasagne,"Mixed Vegetables 4 sheets,Lasagne Sheets 6-7,M...",0.7839803494414336,https://www.sanjeevkapoor.com/Recipe/Roasted-V...
2,Corn Aubergine Bake,"Corn kernels boiled 1 cup,Aubergines/brinjals ...",0.7812956231863569,https://www.sanjeevkapoor.com/Recipe/Corn-Aube...
3,Lightning Lasagne,"Lasagne sheets 6-8,Tomato sauce to taste,Green...",0.7809717383125436,https://www.sanjeevkapoor.com/Recipe/Lightning...
4,Wholemeal Fresh Dough Thin Crust Pizza,"Wholewheat flour 1 1/2 cups,Wheat bran 3 table...",0.77562340369064,https://www.sanjeevkapoor.com/Recipe/Wholemeal...


In [814]:
# Model 7(1) + True tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url
0,Roasted Vegetable Lasagne,"Mixed Vegetables 4 sheets,Lasagne Sheets 6-7,M...",0.8943361523547702,https://www.sanjeevkapoor.com/Recipe/Roasted-V...
1,Vegetable Lasagna,"Lasagna Sheets 6,White Sauce to taste,White sa...",0.8923199104525711,https://www.sanjeevkapoor.com/Recipe/Vegetable...
2,Middle East Wraps,"Rotis 4,Zucchini 1 1/2 teaspoons,Yogurt 1/2 cu...",0.8703831637512981,https://www.sanjeevkapoor.com/Recipe/Middle-Ea...
3,Papdi Pizza,"Crisp puris (papdi) 20,Onion 1 large,Pizza sau...",0.8699079456384686,https://www.sanjeevkapoor.com/Recipe/Papdi-Piz...
4,Baked Vegetable Cannelloni,Green zucchini cut into batons with skin 1 sma...,0.8698810409845645,https://www.sanjeevkapoor.com/Recipe/Baked-Veg...


In [831]:
# Model 2 + False tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1633375022.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Veg Sliders,Red kidney beans (rajma) soaked overnight and ...,0.9974787595055777,https://www.sanjeevkapoor.com/Recipe/Veg-Slide...
1,Penne With Creamy Spinach And Corn,"Penne pasta boiled 3 cups,White sauce 2 cups,M...",0.9974630804055257,https://www.sanjeevkapoor.com/Recipe/Penne-wit...
2,Cheese French Fries,"Parmesan cheese powder 1 cup,Medium potatoes w...",0.9974275473242656,https://www.sanjeevkapoor.com/Recipe/Cheese-Fr...
3,Smooth Pizza,"Refined Flour,Wheat Bran 1 1/2 cups,Wheat bran...",0.9973532540250447,https://www.sanjeevkapoor.com/Recipe/Smooth-Pi...
4,Multigrain Honey Mustard Chicken Sandwich,"Multigrain bread 8 slices,Chicken boiled and s...",0.9973015386276705,https://www.sanjeevkapoor.com/Recipe/Multigrai...


In [843]:
# Model 2 + True tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1992341500.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Wholemeal Fresh Dough Thin Crust Pizza,"Wholewheat flour 1 1/2 cups,Wheat bran 3 table...",0.9985251507656523,https://www.sanjeevkapoor.com/Recipe/Wholemeal...
1,Baked Cheesy Potato Shells,"Processed cheese grated 1/2 cup,Baby potatoes ...",0.9982359841327128,https://www.sanjeevkapoor.com/Recipe/Baked-Che...
2,Hot Dog,"Hot dog rolls 4,Chicken sausages 4,Butter 2 ta...",0.9978818733326846,https://www.sanjeevkapoor.com/Recipe/Hot-Dog.html
3,Egg Burger,"Eggs 4,Burger Bun 4,Three coloured capsicums c...",0.9978787530633337,https://www.sanjeevkapoor.com/Recipe/Egg-Burge...
4,Cajun Fish Burger,"Basa fillets cut into 4 inch pieces 2,Burger b...",0.9978368520239158,https://www.sanjeevkapoor.com/Recipe/Cajun-Fis...


In [856]:
# Model 8(2) + False tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3447942711.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Baked Vegetable Cannelloni,Green zucchini cut into batons with skin 1 sma...,0.9275469130017491,https://www.sanjeevkapoor.com/Recipe/Baked-Veg...
1,Ratatouille,"Long brinjals 2 medium,Tomato concasse 3 table...",0.9223096021078312,https://www.sanjeevkapoor.com/Recipe/Ratatouil...
2,Pizza Bowl - Sk Khazana,"Sourdough bread 1,Cheese slices 10-12,Pizza sa...",0.9125521378473056,https://www.sanjeevkapoor.com/Recipe/Pizza-Bow...
3,Lightning Lasagne,"Lasagne sheets 6-8,Tomato sauce to taste,Green...",0.9106608838361456,https://www.sanjeevkapoor.com/Recipe/Lightning...
4,Smooth Pizza,"Refined Flour,Wheat Bran 1 1/2 cups,Wheat bran...",0.9069239960160799,https://www.sanjeevkapoor.com/Recipe/Smooth-Pi...


In [870]:
# Model 8(2) + True tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4078852487.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Baked Vegetable Cannelloni,Green zucchini cut into batons with skin 1 sma...,0.9328343782618667,https://www.sanjeevkapoor.com/Recipe/Baked-Veg...
1,Ratatouille,"Long brinjals 2 medium,Tomato concasse 3 table...",0.9313157311306215,https://www.sanjeevkapoor.com/Recipe/Ratatouil...
2,Desi Lasagne,"Wholewheat Flour rotis 3,Soya Granules soaked ...",0.9239421945840306,https://www.sanjeevkapoor.com/Recipe/Desi-Lasa...
3,Pizza Bowl - Sk Khazana,"Sourdough bread 1,Cheese slices 10-12,Pizza sa...",0.9162587116356647,https://www.sanjeevkapoor.com/Recipe/Pizza-Bow...
4,Soya Granule Lasagne,"Lasagne 9 sheets,Soya granules 1 cup,Olive oil...",0.9159245236420476,https://www.sanjeevkapoor.com/Recipe/Soya-Gran...


In [882]:
# Model 9(2) + False tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2934798296.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Baked Vegetable Cannelloni,Green zucchini cut into batons with skin 1 sma...,0.9108835703197149,https://www.sanjeevkapoor.com/Recipe/Baked-Veg...
1,Lightning Lasagne,"Lasagne sheets 6-8,Tomato sauce to taste,Green...",0.8967120651003011,https://www.sanjeevkapoor.com/Recipe/Lightning...
2,Smooth Pizza,"Refined Flour,Wheat Bran 1 1/2 cups,Wheat bran...",0.8753160363094354,https://www.sanjeevkapoor.com/Recipe/Smooth-Pi...
3,Makki Ki Roti Sarson Ka Saag Pizza,"Makki ki roti 4,Sarson ka saag 2 cups,Tomatoes...",0.8698910409397669,https://www.sanjeevkapoor.com/Recipe/Makki-Ki-...
4,Desi Lasagne,"Wholewheat Flour rotis 3,Soya Granules soaked ...",0.8671493257055072,https://www.sanjeevkapoor.com/Recipe/Desi-Lasa...


In [894]:
# Model 9(2) + True tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2669468391.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Baked Vegetable Cannelloni,Green zucchini cut into batons with skin 1 sma...,0.8977361400083441,https://www.sanjeevkapoor.com/Recipe/Baked-Veg...
1,Makki Ki Roti Sarson Ka Saag Pizza,"Makki ki roti 4,Sarson ka saag 2 cups,Tomatoes...",0.8898048563508018,https://www.sanjeevkapoor.com/Recipe/Makki-Ki-...
2,Vegetable Lasagna,"Lasagna Sheets 6,White Sauce to taste,White sa...",0.8867579140443508,https://www.sanjeevkapoor.com/Recipe/Vegetable...
3,Desi Lasagne,"Wholewheat Flour rotis 3,Soya Granules soaked ...",0.8816991759070998,https://www.sanjeevkapoor.com/Recipe/Desi-Lasa...
4,Lightning Lasagne,"Lasagne sheets 6-8,Tomato sauce to taste,Green...",0.8715394799980369,https://www.sanjeevkapoor.com/Recipe/Lightning...


In [906]:
# Model 10(2) + False tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2013326127.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


,recipe,ingredients,score,url
0,Lightning Lasagne,"Lasagne sheets 6-8,Tomato sauce to taste,Green...",0.8597809027950434,https://www.sanjeevkapoor.com/Recipe/Lightning...
1,Wholesome Spaghetti,"Boneless Chicken 1 inch pieces 250 grams,Whole...",0.8417886180445097,https://www.sanjeevkapoor.com/Recipe/Wholesome...
2,Baked Vegetable Cannelloni,Green zucchini cut into batons with skin 1 sma...,0.8396171345189929,https://www.sanjeevkapoor.com/Recipe/Baked-Veg...
3,Pizza Quesadillas,"Tortillas readymade 4,Pizza sauce 1 tablespoon...",0.8310648574340735,https://www.sanjeevkapoor.com/Recipe/Pizza-Que...
4,Pizza Cones,"Pizza sauce as required,Dough readymade. for p...",0.8303336955848897,https://www.sanjeevkapoor.com/Recipe/Pizza-Con...


In [918]:
# Model 10(2) + True tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4131289077.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Burritos With Salsa Fresca,"Refried beans 1/2 cup,Cheese grated 1/2 cup,On...",0.85425228200179,https://www.sanjeevkapoor.com/Recipe/Burritos-...
1,Vegetable Lasagna,"Lasagna Sheets 6,White Sauce to taste,White sa...",0.8354506728346675,https://www.sanjeevkapoor.com/Recipe/Vegetable...
2,Desi Lasagne,"Wholewheat Flour rotis 3,Soya Granules soaked ...",0.8330279910799501,https://www.sanjeevkapoor.com/Recipe/Desi-Lasa...
3,Pizza Base Roll,"Pizza base 2,Onions,Oil 2 tablespoons,Onions c...",0.8297716465132288,https://www.sanjeevkapoor.com/Recipe/Pizza-Bas...
4,Creole Mushroom Rice,"Button mushrooms 1 cup,Rice 2 cups,Oil 1 table...",0.8277438582588685,https://www.sanjeevkapoor.com/Recipe/Creole-Mu...


In [930]:
# Model 11(2) + False tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\535754922.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Baked Vegetable Cannelloni,Green zucchini cut into batons with skin 1 sma...,0.9383429846056366,https://www.sanjeevkapoor.com/Recipe/Baked-Veg...
1,Wholemeal Fresh Dough Thin Crust Pizza,"Wholewheat flour 1 1/2 cups,Wheat bran 3 table...",0.9278849149580923,https://www.sanjeevkapoor.com/Recipe/Wholemeal...
2,Roasted Vegetable Lasagne,"Mixed Vegetables 4 sheets,Lasagne Sheets 6-7,M...",0.9248856867901369,https://www.sanjeevkapoor.com/Recipe/Roasted-V...
3,Ratatouille,"Long brinjals 2 medium,Tomato concasse 3 table...",0.9224127167297814,https://www.sanjeevkapoor.com/Recipe/Ratatouil...
4,Lightning Lasagne,"Lasagne sheets 6-8,Tomato sauce to taste,Green...",0.9201173683804433,https://www.sanjeevkapoor.com/Recipe/Lightning...


In [942]:
# Model 11(2) + True tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3489482770.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Wholemeal Fresh Dough Thin Crust Pizza,"Wholewheat flour 1 1/2 cups,Wheat bran 3 table...",0.9343524955285062,https://www.sanjeevkapoor.com/Recipe/Wholemeal...
1,Roasted Vegetable Lasagne,"Mixed Vegetables 4 sheets,Lasagne Sheets 6-7,M...",0.9265652965932879,https://www.sanjeevkapoor.com/Recipe/Roasted-V...
2,Baked Vegetable Cannelloni,Green zucchini cut into batons with skin 1 sma...,0.9230436558641696,https://www.sanjeevkapoor.com/Recipe/Baked-Veg...
3,Middle East Wraps,"Rotis 4,Zucchini 1 1/2 teaspoons,Yogurt 1/2 cu...",0.9184820195115365,https://www.sanjeevkapoor.com/Recipe/Middle-Ea...
4,Vegetable Lasagna,"Lasagna Sheets 6,White Sauce to taste,White sa...",0.9179285408276582,https://www.sanjeevkapoor.com/Recipe/Vegetable...


In [955]:
# Model 12(2) + False tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4082322462.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Lightning Lasagne,"Lasagne sheets 6-8,Tomato sauce to taste,Green...",0.872165714610947,https://www.sanjeevkapoor.com/Recipe/Lightning...
1,Baked Vegetable Cannelloni,Green zucchini cut into batons with skin 1 sma...,0.8681270119577649,https://www.sanjeevkapoor.com/Recipe/Baked-Veg...
2,Vegetable Lasagna,"Lasagna Sheets 6,White Sauce to taste,White sa...",0.8401753244085706,https://www.sanjeevkapoor.com/Recipe/Vegetable...
3,Minestrone,"Macaroni 2 tablespoons,Tomato puree 4 1/2 cups...",0.8357458464791376,https://www.sanjeevkapoor.com/Recipe/Minestron...
4,Italian Thin Crust Pizza,"Refined flour 450 gms + for dusting,Tomato sau...",0.8335961054828382,https://www.sanjeevkapoor.com/Recipe/Italian-T...


In [967]:
# Model 12(2) + True tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2426173970.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Vegetable Lasagna,"Lasagna Sheets 6,White Sauce to taste,White sa...",0.862777077220725,https://www.sanjeevkapoor.com/Recipe/Vegetable...
1,Spaghetti With Baked Beans -Sk Khazana,"Spaghetti cooked 400 grams,Baked beans 1 1/2 c...",0.8532203518439672,https://www.sanjeevkapoor.com/Recipe/Spaghetti...
2,Ratatouille,"Eggplants long variety 3 medium,Zucchini 1 1/2...",0.8450821110489027,https://www.sanjeevkapoor.com/Recipe/Ratatouil...
3,Wholemeal Fresh Dough Thin Crust Pizza,"Wholewheat flour 1 1/2 cups,Wheat bran 3 table...",0.841311972392464,https://www.sanjeevkapoor.com/Recipe/Wholemeal...
4,Baked Vegetable Cannelloni,Green zucchini cut into batons with skin 1 sma...,0.8380630020969997,https://www.sanjeevkapoor.com/Recipe/Baked-Veg...


In [979]:
# Model 13(2) + False tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1986226641.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Lightning Lasagne,"Lasagne sheets 6-8,Tomato sauce to taste,Green...",0.7965373419889806,https://www.sanjeevkapoor.com/Recipe/Lightning...
1,Deep Pan Pizza,"Refined flour 1 1/4 cups,Tomato sauce 1/2 cup,...",0.7743943298956975,https://www.sanjeevkapoor.com/Recipe/Deep-Pan-...
2,Baked Vegetable Cannelloni,Green zucchini cut into batons with skin 1 sma...,0.7652752483138295,https://www.sanjeevkapoor.com/Recipe/Baked-Veg...
3,Italian Thin Crust Pizza,"Refined flour 450 gms + for dusting,Tomato sau...",0.7651493419553641,https://www.sanjeevkapoor.com/Recipe/Italian-T...
4,Wholemeal Fresh Dough Thin Crust Pizza,"Wholewheat flour 1 1/2 cups,Wheat bran 3 table...",0.7561815122443166,https://www.sanjeevkapoor.com/Recipe/Wholemeal...


In [991]:
# Model 13(2) + True tests
input = "tomato sauce, pasta, zucchini, onion, cheese"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4233235582.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Wholemeal Fresh Dough Thin Crust Pizza,"Wholewheat flour 1 1/2 cups,Wheat bran 3 table...",0.8180270791661969,https://www.sanjeevkapoor.com/Recipe/Wholemeal...
1,Instant Noodles In Tomato Sauce,"Instant Noodles 200 grams,Tomato Concasse 1 1/...",0.7935605967497472,https://www.sanjeevkapoor.com/Recipe/Instant-N...
2,Ratatouille,"Eggplants long variety 3 medium,Zucchini 1 1/2...",0.792922841196908,https://www.sanjeevkapoor.com/Recipe/Ratatouil...
3,Baked Bean Wraps,"Canned Black beans 200 grams,Wholewheat Flour ...",0.7881322114362306,https://www.sanjeevkapoor.com/Recipe/Baked-Bea...
4,Ratatouille,"Long brinjals 2 medium,Tomato concasse 3 table...",0.7880279682468188,https://www.sanjeevkapoor.com/Recipe/Ratatouil...


### Peanut

In [633]:
# Model 1 + False tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2863015847.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Gado Gado,"Vegetables boiled, 1 inch batons 2 medium,Pean...",0.6123077466753061,https://www.sanjeevkapoor.com/Recipe/Gado-Gado...
1,Vada Pav Chutney,"Scraped coconut fresh 1 cup,Dried red chillies...",0.5938792093941047,https://www.sanjeevkapoor.com/Recipe/Vada-Pav-...
2,Jain Sabudana Vada,Sago (sabudana) soaked in 1 cup water for 5-6 ...,0.5921245865276321,https://www.sanjeevkapoor.com/Recipe/Jain-Sabu...
3,Brown Imli Rice,"Tamarind pulp 3 tablespoons,Brown rice soaked ...",0.5897717086357485,https://www.sanjeevkapoor.com/Recipe/Brown-Iml...
4,Farhali Idli,"Samo Rice soaked and drained 1 cup,Sago soaked...",0.5870475253116869,https://www.sanjeevkapoor.com/Recipe/Farhali-I...


In [644]:
# Model 1 + True tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Chatpata Chivda,"Nylon Poha 250 grams,Peanuts,Oil,Curry leaves ...",0.7035777019103345,https://www.sanjeevkapoor.com/Recipe/Chatpata-...
1,Gado Gado,"Vegetables boiled, 1 inch batons 2 medium,Pean...",0.6827887398670567,https://www.sanjeevkapoor.com/Recipe/Gado-Gado...
2,Pastama,"Penne boiled 1 cup,Peanuts,Olive oil 2 tablesp...",0.651354460856366,https://www.sanjeevkapoor.com/Recipe/Pastama-S...
3,Long Beans With Honey Sesame,"String beans (chavli) long beans 250 grams,Hon...",0.6508369598011421,https://www.sanjeevkapoor.com/Recipe/Long-Bean...
4,Soppina Palya,"Chavli Leaves 4 bunches,Peanuts 1 tablespoon,M...",0.6480247058326947,https://www.sanjeevkapoor.com/Recipe/Soppina-P...


In [815]:
# Model 7(1) + True tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url
0,Gado Gado,"Vegetables boiled, 1 inch batons 2 medium,Pean...",0.7733296069058644,https://www.sanjeevkapoor.com/Recipe/Gado-Gado...
1,Chatpata Chivda,"Nylon Poha 250 grams,Peanuts,Oil,Curry leaves ...",0.7605787757911618,https://www.sanjeevkapoor.com/Recipe/Chatpata-...
2,Tilache Laddoo,"Sesame Seeds 250 grams,Jaggery grated 1 cup,Pe...",0.712773936433761,https://www.sanjeevkapoor.com/Recipe/Tilache-L...
3,Baghare Baingan,"Small brinjals (baingan) 250 grams,Medium onio...",0.6690408421706977,https://www.sanjeevkapoor.com/Recipe/Baghare-B...
4,Bharli Vangi,"Medium brinjals 8,Tamarind pulp 1 tablespoon,F...",0.6521482020673951,https://www.sanjeevkapoor.com/Recipe/Bharli-Va...


In [832]:
# Model 2 + False tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1633375022.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Gado Gado,"Vegetables boiled, 1 inch batons 2 medium,Pean...",0.9983402056804908,https://www.sanjeevkapoor.com/Recipe/Gado-Gado...
1,Olive Upma,"Black olives sliced 10-12,Semolina 1 cup,Extra...",0.9971982864932143,https://www.sanjeevkapoor.com/Recipe/Olive-Upm...
2,Mysore Mushroom Masala Dosa - Sk Khazana,"Button mushrooms thickly sliced 12-15,Dosa bat...",0.9971292829758849,https://www.sanjeevkapoor.com/Recipe/Mysore-Mu...
3,Rajgire Ka Thepla,"Rajgira Flour 2 cups + for dusting,Rock salt (...",0.9969983366204265,https://www.sanjeevkapoor.com/Recipe/Rajgire-K...
4,Avial,"Banana peeled and cut into fingers 1,Yam peele...",0.9968586943629534,https://www.sanjeevkapoor.com/Recipe/Avial-Tur...


In [844]:
# Model 2 + True tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1992341500.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Olive Upma,"Black olives sliced 10-12,Semolina 1 cup,Extra...",0.9972533422085803,https://www.sanjeevkapoor.com/Recipe/Olive-Upm...
1,Avial,"Banana peeled and cut into fingers 1,Yam peele...",0.9972494699178607,https://www.sanjeevkapoor.com/Recipe/Avial-Tur...
2,Malabar Chemeen Kari,"Prawns 500 grams,Green mangoes Scraped 1 1/2 c...",0.9970839386011301,https://www.sanjeevkapoor.com/Recipe/Malabar-C...
3,Batata Puri,"Medium potatoes medium, boiled and peeled 4,Sa...",0.9967471985853569,https://www.sanjeevkapoor.com/Recipe/Batata-Pu...
4,Maddur Vade,"Semolina (rawa/suji) 1 cup,Refined flour (maid...",0.9960522176969651,https://www.sanjeevkapoor.com/Recipe/Maddur-Va...


In [857]:
# Model 8(2) + False tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3447942711.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Gado Gado,"Vegetables boiled, 1 inch batons 2 medium,Pean...",0.8599484577300198,https://www.sanjeevkapoor.com/Recipe/Gado-Gado...
1,Soppina Palya,"Green amaranth leaves chopped 4 bunches,Roaste...",0.8396090733179424,https://www.sanjeevkapoor.com/Recipe/Soppina-P...
2,Brown Imli Rice,"Tamarind pulp 3 tablespoons,Brown rice soaked ...",0.8126008045709552,https://www.sanjeevkapoor.com/Recipe/Brown-Iml...
3,Soppina Palya,"Chavli Leaves 4 bunches,Peanuts 1 tablespoon,M...",0.809336083855168,https://www.sanjeevkapoor.com/Recipe/Soppina-P...
4,Thecha - Sk Khazana,"Green chillies slit 7-8,Peanuts roasted 1 tabl...",0.7971325089828187,https://www.sanjeevkapoor.com/Recipe/Thecha---...


In [871]:
# Model 8(2) + True tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4078852487.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Gado Gado,"Vegetables boiled, 1 inch batons 2 medium,Pean...",0.8545175799323963,https://www.sanjeevkapoor.com/Recipe/Gado-Gado...
1,Soppina Palya,"Chavli Leaves 4 bunches,Peanuts 1 tablespoon,M...",0.8327010121184024,https://www.sanjeevkapoor.com/Recipe/Soppina-P...
2,Soppina Palya,"Green amaranth leaves chopped 4 bunches,Roaste...",0.8191934937959774,https://www.sanjeevkapoor.com/Recipe/Soppina-P...
3,Brown Imli Rice,"Tamarind pulp 3 tablespoons,Brown rice soaked ...",0.8134888643563536,https://www.sanjeevkapoor.com/Recipe/Brown-Iml...
4,Chatpata Chivda,"Nylon Poha 250 grams,Peanuts,Oil,Curry leaves ...",0.8117386158264376,https://www.sanjeevkapoor.com/Recipe/Chatpata-...


In [883]:
# Model 9(2) + False tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2934798296.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Chatpata Chivda,"Nylon Poha 250 grams,Peanuts,Oil,Curry leaves ...",0.7199313834042448,https://www.sanjeevkapoor.com/Recipe/Chatpata-...
1,Molaga Podi,"Sesame Seeds 1/2 cup,Dried Red Chillies 4 tabl...",0.7028983753247849,https://www.sanjeevkapoor.com/Recipe/Molaga-Po...
2,Baghare Baingan,"Small brinjals (baingan) 250 grams,Medium onio...",0.7014926933754524,https://www.sanjeevkapoor.com/Recipe/Baghare-B...
3,Mixed Vegetable Tamarind Rice,"Rice soaked 1 1/4 cups,Tamarind pulp 3 tablesp...",0.6930660515399367,https://www.sanjeevkapoor.com/Recipe/Mixed-Veg...
4,Tilache Laddoo,"Sesame Seeds 250 grams,Jaggery grated 1 cup,Pe...",0.6875226728959676,https://www.sanjeevkapoor.com/Recipe/Tilache-L...


In [895]:
# Model 9(2) + True tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2669468391.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Chatpata Chivda,"Nylon Poha 250 grams,Peanuts,Oil,Curry leaves ...",0.7958850793455585,https://www.sanjeevkapoor.com/Recipe/Chatpata-...
1,Tilache Laddoo,"Sesame Seeds 250 grams,Jaggery grated 1 cup,Pe...",0.7676512760669114,https://www.sanjeevkapoor.com/Recipe/Tilache-L...
2,Baghare Baingan,"Small brinjals (baingan) 250 grams,Medium onio...",0.743061938757053,https://www.sanjeevkapoor.com/Recipe/Baghare-B...
3,Mixed Vegetable Tamarind Rice,"Rice soaked 1 1/4 cups,Tamarind pulp 3 tablesp...",0.7189178362337777,https://www.sanjeevkapoor.com/Recipe/Mixed-Veg...
4,Khatta Meetha Chiwda,"Pressed Rice 2 cups,Oil 1 tablespoon,Asafoetid...",0.7148200137312944,https://www.sanjeevkapoor.com/Recipe/Khatta-Me...


In [907]:
# Model 10(2) + False tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2013326127.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Soppina Palya,"Chavli Leaves 4 bunches,Peanuts 1 tablespoon,M...",0.6417674888930873,https://www.sanjeevkapoor.com/Recipe/Soppina-P...
1,Sweet Wantons,"Wonton wrappers 8,Peanuts roasted 1/4 cup,Seed...",0.5845325895760519,https://www.sanjeevkapoor.com/Recipe/Sweet-Wan...
2,Soppina Palya,"Green amaranth leaves chopped 4 bunches,Roaste...",0.578188882317,https://www.sanjeevkapoor.com/Recipe/Soppina-P...
3,Mirchi Ka Salan-Sk Khazana,"Big green chillies slit and deep fried 18-20,O...",0.5766743414547919,https://www.sanjeevkapoor.com/Recipe/Mirchi-ka...
4,Suran Aloo Subzi,"Yam sliced 400 grams,Potatoes boiled 2 large,G...",0.5746285789620807,https://www.sanjeevkapoor.com/Recipe/Suran-Alo...


In [919]:
# Model 10(2) + True tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4131289077.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Soppina Palya,"Chavli Leaves 4 bunches,Peanuts 1 tablespoon,M...",0.7001864366360746,https://www.sanjeevkapoor.com/Recipe/Soppina-P...
1,Chatpata Chivda,"Nylon Poha 250 grams,Peanuts,Oil,Curry leaves ...",0.6841592479463694,https://www.sanjeevkapoor.com/Recipe/Chatpata-...
2,Tilache Laddoo,"Sesame Seeds 250 grams,Jaggery grated 1 cup,Pe...",0.6624747825974969,https://www.sanjeevkapoor.com/Recipe/Tilache-L...
3,Pastama,"Penne boiled 1 cup,Peanuts,Olive oil 2 tablesp...",0.6486006004230244,https://www.sanjeevkapoor.com/Recipe/Pastama-S...
4,Long Beans With Honey Sesame,"String beans (chavli) long beans 250 grams,Hon...",0.6319076034777497,https://www.sanjeevkapoor.com/Recipe/Long-Bean...


In [931]:
# Model 11(2) + False tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\535754922.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Gado Gado,"Vegetables boiled, 1 inch batons 2 medium,Pean...",0.8453211435747907,https://www.sanjeevkapoor.com/Recipe/Gado-Gado...
1,Soppina Palya,"Chavli Leaves 4 bunches,Peanuts 1 tablespoon,M...",0.8046691012138771,https://www.sanjeevkapoor.com/Recipe/Soppina-P...
2,Raw Mango Rice,"Raw mangoes peeled and grated 1 large,Rice coo...",0.7989786206333315,https://www.sanjeevkapoor.com/Recipe/Raw-Mango...
3,Brown Imli Rice,"Tamarind pulp 3 tablespoons,Brown rice soaked ...",0.7989178075206665,https://www.sanjeevkapoor.com/Recipe/Brown-Iml...
4,Soppina Palya,"Green amaranth leaves chopped 4 bunches,Roaste...",0.7910045991158939,https://www.sanjeevkapoor.com/Recipe/Soppina-P...


In [943]:
# Model 11(2) + True tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3489482770.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Gado Gado,"Vegetables boiled, 1 inch batons 2 medium,Pean...",0.8540612163366972,https://www.sanjeevkapoor.com/Recipe/Gado-Gado...
1,Chatpata Chivda,"Nylon Poha 250 grams,Peanuts,Oil,Curry leaves ...",0.827165697893097,https://www.sanjeevkapoor.com/Recipe/Chatpata-...
2,Soppina Palya,"Chavli Leaves 4 bunches,Peanuts 1 tablespoon,M...",0.8246260850959196,https://www.sanjeevkapoor.com/Recipe/Soppina-P...
3,Brown Imli Rice,"Tamarind pulp 3 tablespoons,Brown rice soaked ...",0.8175554680525043,https://www.sanjeevkapoor.com/Recipe/Brown-Iml...
4,Capsicum Kayras,Green capsicums cut into 1/2 inch pieces 5-6 m...,0.8050906575664981,https://www.sanjeevkapoor.com/Recipe/Capsicum-...


In [956]:
# Model 12(2) + False tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4082322462.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Vattal Kozhambu,"Vattal 2 tablespoons,Sesame oil (til oil) 4 ta...",0.670516347985898,https://www.sanjeevkapoor.com/Recipe/Vattal-Ko...
1,Farhali Idli,"Samo Rice soaked and drained 1 cup,Sago soaked...",0.6701049369850524,https://www.sanjeevkapoor.com/Recipe/Farhali-I...
2,Uppu Seedai,"Rice flour 1/2 cup,Split skinless black gram (...",0.6502239172616278,https://www.sanjeevkapoor.com/Recipe/Uppu-Seed...
3,Gado Gado,"Vegetables boiled, 1 inch batons 2 medium,Pean...",0.6455858239288549,https://www.sanjeevkapoor.com/Recipe/Gado-Gado...
4,Soppina Palya,"Green amaranth leaves chopped 4 bunches,Roaste...",0.6424344838115813,https://www.sanjeevkapoor.com/Recipe/Soppina-P...


In [968]:
# Model 12(2) + True tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2426173970.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Chatpata Chivda,"Nylon Poha 250 grams,Peanuts,Oil,Curry leaves ...",0.7325221332247732,https://www.sanjeevkapoor.com/Recipe/Chatpata-...
1,Vattal Kozhambu,"Vattal 2 tablespoons,Sesame oil (til oil) 4 ta...",0.7149202486989079,https://www.sanjeevkapoor.com/Recipe/Vattal-Ko...
2,Tilache Laddoo,"Sesame Seeds 250 grams,Jaggery grated 1 cup,Pe...",0.7068164646207344,https://www.sanjeevkapoor.com/Recipe/Tilache-L...
3,Gado Gado,"Vegetables boiled, 1 inch batons 2 medium,Pean...",0.688649154776267,https://www.sanjeevkapoor.com/Recipe/Gado-Gado...
4,Farhali Idli,"Samo Rice soaked and drained 1 cup,Sago soaked...",0.6874301057939237,https://www.sanjeevkapoor.com/Recipe/Farhali-I...


In [980]:
# Model 13(2) + False tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1986226641.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Chatpata Chivda,"Nylon Poha 250 grams,Peanuts,Oil,Curry leaves ...",0.6255681884840569,https://www.sanjeevkapoor.com/Recipe/Chatpata-...
1,Soppina Palya,"Chavli Leaves 4 bunches,Peanuts 1 tablespoon,M...",0.5862011699890393,https://www.sanjeevkapoor.com/Recipe/Soppina-P...
2,Molaga Podi,"Sesame Seeds 1/2 cup,Dried Red Chillies 4 tabl...",0.575163230757237,https://www.sanjeevkapoor.com/Recipe/Molaga-Po...
3,Five Spice Chicken And Pear Stir Fry,"Five spice powder 1/2 teaspoon,Boneless U.S. C...",0.5690026491373492,https://www.sanjeevkapoor.com/Recipe/Five-Spic...
4,Capsicum Kayras,Green capsicums cut into 1/2 inch pieces 5-6 m...,0.5634806907925225,https://www.sanjeevkapoor.com/Recipe/Capsicum-...


In [992]:
# Model 13(2) + True tests
input = "peanut"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4233235582.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Chatpata Chivda,"Nylon Poha 250 grams,Peanuts,Oil,Curry leaves ...",0.7365920962455864,https://www.sanjeevkapoor.com/Recipe/Chatpata-...
1,Soppina Palya,"Chavli Leaves 4 bunches,Peanuts 1 tablespoon,M...",0.6411835282549301,https://www.sanjeevkapoor.com/Recipe/Soppina-P...
2,Tilache Laddoo,"Sesame Seeds 250 grams,Jaggery grated 1 cup,Pe...",0.6392363197731,https://www.sanjeevkapoor.com/Recipe/Tilache-L...
3,Puliyogare,"Rice 2 cups,Tamarind Paste 1 1/2 tablespoons,M...",0.5919075772044494,https://www.sanjeevkapoor.com/Recipe/Puliyogar...
4,Pastama,"Penne boiled 1 cup,Peanuts,Olive oil 2 tablesp...",0.5893245510445257,https://www.sanjeevkapoor.com/Recipe/Pastama-S...


### Honey, peanut, chocolate

In [634]:
# Model 1 + False tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2863015847.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Lemon Banana Lassi,"Bananas 3 medium,Yogurt 1 1/2 cups,Lemon juice...",0.7027388288568961,https://www.sanjeevkapoor.com/Recipe/Lemon-Ban...
1,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.649955416260654,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...
2,Honey Nut Icecream,"Vanilla Ice Cream 1 litre,Honey 3 tablespoons,...",0.6098980305330413,https://www.sanjeevkapoor.com/Recipe/Honey-Nut...
3,Honey Cherry Energy Bars,"Honey 1/2 cup,Cherries 1/2 cup,Desiccated coco...",0.6052187917669141,https://www.sanjeevkapoor.com/Recipe/Honey-Che...
4,Chocolate And Peanut Butter Smoothie,"Chocolate nibs 1 tablespoon,Peanut butter 1/2 ...",0.604651797223539,https://www.sanjeevkapoor.com/Recipe/Chocolate...


In [645]:
# Model 1 + True tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.7331699897374464,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...
1,Matcha Latte - Sk Khazana,"Matcha powder 4 tablespo to garnish,Milk at ro...",0.6813904087406046,https://www.sanjeevkapoor.com/Recipe/Matcha-La...
2,Oats And Flax Seeds Pops,"Oats roasted 1 cup,Flax seeds 1/2 cup,Peanut...",0.681011428336651,https://www.sanjeevkapoor.com/Recipe/Oats-and-...
3,Honey Nut Icecream,"Vanilla Ice Cream 1 litre,Honey 3 tablespoons,...",0.6792946528371568,https://www.sanjeevkapoor.com/Recipe/Honey-Nut...
4,Lemon Banana Lassi,"Bananas 3 medium,Yogurt 1 1/2 cups,Lemon juice...",0.6741917009162086,https://www.sanjeevkapoor.com/Recipe/Lemon-Ban...


In [816]:
# Model 7(1) + True tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url
0,Matcha Latte - Sk Khazana,"Matcha powder 4 tablespo to garnish,Milk at ro...",0.8056593347654427,https://www.sanjeevkapoor.com/Recipe/Matcha-La...
1,Chilled Green Tea And Honey Drink,"Green tea powder 1 gram,Honey 2 teaspoon",0.7905213186166418,https://www.sanjeevkapoor.com/Recipe/Chilled-G...
2,Daarsaan,"Flat noodles 200 grams,Oil to deep fry,Honey 1...",0.7889138805969008,https://www.sanjeevkapoor.com/Recipe/Daarsaan-...
3,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.7373145441986745,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...
4,Oats And Flax Seeds Pops,"Oats roasted 1 cup,Flax seeds 1/2 cup,Peanut...",0.7241123251011004,https://www.sanjeevkapoor.com/Recipe/Oats-and-...


In [833]:
# Model 2 + False tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

# not good

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1633375022.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Milkmaid Darsaan,"Condensed Milk 1/4 cup (100 gms),Noodles boile...",0.9983025464064303,https://www.sanjeevkapoor.com/Recipe/Milkmaid-...
1,Honey Nut Icecream,"Vanilla Ice Cream 1 litre,Honey 3 tablespoons,...",0.9982821535512723,https://www.sanjeevkapoor.com/Recipe/Honey-Nut...
2,Candied Pumpkin Seeds,"Sugar 1 cup,Pumpkin seeds 2 cup,Sesame seeds (...",0.9978308106586131,https://www.sanjeevkapoor.com/Recipe/Candied-P...
3,Sesame Crackers,"White sesame seeds 1/4 cup,Pumpkin seeds 1/4 c...",0.9976048111462856,https://www.sanjeevkapoor.com/Recipe/Sesame-Cr...
4,Mango And Chilli Sorbet,"Mango puree 2 1/2 cups,Birds eye chillies 1 t...",0.997450855187234,https://www.sanjeevkapoor.com/Recipe/Mango-and...


In [845]:
# Model 2 + True tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1992341500.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Berry Berry Beautiful,"Orange juice 600 millilitres,Strawberries 8 la...",0.999140166501249,https://www.sanjeevkapoor.com/Recipe/Berry-Ber...
1,Milkmaid Darsaan,"Condensed Milk 1/4 cup (100 gms),Noodles boile...",0.9986091869625122,https://www.sanjeevkapoor.com/Recipe/Milkmaid-...
2,Candied Pumpkin Seeds,"Sugar 1 cup,Pumpkin seeds 2 cup,Sesame seeds (...",0.9984205163849329,https://www.sanjeevkapoor.com/Recipe/Candied-P...
3,Sweet Rice,"Rice cooked 1 cup,Sugar 1/4 cup,Unsalted butte...",0.998158994946077,https://www.sanjeevkapoor.com/Recipe/Sweet-Ric...
4,Honey Nut Icecream,"Vanilla Ice Cream 1 litre,Honey 3 tablespoons,...",0.9980986184862195,https://www.sanjeevkapoor.com/Recipe/Honey-Nut...


In [858]:
# Model 8(2) + False tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

# not good

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3447942711.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.8681329840502294,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...
1,Chilled Green Tea And Honey Drink,"Green tea powder 1 gram,Honey 2 teaspoon",0.8384791309473173,https://www.sanjeevkapoor.com/Recipe/Chilled-G...
2,Lemon Banana Lassi,"Bananas 3 medium,Yogurt 1 1/2 cups,Lemon juice...",0.8328164990138538,https://www.sanjeevkapoor.com/Recipe/Lemon-Ban...
3,Breakfast Triffle,"Yogurt 1 1/2 cups,Muesli 2 cups,Honey 4 tables...",0.8226273962919357,https://www.sanjeevkapoor.com/Recipe/Breakfast...
4,Chocolate And Peanut Butter Smoothie,"Chocolate nibs 1 tablespoon,Peanut butter 1/2 ...",0.8142945235182613,https://www.sanjeevkapoor.com/Recipe/Chocolate...


In [872]:
# Model 8(2) + True tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4078852487.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.8916098324259158,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...
1,Honey Nut Icecream,"Vanilla Ice Cream 1 litre,Honey 3 tablespoons,...",0.8774155983711013,https://www.sanjeevkapoor.com/Recipe/Honey-Nut...
2,Chilled Green Tea And Honey Drink,"Green tea powder 1 gram,Honey 2 teaspoon",0.8738947476364874,https://www.sanjeevkapoor.com/Recipe/Chilled-G...
3,Matcha Latte - Sk Khazana,"Matcha powder 4 tablespo to garnish,Milk at ro...",0.8640263099578256,https://www.sanjeevkapoor.com/Recipe/Matcha-La...
4,Chocolate And Peanut Butter Smoothie,"Chocolate nibs 1 tablespoon,Peanut butter 1/2 ...",0.8607252349341776,https://www.sanjeevkapoor.com/Recipe/Chocolate...


In [884]:
# Model 9(2) + False tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

# not good

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2934798296.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.7259116320812462,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...
1,Spiced Stuffed Apples,"Medium apples 4,Seedless Dates chopped 1/2 cup...",0.7121957622593826,https://www.sanjeevkapoor.com/Recipe/Spiced-St...
2,Honey Nut Icecream,"Vanilla Ice Cream 1 litre,Honey 3 tablespoons,...",0.7077129384569625,https://www.sanjeevkapoor.com/Recipe/Honey-Nut...
3,Daarsaan,"Flat noodles 200 grams,Oil to deep fry,Honey 1...",0.7067069162382426,https://www.sanjeevkapoor.com/Recipe/Daarsaan-...
4,Matcha Latte - Sk Khazana,"Matcha powder 4 tablespo to garnish,Milk at ro...",0.7054649852447624,https://www.sanjeevkapoor.com/Recipe/Matcha-La...


In [896]:
# Model 9(2) + True tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2669468391.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.8043583331062261,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...
1,Matcha Latte - Sk Khazana,"Matcha powder 4 tablespo to garnish,Milk at ro...",0.7930632269784484,https://www.sanjeevkapoor.com/Recipe/Matcha-La...
2,Peanut Butter Chocolate Brownie Icecream - Coo...,"Peanut butter 2 tablespoons,Chocolate brownies...",0.7895579244201916,https://www.sanjeevkapoor.com/Recipe/Peanut-Bu...
3,Honey Nut Icecream,"Vanilla Ice Cream 1 litre,Honey 3 tablespoons,...",0.7761195083584048,https://www.sanjeevkapoor.com/Recipe/Honey-Nut...
4,Daarsaan,"Flat noodles 200 grams,Oil to deep fry,Honey 1...",0.7751652453049515,https://www.sanjeevkapoor.com/Recipe/Daarsaan-...


In [908]:
# Model 10(2) + False tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

# not good

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2013326127.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.8200513733235855,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...
1,Oats And Flax Seeds Pops,"Oats roasted 1 cup,Flax seeds 1/2 cup,Peanut...",0.730721175740211,https://www.sanjeevkapoor.com/Recipe/Oats-and-...
2,Walnut Oatmeal Laddoos - Sk Khazana,"California walnuts 20-25,Oats roasted 3/4 cup,...",0.7107033662047644,https://www.sanjeevkapoor.com/Recipe/Walnut-Oa...
3,Pineapple Power,"Pineapple Juice canned 4 cups,Fresh Pineapple ...",0.7106031866747697,https://www.sanjeevkapoor.com/Recipe/Pineapple...
4,Peanut Raisin Bar,"Roasted Peanuts roasted and crushed 1 cup,Seed...",0.6893658479561136,https://www.sanjeevkapoor.com/Recipe/Peanut-Ra...


In [920]:
# Model 10(2) + True tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4131289077.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.842056570877894,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...
1,Oats And Flax Seeds Pops,"Oats roasted 1 cup,Flax seeds 1/2 cup,Peanut...",0.8004621040285208,https://www.sanjeevkapoor.com/Recipe/Oats-and-...
2,Walnut Oatmeal Laddoos - Sk Khazana,"California walnuts 20-25,Oats roasted 3/4 cup,...",0.753509925967329,https://www.sanjeevkapoor.com/Recipe/Walnut-Oa...
3,Peanut Raisin Bar,"Roasted Peanuts roasted and crushed 1 cup,Seed...",0.7459282341939879,https://www.sanjeevkapoor.com/Recipe/Peanut-Ra...
4,Pineapple Power,"Pineapple Juice canned 4 cups,Fresh Pineapple ...",0.7344579340855839,https://www.sanjeevkapoor.com/Recipe/Pineapple...


In [932]:
# Model 11(2) + False tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

# not good

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\535754922.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.8647207471199042,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...
1,Honey Nut Icecream,"Vanilla Ice Cream 1 litre,Honey 3 tablespoons,...",0.829913318164378,https://www.sanjeevkapoor.com/Recipe/Honey-Nut...
2,Honey Flapjacks,"Honey 200 grams,Demerara sugar 200 grams,Porri...",0.8177339217236076,https://www.sanjeevkapoor.com/Recipe/Honey-Fla...
3,Daarsaan,"Flat noodles 200 grams,Oil to deep fry,Honey 1...",0.808308842840006,https://www.sanjeevkapoor.com/Recipe/Daarsaan-...
4,Homemade Snack Bars,Sunflower seeds (charoli/chironji) roasted 15 ...,0.8008507008229384,https://www.sanjeevkapoor.com/Recipe/Homemade-...


In [944]:
# Model 11(2) + True tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3489482770.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Honey Nut Icecream,"Vanilla Ice Cream 1 litre,Honey 3 tablespoons,...",0.8847090374695759,https://www.sanjeevkapoor.com/Recipe/Honey-Nut...
1,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.8841556572588876,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...
2,Honey Flapjacks,"Honey 200 grams,Demerara sugar 200 grams,Porri...",0.8607724327710375,https://www.sanjeevkapoor.com/Recipe/Honey-Fla...
3,Homemade Snack Bars,Sunflower seeds (charoli/chironji) roasted 15 ...,0.8374383465951807,https://www.sanjeevkapoor.com/Recipe/Homemade-...
4,Daarsaan,"Flat noodles 200 grams,Oil to deep fry,Honey 1...",0.8334491075518408,https://www.sanjeevkapoor.com/Recipe/Daarsaan-...


In [957]:
# Model 12(2) + False tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

# not good

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4082322462.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Lemon Banana Lassi,"Bananas 3 medium,Yogurt 1 1/2 cups,Lemon juice...",0.7439613144219885,https://www.sanjeevkapoor.com/Recipe/Lemon-Ban...
1,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.7352269839577967,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...
2,Honey Nut Icecream,"Vanilla Ice Cream 1 litre,Honey 3 tablespoons,...",0.7217142082970609,https://www.sanjeevkapoor.com/Recipe/Honey-Nut...
3,5 Seeds Chocolate Magas,"Melon seeds (magaz) roasted 1 tablespoon,black...",0.6945092722846351,https://www.sanjeevkapoor.com/Recipe/5-Seeds-C...
4,Caramel Banana With White And Black Sesame Seeds,"Ripe bananas 4-5,Sugar 1 cup,White sesame seed...",0.694448342377924,https://www.sanjeevkapoor.com/Recipe/Caramel--...


In [969]:
# Model 12(2) + True tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2426173970.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.8035447390441799,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...
1,Honey Nut Icecream,"Vanilla Ice Cream 1 litre,Honey 3 tablespoons,...",0.7910856748550975,https://www.sanjeevkapoor.com/Recipe/Honey-Nut...
2,Caramel Banana With White And Black Sesame Seeds,"Ripe bananas 4-5,Sugar 1 cup,White sesame seed...",0.7633666852983787,https://www.sanjeevkapoor.com/Recipe/Caramel--...
3,Matcha Latte - Sk Khazana,"Matcha powder 4 tablespo to garnish,Milk at ro...",0.7430640077679955,https://www.sanjeevkapoor.com/Recipe/Matcha-La...
4,Peanut Butter Chocolate Brownie Icecream - Coo...,"Peanut butter 2 tablespoons,Chocolate brownies...",0.7386388052958406,https://www.sanjeevkapoor.com/Recipe/Peanut-Bu...


In [981]:
# Model 13(2) + False tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

# not good

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1986226641.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Matcha Latte - Sk Khazana,"Matcha powder 4 tablespo to garnish,Milk at ro...",0.625253015952554,https://www.sanjeevkapoor.com/Recipe/Matcha-La...
1,Chocolate And Peanut Butter Smoothie,"Chocolate nibs 1 tablespoon,Peanut butter 1/2 ...",0.6198494512192143,https://www.sanjeevkapoor.com/Recipe/Chocolate...
2,Chocolate And Peanut Butter Smoothie,"Chocolate nibs 1 tablespoon,Cocoa powder 1/2 t...",0.5968085427634304,https://www.sanjeevkapoor.com/Recipe/Chocolate...
3,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.5921383000856484,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...
4,Honey Nut Icecream,"Vanilla Ice Cream 1 litre,Honey 3 tablespoons,...",0.5847912616162729,https://www.sanjeevkapoor.com/Recipe/Honey-Nut...


In [993]:
# Model 13(2) + True tests
input = "honey, peanut, chocolate"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4233235582.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Matcha Latte - Sk Khazana,"Matcha powder 4 tablespo to garnish,Milk at ro...",0.7094957592969318,https://www.sanjeevkapoor.com/Recipe/Matcha-La...
1,Peanut Butter Chocolate Brownie Icecream - Coo...,"Peanut butter 2 tablespoons,Chocolate brownies...",0.671243046383879,https://www.sanjeevkapoor.com/Recipe/Peanut-Bu...
2,Peanut Butter,"Peanuts 2 cups,Oil 2 teaspoons,Sea salt to tas...",0.671146974013991,https://www.sanjeevkapoor.com/Recipe/Peanut-Bu...
3,Honey Nut Icecream,"Vanilla Ice Cream 1 litre,Honey 3 tablespoons,...",0.6603954427377818,https://www.sanjeevkapoor.com/Recipe/Honey-Nut...
4,Chewy Oat Cookies,"Oats rolled 2 3/4 cups,Butter 3/4 cup,Casto...",0.6602213701284422,https://www.sanjeevkapoor.com/Recipe/Chewy-Oat...


### Onion, Tomato, Carrot

In [996]:
# Model 1 + True tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\452073778.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.25)


,recipe,ingredients,score,url
0,Hearty Egg Bites,"Eggs hardboiled 2,Oil 1 tablespo for shallow f...",0.7718478760293228,https://www.sanjeevkapoor.com/Recipe/Hearty-Eg...
1,Classic Roast Chicken,"Chicken 800 grams,Basmati rice cooked 2 cups,O...",0.762170647292737,https://www.sanjeevkapoor.com/Recipe/Classic-R...
2,Paushtik Tomato Soup,"Tomatoes roughly chopped 6,Carrots 2 teaspoons...",0.7510044826232358,https://www.sanjeevkapoor.com/Recipe/Paushtik-...
3,Fried Onions For Biryani,"Onions 3 medium,Oil to deep fry",0.7464450305797005,https://www.sanjeevkapoor.com/Recipe/Fried-Oni...
4,Carrot Puree (For Babies Between 4-6 Months),Carrot 1 medium,0.746444988067183,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...


In [998]:
# Model 7(1) + True tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\81244826.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=100, alpha=0.2)


,recipe,ingredients,score,url
0,Hearty Egg Bites,"Eggs hardboiled 2,Oil 1 tablespo for shallow f...",0.7938617369322567,https://www.sanjeevkapoor.com/Recipe/Hearty-Eg...
1,Beetroot Soup,"Beetroots, thinly sliced 3 medium,Oil 2 tables...",0.7862706229643741,https://www.sanjeevkapoor.com/Recipe/Beetroot-...
2,Classic Roast Chicken,"Chicken 800 grams,Basmati rice cooked 2 cups,O...",0.7787463131410763,https://www.sanjeevkapoor.com/Recipe/Classic-R...
3,Paushtik Tomato Soup,"Tomatoes roughly chopped 6,Carrots 2 teaspoons...",0.7673847328861519,https://www.sanjeevkapoor.com/Recipe/Paushtik-...
4,Sabz Makai Kali Mirch Ka Shorba,"Carrots seeded, 1/2 cm cubes 1/2 medium,Sweet ...",0.7619592031458751,https://www.sanjeevkapoor.com/Recipe/Sabz-Maka...


In [834]:
# Model 2 + False tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1633375022.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Thukpa-Sk Khazana,"Medium carrot 1,Button mushrooms 5,Medium cabb...",0.9967962256303711,https://www.sanjeevkapoor.com/Recipe/Thukpa-SK...
1,Tomato Carrot And Coconut Milk Soup,"Tomatoes cut into cubes 5-6 medium,Carrots pee...",0.9950863689494533,https://www.sanjeevkapoor.com/Recipe/Tomato-Ca...
2,Thai Paneer Green Curry,"Cottage Cheese 450 grams,Basil Leaves 2 stalks...",0.9937740908147961,https://www.sanjeevkapoor.com/Recipe/Thai-Pane...
3,Curried Chicken Rice,"Curry powder 1 tablespoon,Boneless chicken cut...",0.9936774812558825,https://www.sanjeevkapoor.com/Recipe/Curried-C...
4,Chunky Masala Potatoes,"Potatoes 6 large,Olive oil 2 tablespoons,Cumin...",0.9935016768841283,https://www.sanjeevkapoor.com/Recipe/Chunky-Ma...


In [846]:
# Model 2 + True tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1992341500.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.025)


,recipe,ingredients,score,url
0,Thukpa-Sk Khazana,"Medium carrot 1,Button mushrooms 5,Medium cabb...",0.9984892023430978,https://www.sanjeevkapoor.com/Recipe/Thukpa-SK...
1,Lemon And Coriander Soup,"Lemon juice 2 tablespoons,Fresh coriander leav...",0.9963048898180096,https://www.sanjeevkapoor.com/Recipe/Lemon-And...
2,Oven Baked Fish With Spicy Harissa,"Pomfret 2,Yellow capsicum seeded and chopped 1...",0.9962438397707281,https://www.sanjeevkapoor.com/Recipe/Oven-Bake...
3,Chicken Stuffed Chilli,"Chicken mince 1 cup,Green chillies slit and se...",0.9962211361785717,https://www.sanjeevkapoor.com/Recipe/Chicken-S...
4,Lentil And Coriander Soup,"Split Red Lentil 3/4 cup,Fresh Coriander Leave...",0.9961103865136296,https://www.sanjeevkapoor.com/Recipe/Lentil-An...


In [859]:
# Model 8(2) + False tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3447942711.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Carrot Puree (For Babies Between 4-6 Months),Carrot 1 medium,0.9031392055765368,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...
1,Carrot Puree,Carrot 1 medium,0.9031392055765368,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...
2,Carrot And Tomato Smoothie,"Carrots 4 medium,Tomatoes 6 medium,Celery chop...",0.7929336099836553,https://www.sanjeevkapoor.com/Recipe/Carrot-An...
3,Mixed Vegetable Idli,"Parboiled Rice 1 cup,Carrot 1/2 cup,Carrot gra...",0.7152913161572494,https://www.sanjeevkapoor.com/Recipe/Mixed-Veg...
4,Hearty Egg Bites,"Eggs hardboiled 2,Oil 1 tablespo for shallow f...",0.7099231957940357,https://www.sanjeevkapoor.com/Recipe/Hearty-Eg...


In [873]:
# Model 8(2) + True tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4078852487.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.15)


,recipe,ingredients,score,url
0,Hearty Egg Bites,"Eggs hardboiled 2,Oil 1 tablespo for shallow f...",0.8391593056291694,https://www.sanjeevkapoor.com/Recipe/Hearty-Eg...
1,Paushtik Tomato Soup,"Tomatoes roughly chopped 6,Carrots 2 teaspoons...",0.793376067771532,https://www.sanjeevkapoor.com/Recipe/Paushtik-...
2,Sabz Makai Kali Mirch Ka Shorba,"Carrots seeded, 1/2 cm cubes 1/2 medium,Sweet ...",0.7805726911830804,https://www.sanjeevkapoor.com/Recipe/Sabz-Maka...
3,Beetroot Soup,"Beetroots, thinly sliced 3 medium,Oil 2 tables...",0.7595458965306553,https://www.sanjeevkapoor.com/Recipe/Beetroot-...
4,Cabbage Soup,"Cabbage diced 1 medium,Onions,Onion diced 2 me...",0.7566328354621981,https://www.sanjeevkapoor.com/Recipe/Cabbage-S...


In [885]:
# Model 9(2) + False tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2934798296.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Carrot Puree (For Babies Between 4-6 Months),Carrot 1 medium,0.8926042922173345,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...
1,Carrot Puree,Carrot 1 medium,0.8926042922173345,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...
2,Carrot And Tomato Smoothie,"Carrots 4 medium,Tomatoes 6 medium,Celery chop...",0.6938373999348028,https://www.sanjeevkapoor.com/Recipe/Carrot-An...
3,Beetroot Soup,"Beetroots, thinly sliced 3 medium,Oil 2 tables...",0.6712316078739613,https://www.sanjeevkapoor.com/Recipe/Beetroot-...
4,Schezwan Pulao,"Basmati Rice soaked 1 1/2 cups,Schezwan Chutne...",0.6678030268437245,https://www.sanjeevkapoor.com/Recipe/Schezwan-...


In [897]:
# Model 9(2) + True tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2669468391.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.2)


,recipe,ingredients,score,url
0,Beetroot Soup,"Beetroots, thinly sliced 3 medium,Oil 2 tables...",0.8053331849527845,https://www.sanjeevkapoor.com/Recipe/Beetroot-...
1,Paushtik Tomato Soup,"Tomatoes roughly chopped 6,Carrots 2 teaspoons...",0.7862108346726537,https://www.sanjeevkapoor.com/Recipe/Paushtik-...
2,Hearty Egg Bites,"Eggs hardboiled 2,Oil 1 tablespo for shallow f...",0.7796823929830292,https://www.sanjeevkapoor.com/Recipe/Hearty-Eg...
3,Classic Roast Chicken,"Chicken 800 grams,Basmati rice cooked 2 cups,O...",0.754043883747018,https://www.sanjeevkapoor.com/Recipe/Classic-R...
4,Carrot And Coriander Soup,"Carrots chopped 650 grams,Fresh coriander leav...",0.7490156797303482,https://www.sanjeevkapoor.com/Recipe/Carrot-An...


In [909]:
# Model 10(2) + False tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2013326127.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Carrot Puree (For Babies Between 4-6 Months),Carrot 1 medium,0.8819736919318881,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...
1,Carrot Puree,Carrot 1 medium,0.8819736919318881,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...
2,Paushtik Tomato Soup,"Tomatoes roughly chopped 6,Carrots 2 teaspoons...",0.6719136118811788,https://www.sanjeevkapoor.com/Recipe/Paushtik-...
3,Classic Roast Chicken,"Chicken 800 grams,Basmati rice cooked 2 cups,O...",0.6666492859827567,https://www.sanjeevkapoor.com/Recipe/Classic-R...
4,Carrot And Tomato Smoothie,"Carrots 4 medium,Tomatoes 6 medium,Celery chop...",0.6550809626314398,https://www.sanjeevkapoor.com/Recipe/Carrot-An...


In [921]:
# Model 10(2) + True tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4131289077.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=50, alpha=0.25)


,recipe,ingredients,score,url
0,Paushtik Tomato Soup,"Tomatoes roughly chopped 6,Carrots 2 teaspoons...",0.7902889730757178,https://www.sanjeevkapoor.com/Recipe/Paushtik-...
1,Hearty Egg Bites,"Eggs hardboiled 2,Oil 1 tablespo for shallow f...",0.7821707791822515,https://www.sanjeevkapoor.com/Recipe/Hearty-Eg...
2,Classic Roast Chicken,"Chicken 800 grams,Basmati rice cooked 2 cups,O...",0.7781644538286152,https://www.sanjeevkapoor.com/Recipe/Classic-R...
3,Mixed Vegetable Soup,"Carrot,Milk 2 tablespoons,Bay leaf 1,Onion 1 m...",0.7413491367084345,https://www.sanjeevkapoor.com/Recipe/Mixed-Veg...
4,Chicken Masala Noodles,Boneless chicken cut into small pieces 100 gra...,0.736565274459535,https://www.sanjeevkapoor.com/Recipe/Chicken-M...


In [933]:
# Model 11(2) + False tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\535754922.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Carrot Puree (For Babies Between 4-6 Months),Carrot 1 medium,0.8950695629194904,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...
1,Carrot Puree,Carrot 1 medium,0.8950695629194904,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...
2,Carrot And Tomato Smoothie,"Carrots 4 medium,Tomatoes 6 medium,Celery chop...",0.7653987346508992,https://www.sanjeevkapoor.com/Recipe/Carrot-An...
3,Classic Roast Chicken,"Chicken 800 grams,Basmati rice cooked 2 cups,O...",0.7128196323344551,https://www.sanjeevkapoor.com/Recipe/Classic-R...
4,Sabz Makai Kali Mirch Ka Shorba,"Carrots seeded, 1/2 cm cubes 1/2 medium,Sweet ...",0.7099150031788999,https://www.sanjeevkapoor.com/Recipe/Sabz-Maka...


In [945]:
# Model 11(2) + True tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\3489482770.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.15)


,recipe,ingredients,score,url
0,Hearty Egg Bites,"Eggs hardboiled 2,Oil 1 tablespo for shallow f...",0.8265974672585296,https://www.sanjeevkapoor.com/Recipe/Hearty-Eg...
1,Classic Roast Chicken,"Chicken 800 grams,Basmati rice cooked 2 cups,O...",0.7978030277031547,https://www.sanjeevkapoor.com/Recipe/Classic-R...
2,Paushtik Tomato Soup,"Tomatoes roughly chopped 6,Carrots 2 teaspoons...",0.7764154665271246,https://www.sanjeevkapoor.com/Recipe/Paushtik-...
3,Beetroot Soup,"Beetroots, thinly sliced 3 medium,Oil 2 tables...",0.7763310567429237,https://www.sanjeevkapoor.com/Recipe/Beetroot-...
4,Sabz Makai Kali Mirch Ka Shorba,"Carrots seeded, 1/2 cm cubes 1/2 medium,Sweet ...",0.7761680058655858,https://www.sanjeevkapoor.com/Recipe/Sabz-Maka...


In [958]:
# Model 12(2) + False tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4082322462.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Carrot Puree (For Babies Between 4-6 Months),Carrot 1 medium,0.8832992789023959,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...
1,Carrot Puree,Carrot 1 medium,0.8832992789023959,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...
2,Carrot And Tomato Smoothie,"Carrots 4 medium,Tomatoes 6 medium,Celery chop...",0.7125235590690451,https://www.sanjeevkapoor.com/Recipe/Carrot-An...
3,Classic Roast Chicken,"Chicken 800 grams,Basmati rice cooked 2 cups,O...",0.6321164962221408,https://www.sanjeevkapoor.com/Recipe/Classic-R...
4,Mixed Vegetable Idli,"Parboiled Rice 1 cup,Carrot 1/2 cup,Carrot gra...",0.6192559005686562,https://www.sanjeevkapoor.com/Recipe/Mixed-Veg...


In [970]:
# Model 12(2) + True tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\2426173970.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.2)


,recipe,ingredients,score,url
0,Classic Roast Chicken,"Chicken 800 grams,Basmati rice cooked 2 cups,O...",0.7722011897342087,https://www.sanjeevkapoor.com/Recipe/Classic-R...
1,Paushtik Tomato Soup,"Tomatoes roughly chopped 6,Carrots 2 teaspoons...",0.7546537558498614,https://www.sanjeevkapoor.com/Recipe/Paushtik-...
2,Hearty Egg Bites,"Eggs hardboiled 2,Oil 1 tablespo for shallow f...",0.7455922657013325,https://www.sanjeevkapoor.com/Recipe/Hearty-Eg...
3,Sabz Makai Kali Mirch Ka Shorba,"Carrots seeded, 1/2 cm cubes 1/2 medium,Sweet ...",0.7232209814714649,https://www.sanjeevkapoor.com/Recipe/Sabz-Maka...
4,Carrot And Mushroom Medley,"Carrots cut into roundels 2,Button mushrooms c...",0.721939643487524,https://www.sanjeevkapoor.com/Recipe/Carrot-An...


In [982]:
# Model 13(2) + False tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\1986226641.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Carrot Puree (For Babies Between 4-6 Months),Carrot 1 medium,0.8992927452284204,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...
1,Carrot Puree,Carrot 1 medium,0.8992927452284204,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...
2,Carrot And Tomato Smoothie,"Carrots 4 medium,Tomatoes 6 medium,Celery chop...",0.669713613616574,https://www.sanjeevkapoor.com/Recipe/Carrot-An...
3,Mixed Vegetable Idli,"Parboiled Rice 1 cup,Carrot 1/2 cup,Carrot gra...",0.6197094992349996,https://www.sanjeevkapoor.com/Recipe/Mixed-Veg...
4,Sabz Makai Kali Mirch Ka Shorba,"Carrots seeded, 1/2 cm cubes 1/2 medium,Sweet ...",0.5942528921727447,https://www.sanjeevkapoor.com/Recipe/Sabz-Maka...


In [994]:
# Model 13(2) + True tests
input = "onion, tomato, carrot"
rec = get_recs(input)
rec

C:\Users\12242\AppData\Local\Temp\ipykernel_36584\4233235582.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


Successfully loaded model
Word2Vec(vocab=7019, vector_size=150, alpha=0.25)


,recipe,ingredients,score,url
0,Classic Roast Chicken,"Chicken 800 grams,Basmati rice cooked 2 cups,O...",0.7576301160346057,https://www.sanjeevkapoor.com/Recipe/Classic-R...
1,Hearty Egg Bites,"Eggs hardboiled 2,Oil 1 tablespo for shallow f...",0.7448234989613464,https://www.sanjeevkapoor.com/Recipe/Hearty-Eg...
2,Fried Onions For Biryani,"Onions 3 medium,Oil to deep fry",0.742508697232988,https://www.sanjeevkapoor.com/Recipe/Fried-Oni...
3,Carrot Puree (For Babies Between 4-6 Months),Carrot 1 medium,0.7425086460278092,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...
4,Carrot Puree,Carrot 1 medium,0.7425086460278092,https://www.sanjeevkapoor.com/Recipe/Carrot-Pu...


# Streamlit

In [1050]:
import streamlit as st
import pandas as pd
import numpy as np
# from streamlit import SessionState
import SessionState
import os
from PIL import Image

# import config, rec_sys
# from ingredient_parser import ingredient_parser

# from word2vec_rec import get_recs

import nltk

ModuleNotFoundError: No module named 'SessionState'

In [1033]:
pip install --upgrade streamlit

In [1027]:
pip install streamlit

  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=471009df25c9c389c115fca13683a28007100bdef3462844c09e5cd8011d8711
  Stored in directory: c:\users\12242\appdata\local\pip\cache\wheels\2d\f0\a8\1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built validators


In [1041]:
import streamlit as st
st.write(st.__version__)

2022-08-03 22:26:58.867 
  command:

    streamlit run C:\Users\12242\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [ ]:
try:
    nltk.data.find("wordnet")
except LookupError:
    nltk.download("wordnet")


def make_clickable(name, link):
    # target _blank to open new window
    # extract clickable text to display for your link
    text = name
    return f'<a target="_blank" href="{link}">{text}</a>'


def main():
    image = Image.open("input/wordcloud.png").resize((680, 150))
    st.image(image)
    st.markdown("# *What's Cooking? :cooking:*")

    st.markdown(
        "An ML powered app by Jack Leitch <a href='https://github.com/jackmleitch/whatscooking-deployment' > <img src='https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/Octicons-mark-github.svg/600px-Octicons-mark-github.svg.png' width='20' height='20' > </a> ",
        unsafe_allow_html=True,
    )
    st.markdown(
        "## Given a list of ingredients, what different recipes can I can make? :tomato: "
    )
    st.markdown(
        "For example, say I want to use up some food in my apartment, what can I cook? :house: My ML based model will look through over 4500 recipes to find matches for you... :mag: Try it out for yourself below! :arrow_down:"
    )

    st.text("")

    session_state = SessionState.get(
        recipe_df="",
        recipes="",
        model_computed=False,
        execute_recsys=False,
        recipe_df_clean="",
    )

    ingredients = st.text_input(
        "Enter ingredients you would like to cook with (seperate ingredients with a comma)",
        "onion, chorizo, chicken thighs, paella rice, frozen peas, prawns",
    )
    session_state.execute_recsys = st.button("Give me recommendations!")

    if session_state.execute_recsys:

        col1, col2, col3 = st.beta_columns([1, 6, 1])
        with col2:
            gif_runner = st.image("input/cooking_gif.gif")
        # recipe = rec_sys.RecSys(ingredients)
        recipe = get_recs(ingredients, mean=True)
        gif_runner.empty()
        session_state.recipe_df_clean = recipe.copy()
        # link is the column with hyperlinks
        recipe["url"] = recipe.apply(
            lambda row: make_clickable(row["recipe"], row["url"]), axis=1
        )
        recipe_display = recipe[["recipe", "url", "ingredients"]]
        session_state.recipe_display = recipe_display.to_html(escape=False)
        session_state.recipes = recipe.recipe.values.tolist()
        session_state.model_computed = True
        session_state.execute_recsys = False

    if session_state.model_computed:
        # st.write("Either pick a particular recipe or see the top 5 recommendations.")
        recipe_all_box = st.selectbox(
            "Either see the top 5 recommendations or pick a particular recipe ya fancy",
            ["Show me them all!", "Select a single recipe"],
        )
        if recipe_all_box == "Show me them all!":
            st.write(session_state.recipe_display, unsafe_allow_html=True)
        else:
            selection = st.selectbox(
                "Select a delicious recipe", options=session_state.recipes
            )
            selection_details = session_state.recipe_df_clean.loc[
                session_state.recipe_df_clean.recipe == selection
            ]
            st.markdown(f"# {selection_details.recipe.values[0]}")
            st.subheader(f"Website: {selection_details.url.values[0]}")
            ingredients_disp = selection_details.ingredients.values[0].split(",")

            st.subheader("Ingredients:")
            col1, col2 = st.beta_columns(2)
            ingredients_disp = [
                ingred
                for ingred in ingredients_disp
                if ingred
                not in [
                    " skin off",
                    " bone out",
                    " from sustainable sources",
                    " minced",
                ]
            ]
            ingredients_disp1 = ingredients_disp[len(ingredients_disp) // 2 :]
            ingredients_disp2 = ingredients_disp[: len(ingredients_disp) // 2]
            for ingred in ingredients_disp1:
                col1.markdown(f"* {ingred}")
            for ingred in ingredients_disp2:
                col2.markdown(f"* {ingred}")
            # st.write(f"Score: {selection_details.score.values[0]}")

    # sidebar stuff
    with st.sidebar.beta_expander("How it works?", expanded=True):
        st.markdown("## How it works? :thought_balloon:")
        st.write(
            "For an in depth overview of the ML methods used and how I created this app, three blog posts are below."
        )
        blog1 = "https://jackmleitch.medium.com/using-beautifulsoup-to-help-make-beautiful-soups-d2670a1d1d52"
        blog2 = "https://towardsdatascience.com/building-a-recipe-recommendation-api-using-scikit-learn-nltk-docker-flask-and-heroku-bfc6c4bdd2d4"
        blog3 = "https://towardsdatascience.com/building-a-recipe-recommendation-system-297c229dda7b"
        st.markdown(
            f"1. [Web Scraping Cooking Data With Beautiful Soup]({blog1})"
        )
        st.markdown(
            f"2. [Building a Recipe Recommendation API using Scikit-Learn, NLTK, Docker, Flask, and Heroku]({blog2})"
        )
        st.markdown(
            f"3. [Building a Recipe Recommendation System Using Word2Vec, Scikit-Learn, and Streamlit]({blog3})"
        )


if __name__ == "__main__":
    main()

# Streamlit RA

In [1052]:
import streamlit as st